In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

#https://github.com/mwaskom/seaborn/issues/1009

In [2]:
import os
print (os.path.abspath(os.curdir))

H:\DC\个人征信\code


# 训练集:

In [110]:
训练放款时间表 = pd.read_csv("../train/loan_time_train.txt",header=None,names=['用户标识','放款时间'])
训练放款时间表['放款时间']=训练放款时间表['放款时间']//86400

训练用户表 = pd.read_csv("../train/user_info_train.txt",header=None,
                    names=['用户标识','用户性别','用户职业','用户教育程度',
                           '用户婚姻状态', '用户户口类型'])

训练信用卡账单表=pd.read_csv("../train/bill_detail_train.txt",header=None,
                    names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态'])
训练信用卡账单表['时间']=训练信用卡账单表['时间']//86400

训练信用卡账单表 = pd.merge(训练信用卡账单表, 训练放款时间表,how='inner', on = "用户标识")

用户浏览行为 = pd.read_csv("../train/browse_history_train.txt",header=None,
                    names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
用户浏览行为['浏览时间']=用户浏览行为['浏览时间']//86400
用户浏览行为= pd.merge(用户浏览行为, 训练放款时间表,how='inner', on = "用户标识")

银行流水记录=pd.read_csv("../train/bank_detail_train.txt").rename(index=str,
                                                         columns={"uid": "用户标识","timespan": "流水时间",
                                                                  "type":"交易类型","amount":"交易金额","markup":"工资收入标记"})    
银行流水记录['流水时间']=银行流水记录['流水时间']//86400

训练表 = pd.read_csv("../train/overdue_train.txt",header=None,
                    names=['用户标识','标签'])

训练表 = pd.merge(训练表,训练用户表,how='inner',on = "用户标识")
训练表 = pd.merge(训练表,训练放款时间表,how='inner',on = "用户标识")

In [145]:
训练表 = pd.read_csv("../train/overdue_train.txt",header=None,
                    names=['用户标识','标签'])

训练表 = pd.merge(训练表,训练用户表,how='inner',on = "用户标识")
训练表.to_csv("../feature/训练表_20170119_A.csv",index=None,encoding="gb2312")

In [3]:
训练放款时间表 = pd.read_csv("../train/loan_time_train.txt",header=None,names=['用户标识','放款时间'])
训练放款时间表['放款时间']=训练放款时间表['放款时间']//86400

测试放款时间表 = pd.read_csv("../test/loan_time_test.txt",header=None,names=['用户标识','放款时间'])
测试放款时间表['放款时间']=测试放款时间表['放款时间']//86400

In [4]:
训练信用卡账单表=pd.read_csv("../train/bill_detail_train.txt",header=None,
                    names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态'])
训练信用卡账单表['时间']=训练信用卡账单表['时间']//86400

训练信用卡账单表 = pd.merge(训练信用卡账单表, 训练放款时间表,how='inner', on = "用户标识")

# **训练集特征构造:**

训练信用卡账单表补充特征：

In [8]:
d=训练信用卡账单表[(训练信用卡账单表['时间']>0)]

,用户标识,时间,银行标识,上期账单金额,上期还款金额,信用卡额度,本期账单余额,本期账单最低还款额,消费笔数,本期账单金额,调整金额,循环利息,可用余额,预借现金额度,还款状态,放款时间
0,3150,68365,6,18.626118,18.661937,20.664418,18.905766,17.847133,1,0.000000,0.000000,0.00000,0.0,19.971271,0,68516
1,3150,68365,6,18.905766,18.909954,20.664418,19.113305,17.911506,1,0.000000,0.000000,0.00000,0.0,19.971271,0,68516
2,3150,68365,6,19.113305,19.150290,20.664418,19.300194,17.977610,1,0.000000,0.000000,0.00000,0.0,19.971271,0,68516
3,3150,68365,6,19.300194,19.300280,21.000890,20.303240,18.477177,1,0.000000,0.000000,0.00000,0.0,20.307743,0,68516
4,3150,68365,6,20.303240,20.307744,21.000890,20.357134,18.510985,1,0.000000,0.000000,0.00000,0.0,20.307743,0,68516
5,3150,68365,6,19.668245,19.722809,20.664418,19.799044,18.194951,1,0.000000,0.000000,0.00000,0.0,19.971271,0,68516
6,3150,68365,6,20.106043,20.153592,21.069883,20.242691,18.906474,2,0.000000,0.000000,0.00000,0.0,20.376736,0,68516
7,3150,68590,11,0.000000,0.000000,0.000000,16.851878,16.207495,0,0.000000,0.000000,0.00000,0.0,0.000000,0,68516
8,3150,68590,16,19.237914,19.420133,19.460445,-16.292183,14.327077,1,17.322724,0.000000,0.00000,0.0,19.460445,0,68516
9,3150,68590,14,19.754017,19.355084,21.580708,19.971764,19.420654,0,19.976445,18.659787,0.00000,0.0,21.580708,0,68516


# 补充特征：共55596 rows × 466 columns
**放款前：**

In [11]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=训练信用卡账单表[(训练信用卡账单表['时间']>0)]
feature=训练放款时间表
###补充特征，增加 放款前的各种统计信息sum count max min mean std var等
gb=d[(d['时间']<=d['放款时间'])].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

放款前账单sum=gb.sum()
放款前账单sum.columns = ['用户标识', '放款前上期账单金额sum', '放款前上期还款金额sum','放款前信用卡额度sum','放款前本期账单余额sum',
                     '放款前本期账单最低还款额sum','放款前消费笔数sum','放款前本期账单金额sum','放款前调整金额sum','放款前循环利息sum',
                     '放款前可用余额sum','放款前预借现金额度sum','放款前还款状态sum']
feature=pd.merge(feature, 放款前账单sum,how='left', on = "用户标识")
feature['放款前上期还款金额sum与放款前上期账单金额sum差值']=feature['放款前上期还款金额sum']-feature['放款前上期账单金额sum']
feature['放款前信用卡额度sum与放款前本期账单余额sum差值']=feature['放款前信用卡额度sum']-feature['放款前本期账单余额sum']
feature['放款前可用余额sum与放款前预借现金额度sum差值']=feature['放款前可用余额sum']-feature['放款前预借现金额度sum']
feature['放款前本期账单最低还款额sum与放款前循环利息sum之和']=feature['放款前本期账单最低还款额sum']+feature['放款前循环利息sum']

放款前账单count=gb.count()
放款前账单count.columns = ['用户标识', '放款前上期账单金额count', '放款前上期还款金额count','放款前信用卡额度count','放款前本期账单余额count',
                     '放款前本期账单最低还款额count','放款前消费笔数count','放款前本期账单金额count','放款前调整金额count','放款前循环利息count',
                     '放款前可用余额count','放款前预借现金额度count','放款前还款状态count']
feature=pd.merge(feature, 放款前账单count,how='left', on = "用户标识")

放款前账单max=gb.max()
放款前账单max.columns = ['用户标识', '放款前上期账单金额max', '放款前上期还款金额max','放款前信用卡额度max','放款前本期账单余额max',
                     '放款前本期账单最低还款额max','放款前消费笔数max','放款前本期账单金额max','放款前调整金额max','放款前循环利息max',
                     '放款前可用余额max','放款前预借现金额度max','放款前还款状态max']
feature=pd.merge(feature, 放款前账单max,how='left', on = "用户标识")
feature['放款前上期还款金额max与放款前上期账单金额max差值']=feature['放款前上期还款金额max']-feature['放款前上期账单金额max']
feature['放款前信用卡额度max与放款前本期账单余额max差值']=feature['放款前信用卡额度max']-feature['放款前本期账单余额max']
feature['放款前可用余额max与放款前预借现金额度max差值']=feature['放款前可用余额max']-feature['放款前预借现金额度max']
feature['放款前本期账单最低还款额max与放款前循环利息max之和']=feature['放款前本期账单最低还款额max']+feature['放款前循环利息max']

放款前账单min=gb.min()
放款前账单min.columns = ['用户标识', '放款前上期账单金额min', '放款前上期还款金额min','放款前信用卡额度min','放款前本期账单余额min',
                     '放款前本期账单最低还款额min','放款前消费笔数min','放款前本期账单金额min','放款前调整金额min','放款前循环利息min',
                     '放款前可用余额min','放款前预借现金额度min','放款前还款状态min']
feature=pd.merge(feature, 放款前账单min,how='left', on = "用户标识")
feature['放款前上期还款金额min与放款前上期账单金额min差值']=feature['放款前上期还款金额min']-feature['放款前上期账单金额min']
feature['放款前信用卡额度min与放款前本期账单余额min差值']=feature['放款前信用卡额度min']-feature['放款前本期账单余额min']
feature['放款前可用余额min与放款前预借现金额度min差值']=feature['放款前可用余额min']-feature['放款前预借现金额度min']
feature['放款前本期账单最低还款额min与放款前循环利息min之和']=feature['放款前本期账单最低还款额min']+feature['放款前循环利息min']

放款前账单mean=gb.mean()
放款前账单mean.columns = ['用户标识', '放款前上期账单金额mean', '放款前上期还款金额mean','放款前信用卡额度mean','放款前本期账单余额mean',
                     '放款前本期账单最低还款额mean','放款前消费笔数mean','放款前本期账单金额mean','放款前调整金额mean','放款前循环利息mean',
                     '放款前可用余额mean','放款前预借现金额度mean','放款前还款状态mean']
feature=pd.merge(feature, 放款前账单mean,how='left', on = "用户标识")
feature['放款前上期还款金额mean与放款前上期账单金额mean差值']=feature['放款前上期还款金额mean']-feature['放款前上期账单金额mean']
feature['放款前信用卡额度mean与放款前本期账单余额mean差值']=feature['放款前信用卡额度mean']-feature['放款前本期账单余额mean']
feature['放款前可用余额mean与放款前预借现金额度mean差值']=feature['放款前可用余额mean']-feature['放款前预借现金额度mean']
feature['放款前本期账单最低还款额mean与放款前循环利息mean之和']=feature['放款前本期账单最低还款额mean']+feature['放款前循环利息mean']

放款前账单median=gb.median()
放款前账单median.columns = ['用户标识', '放款前上期账单金额median', '放款前上期还款金额median','放款前信用卡额度median','放款前本期账单余额median',
                  '放款前本期账单最低还款额median','放款前消费笔数median','放款前本期账单金额median','放款前调整金额median',
                  '放款前循环利息median','放款前可用余额median','放款前预借现金额度median','放款前还款状态median']
feature=pd.merge(feature, 放款前账单median,how='left', on = "用户标识")
feature['放款前上期还款金额median与放款前上期账单金额median差值']=feature['放款前上期还款金额median']-feature['放款前上期账单金额median']
feature['放款前信用卡额度median与放款前本期账单余额median差值']=feature['放款前信用卡额度median']-feature['放款前本期账单余额median']
feature['放款前可用余额median与放款前预借现金额度median差值']=feature['放款前可用余额median']-feature['放款前预借现金额度median']
feature['放款前本期账单最低还款额median与放款前循环利息median之和']=feature['放款前本期账单最低还款额median']+feature['放款前循环利息median']

放款前账单std=gb.std()
放款前账单std.columns = ['用户标识', '放款前上期账单金额std', '放款前上期还款金额std','放款前信用卡额度std','放款前本期账单余额std',
                     '放款前本期账单最低还款额std','放款前消费笔数std','放款前本期账单金额std','放款前调整金额std','放款前循环利息std',
                     '放款前可用余额std','放款前预借现金额度std','放款前还款状态std']
feature=pd.merge(feature, 放款前账单std,how='left', on = "用户标识")
feature['放款前上期还款金额std与放款前上期账单金额std差值']=feature['放款前上期还款金额std']-feature['放款前上期账单金额std']
feature['放款前信用卡额度std与放款前本期账单余额std差值']=feature['放款前信用卡额度std']-feature['放款前本期账单余额std']
feature['放款前可用余额std与放款前预借现金额度std差值']=feature['放款前可用余额std']-feature['放款前预借现金额度std']
feature['放款前本期账单最低还款额std与放款前循环利息std之和']=feature['放款前本期账单最低还款额std']+feature['放款前循环利息std']

放款前账单var=gb.var()
放款前账单var.columns = ['用户标识', '放款前上期账单金额var', '放款前上期还款金额var','放款前信用卡额度var','放款前本期账单余额var',
                     '放款前本期账单最低还款额var','放款前消费笔数var','放款前本期账单金额var','放款前调整金额var','放款前循环利息var',
                     '放款前可用余额var','放款前预借现金额度var','放款前还款状态var']
feature=pd.merge(feature, 放款前账单var,how='left', on = "用户标识")
feature['放款前上期还款金额var与放款前上期账单金额var差值']=feature['放款前上期还款金额var']-feature['放款前上期账单金额var']
feature['放款前信用卡额度var与放款前本期账单余额var差值']=feature['放款前信用卡额度var']-feature['放款前本期账单余额var']
feature['放款前可用余额var与放款前预借现金额度var差值']=feature['放款前可用余额var']-feature['放款前预借现金额度var']
feature['放款前本期账单最低还款额var与放款前循环利息var之和']=feature['放款前本期账单最低还款额var']+feature['放款前循环利息var']

In [12]:
#统计行和列的缺失值数
#feature1=feature.fillna(-1)
#print((feature1==-1).sum(axis=0))
#print((feature1==-1).sum(axis=1))
#feature#55596 rows × 118 columns
feature#55596 rows × 126 columns 2017.02.03

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,放款前本期账单金额var,放款前调整金额var,放款前循环利息var,放款前可用余额var,放款前预借现金额度var,放款前还款状态var,放款前上期还款金额var与放款前上期账单金额var差值,放款前信用卡额度var与放款前本期账单余额var差值,放款前可用余额var与放款前预借现金额度var差值,放款前本期账单最低还款额var与放款前循环利息var之和
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,136.760811,135.406663,139.365030,176.806257,88.329982,24.0,136.072147,0.000000,...,75.719994,0.000000,0.000000,0.000000,46.134597,0.0,-1.898401,74.765489,-46.134597,233.662905
2,3,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,144.813292,105.148045,338.422883,372.743693,346.818948,2.0,158.766677,0.000000,...,82.491690,0.000000,0.000000,31.789011,91.328029,0.0,-15.183258,51.495389,-59.539018,1.164896
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,296.822014,289.286843,348.958827,297.547743,291.448253,56.0,214.111454,0.000000,...,92.718104,0.000000,0.000000,122.560890,87.826081,0.0,-42.255465,-84.266517,34.734809,20.074344
6,7,68458,113.912497,499.590074,545.060225,44.272435,247.793789,45.0,372.624755,0.000000,...,85.820508,0.000000,0.000000,74.773468,74.967120,0.0,-260.603474,-355.137684,-0.193652,81.503294
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,68458,203.990621,277.888959,286.764829,244.116769,238.446171,1.0,186.410678,0.000000,...,106.325812,0.000000,0.000000,0.000000,96.144117,0.0,-198.432034,-109.449238,-96.144117,25.099748
9,10,68458,1229.021694,1323.621599,1774.741754,1444.023419,497.256464,73.0,1494.956298,0.000000,...,37.548389,0.000000,0.000000,11.467087,86.114235,0.0,-38.842224,-64.015214,-74.647148,240.041562


**去重：**

In [13]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#放款前账单去重统计特征
#d=训练信用卡账单表[(训练信用卡账单表['时间']>0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d[(d['时间']<=d['放款时间'])].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后放款前账单sum=gb.sum()
去重后放款前账单sum.columns = ['用户标识', '去重后放款前上期账单金额sum', '去重后放款前上期还款金额sum','去重后放款前信用卡额度sum'
                       ,'去重后放款前本期账单余额sum','去重后放款前本期账单最低还款额sum','去重后放款前消费笔数sum'
                       ,'去重后放款前本期账单金额sum','去重后放款前调整金额sum','去重后放款前循环利息sum','去重后放款前可用余额sum'
                       ,'去重后放款前预借现金额度sum']
feature=pd.merge(feature, 去重后放款前账单sum,how='left', on = "用户标识")
feature['去重后放款前上期还款金额sum与放款前上期账单金额sum差值']=feature['去重后放款前上期还款金额sum']-feature['去重后放款前上期账单金额sum']
feature['去重后放款前信用卡额度sum与放款前本期账单余额sum差值']=feature['去重后放款前信用卡额度sum']-feature['去重后放款前本期账单余额sum']
feature['去重后放款前可用余额sum与放款前预借现金额度sum差值']=feature['去重后放款前可用余额sum']-feature['去重后放款前预借现金额度sum']
feature['去重后放款前本期账单最低还款额sum与放款前循环利息sum之和']=feature['去重后放款前本期账单最低还款额sum']+feature['去重后放款前循环利息sum']

去重后放款前账单count=gb.count()
去重后放款前账单count.columns = ['用户标识', '去重后放款前上期账单金额count', '去重后放款前上期还款金额count','去重后放款前信用卡额度count'
                         ,'去重后放款前本期账单余额count','去重后放款前本期账单最低还款额count','去重后放款前消费笔数count'
                         ,'去重后放款前本期账单金额count','去重后放款前调整金额count','去重后放款前循环利息count'
                         ,'去重后放款前可用余额count','去重后放款前预借现金额度count']
feature=pd.merge(feature, 去重后放款前账单count,how='left', on = "用户标识")

去重后放款前账单max=gb.max()
去重后放款前账单max.columns = ['用户标识', '去重后放款前上期账单金额max', '去重后放款前上期还款金额max','去重后放款前信用卡额度max'
                       ,'去重后放款前本期账单余额max','去重后放款前本期账单最低还款额max','去重后放款前消费笔数max'
                       ,'去重后放款前本期账单金额max','去重后放款前调整金额max','去重后放款前循环利息max'
                       ,'去重后放款前可用余额max','去重后放款前预借现金额度max']
feature=pd.merge(feature, 去重后放款前账单max,how='left', on = "用户标识")
feature['去重后放款前上期还款金额max与放款前上期账单金额max差值']=feature['去重后放款前上期还款金额max']-feature['去重后放款前上期账单金额max']
feature['去重后放款前信用卡额度max与放款前本期账单余额max差值']=feature['去重后放款前信用卡额度max']-feature['去重后放款前本期账单余额max']
feature['去重后放款前可用余额max与放款前预借现金额度max差值']=feature['去重后放款前可用余额max']-feature['去重后放款前预借现金额度max']
feature['去重后放款前本期账单最低还款额max与放款前循环利息max之和']=feature['去重后放款前本期账单最低还款额max']+feature['去重后放款前循环利息max']

去重后放款前账单min=gb.min()
去重后放款前账单min.columns = ['用户标识', '去重后放款前上期账单金额min', '去重后放款前上期还款金额min','去重后放款前信用卡额度min'
                       ,'去重后放款前本期账单余额min','去重后放款前本期账单最低还款额min','去重后放款前消费笔数min'
                       ,'去重后放款前本期账单金额min','去重后放款前调整金额min','去重后放款前循环利息min'
                       ,'去重后放款前可用余额min','去重后放款前预借现金额度min']
feature=pd.merge(feature, 去重后放款前账单min,how='left', on = "用户标识")
feature['去重后放款前上期还款金额min与放款前上期账单金额min差值']=feature['去重后放款前上期还款金额min']-feature['去重后放款前上期账单金额min']
feature['去重后放款前信用卡额度min与放款前本期账单余额min差值']=feature['去重后放款前信用卡额度min']-feature['去重后放款前本期账单余额min']
feature['去重后放款前可用余额min与放款前预借现金额度min差值']=feature['去重后放款前可用余额min']-feature['去重后放款前预借现金额度min']
feature['去重后放款前本期账单最低还款额min与放款前循环利息min之和']=feature['去重后放款前本期账单最低还款额min']+feature['去重后放款前循环利息min']

去重后放款前账单mean=gb.mean()
去重后放款前账单mean.columns = ['用户标识', '去重后放款前上期账单金额mean', '去重后放款前上期还款金额mean','去重后放款前信用卡额度mean'
                        ,'去重后放款前本期账单余额mean','去重后放款前本期账单最低还款额mean','去重后放款前消费笔数mean'
                        ,'去重后放款前本期账单金额mean','去重后放款前调整金额mean','去重后放款前循环利息mean'
                        ,'去重后放款前可用余额mean','去重后放款前预借现金额度mean']
feature=pd.merge(feature, 去重后放款前账单mean,how='left', on = "用户标识")
feature['去重后放款前上期还款金额mean与放款前上期账单金额mean差值']=feature['去重后放款前上期还款金额mean']-feature['去重后放款前上期账单金额mean']
feature['去重后放款前信用卡额度mean与放款前本期账单余额mean差值']=feature['去重后放款前信用卡额度mean']-feature['去重后放款前本期账单余额mean']
feature['去重后放款前可用余额mean与放款前预借现金额度mean差值']=feature['去重后放款前可用余额mean']-feature['去重后放款前预借现金额度mean']
feature['去重后放款前本期账单最低还款额mean与放款前循环利息mean之和']=feature['去重后放款前本期账单最低还款额mean']+feature['去重后放款前循环利息mean']


去重后放款前账单median=gb.median()
去重后放款前账单median.columns = ['用户标识', '去重后放款前上期账单金额median', '去重后放款前上期还款金额median'
                          ,'去重后放款前信用卡额度median','去重后放款前本期账单余额median','去重后放款前本期账单最低还款额median'
                          ,'去重后放款前消费笔数median','去重后放款前本期账单金额median','去重后放款前调整金额median'
                          ,'去重后放款前循环利息median','去重后放款前可用余额median','去重后放款前预借现金额度median']
feature=pd.merge(feature, 去重后放款前账单median,how='left', on = "用户标识")
feature['去重后放款前上期还款金额median与放款前上期账单金额median差值']=feature['去重后放款前上期还款金额median']-feature['去重后放款前上期账单金额median']
feature['去重后放款前信用卡额度median与放款前本期账单余额median差值']=feature['去重后放款前信用卡额度median']-feature['去重后放款前本期账单余额median']
feature['去重后放款前可用余额median与放款前预借现金额度median差值']=feature['去重后放款前可用余额median']-feature['去重后放款前预借现金额度median']
feature['去重后放款前本期账单最低还款额median与放款前循环利息median之和']=feature['去重后放款前本期账单最低还款额median']+feature['去重后放款前循环利息median']


去重后放款前账单std=gb.std()
去重后放款前账单std.columns = ['用户标识', '去重后放款前上期账单金额std', '去重后放款前上期还款金额std','去重后放款前信用卡额度std'
                       ,'去重后放款前本期账单余额std','去重后放款前本期账单最低还款额std','去重后放款前消费笔数std'
                       ,'去重后放款前本期账单金额std','去重后放款前调整金额std','去重后放款前循环利息std'
                       ,'去重后放款前可用余额std','去重后放款前预借现金额度std']
feature=pd.merge(feature, 去重后放款前账单std,how='left', on = "用户标识")
feature['去重后放款前上期还款金额std与放款前上期账单金额std差值']=feature['去重后放款前上期还款金额std']-feature['去重后放款前上期账单金额std']
feature['去重后放款前信用卡额度std与放款前本期账单余额std差值']=feature['去重后放款前信用卡额度std']-feature['去重后放款前本期账单余额std']
feature['去重后放款前可用余额std与放款前预借现金额度std差值']=feature['去重后放款前可用余额std']-feature['去重后放款前预借现金额度std']
feature['去重后放款前本期账单最低还款额std与放款前循环利息std之和']=feature['去重后放款前本期账单最低还款额std']+feature['去重后放款前循环利息std']

去重后放款前账单var=gb.var()
去重后放款前账单var.columns = ['用户标识', '去重后放款前上期账单金额var', '去重后放款前上期还款金额var','去重后放款前信用卡额度var'
                       ,'去重后放款前本期账单余额var','去重后放款前本期账单最低还款额var','去重后放款前消费笔数var'
                       ,'去重后放款前本期账单金额var','去重后放款前调整金额var','去重后放款前循环利息var','去重后放款前可用余额var','去重后放款前预借现金额度var']
feature=pd.merge(feature, 去重后放款前账单var,how='left', on = "用户标识")
feature['去重后放款前上期还款金额var与放款前上期账单金额var差值']=feature['去重后放款前上期还款金额var']-feature['去重后放款前上期账单金额var']
feature['去重后放款前信用卡额度var与放款前本期账单余额var差值']=feature['去重后放款前信用卡额度var']-feature['去重后放款前本期账单余额var']
feature['去重后放款前可用余额var与放款前预借现金额度var差值']=feature['去重后放款前可用余额var']-feature['去重后放款前预借现金额度var']
feature['去重后放款前本期账单最低还款额var与放款前循环利息var之和']=feature['去重后放款前本期账单最低还款额var']+feature['去重后放款前循环利息var']

feature#55596 rows × 242 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,去重后放款前消费笔数var,去重后放款前本期账单金额var,去重后放款前调整金额var,去重后放款前循环利息var,去重后放款前可用余额var,去重后放款前预借现金额度var,去重后放款前上期还款金额var与放款前上期账单金额var差值,去重后放款前信用卡额度var与放款前本期账单余额var差值,去重后放款前可用余额var与放款前预借现金额度var差值,去重后放款前本期账单最低还款额var与放款前循环利息var之和
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,136.760811,135.406663,139.365030,176.806257,88.329982,24.0,136.072147,0.000000,...,8.750000,75.719994,0.000000,0.000000,0.000000,46.134597,-1.898401,74.765489,-46.134597,233.662905
2,3,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,144.813292,105.148045,338.422883,372.743693,346.818948,2.0,158.766677,0.000000,...,0.200000,82.491690,0.000000,0.000000,31.789011,91.328029,-15.183258,51.495389,-59.539018,1.164896
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,296.822014,289.286843,348.958827,297.547743,291.448253,56.0,214.111454,0.000000,...,9.595588,92.718104,0.000000,0.000000,122.560890,87.826081,-42.255465,-84.266517,34.734809,20.074344
6,7,68458,113.912497,499.590074,545.060225,44.272435,247.793789,45.0,372.624755,0.000000,...,5.810277,94.018040,0.000000,0.000000,64.135840,66.806218,-278.553833,-339.418927,-2.670378,81.344336
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,68458,203.990621,277.888959,286.764829,244.116769,238.446171,1.0,186.410678,0.000000,...,0.071429,106.325812,0.000000,0.000000,0.000000,96.144117,-198.432034,-109.449238,-96.144117,25.099748
9,10,68458,1229.021694,1323.621599,1774.741754,1444.023419,497.256464,73.0,1494.956298,0.000000,...,1.529514,41.203315,0.000000,0.000000,15.974712,85.168881,-41.265223,-78.202090,-69.194169,203.268940


# 补充特征:
**放款后：**

In [14]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#d=训练信用卡账单表[(训练信用卡账单表['时间']>0)]
feature_beifen=feature
###补充特征，增加 放款后的各种统计信息sum count max min mean std var等
gb=d[(d['时间']>d['放款时间'])].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

放款后账单sum=gb.sum()
放款后账单sum.columns = ['用户标识', '放款后上期账单金额sum', '放款后上期还款金额sum','放款后信用卡额度sum','放款后本期账单余额sum',
                     '放款后本期账单最低还款额sum','放款后消费笔数sum','放款后本期账单金额sum','放款后调整金额sum','放款后循环利息sum',
                     '放款后可用余额sum','放款后预借现金额度sum']
feature=pd.merge(feature, 放款后账单sum,how='left', on = "用户标识")
feature['放款后上期还款金额sum与放款后上期账单金额sum差值']=feature['放款后上期还款金额sum']-feature['放款后上期账单金额sum']
feature['放款后信用卡额度sum与放款后本期账单余额sum差值']=feature['放款后信用卡额度sum']-feature['放款后本期账单余额sum']
feature['放款后可用余额sum与放款后预借现金额度sum差值']=feature['放款后可用余额sum']-feature['放款后预借现金额度sum']
feature['放款后本期账单最低还款额sum与放款后循环利息sum之和']=feature['放款后本期账单最低还款额sum']+feature['放款后循环利息sum']

放款后账单count=gb.count()
放款后账单count.columns = ['用户标识', '放款后上期账单金额count', '放款后上期还款金额count','放款后信用卡额度count','放款后本期账单余额count',
                     '放款后本期账单最低还款额count','放款后消费笔数count','放款后本期账单金额count','放款后调整金额count','放款后循环利息count',
                     '放款后可用余额count','放款后预借现金额度count']
feature=pd.merge(feature, 放款后账单count,how='left', on = "用户标识")

放款后账单max=gb.max()
放款后账单max.columns = ['用户标识', '放款后上期账单金额max', '放款后上期还款金额max','放款后信用卡额度max','放款后本期账单余额max',
                     '放款后本期账单最低还款额max','放款后消费笔数max','放款后本期账单金额max','放款后调整金额max','放款后循环利息max',
                     '放款后可用余额max','放款后预借现金额度max']
feature=pd.merge(feature, 放款后账单max,how='left', on = "用户标识")
feature['放款后上期还款金额max与放款后上期账单金额max差值']=feature['放款后上期还款金额max']-feature['放款后上期账单金额max']
feature['放款后信用卡额度max与放款后本期账单余额max差值']=feature['放款后信用卡额度max']-feature['放款后本期账单余额max']
feature['放款后可用余额max与放款后预借现金额度max差值']=feature['放款后可用余额max']-feature['放款后预借现金额度max']
feature['放款后本期账单最低还款额max与放款后循环利息max之和']=feature['放款后本期账单最低还款额max']+feature['放款后循环利息max']

放款后账单min=gb.min()
放款后账单min.columns = ['用户标识', '放款后上期账单金额min', '放款后上期还款金额min','放款后信用卡额度min','放款后本期账单余额min',
                     '放款后本期账单最低还款额min','放款后消费笔数min','放款后本期账单金额min','放款后调整金额min','放款后循环利息min',
                     '放款后可用余额min','放款后预借现金额度min']
feature=pd.merge(feature, 放款后账单min,how='left', on = "用户标识")
feature['放款后上期还款金额min与放款后上期账单金额min差值']=feature['放款后上期还款金额min']-feature['放款后上期账单金额min']
feature['放款后信用卡额度min与放款后本期账单余额min差值']=feature['放款后信用卡额度min']-feature['放款后本期账单余额min']
feature['放款后可用余额min与放款后预借现金额度min差值']=feature['放款后可用余额min']-feature['放款后预借现金额度min']
feature['放款后本期账单最低还款额min与放款后循环利息min之和']=feature['放款后本期账单最低还款额min']+feature['放款后循环利息min']

放款后账单mean=gb.mean()
放款后账单mean.columns = ['用户标识', '放款后上期账单金额mean', '放款后上期还款金额mean','放款后信用卡额度mean','放款后本期账单余额mean',
                     '放款后本期账单最低还款额mean','放款后消费笔数mean','放款后本期账单金额mean','放款后调整金额mean','放款后循环利息mean',
                     '放款后可用余额mean','放款后预借现金额度mean']
feature=pd.merge(feature, 放款后账单mean,how='left', on = "用户标识")
feature['放款后上期还款金额mean与放款后上期账单金额mean差值']=feature['放款后上期还款金额mean']-feature['放款后上期账单金额mean']
feature['放款后信用卡额度mean与放款后本期账单余额mean差值']=feature['放款后信用卡额度mean']-feature['放款后本期账单余额mean']
feature['放款后可用余额mean与放款后预借现金额度mean差值']=feature['放款后可用余额mean']-feature['放款后预借现金额度mean']
feature['放款后本期账单最低还款额mean与放款后循环利息mean之和']=feature['放款后本期账单最低还款额mean']+feature['放款后循环利息mean']

放款后账单median=gb.median()
放款后账单median.columns = ['用户标识', '放款后上期账单金额median', '放款后上期还款金额median','放款后信用卡额度median','放款后本期账单余额median',
                  '放款后本期账单最低还款额median','放款后消费笔数median','放款后本期账单金额median','放款后调整金额median',
                  '放款后循环利息median','放款后可用余额median','放款后预借现金额度median']
feature=pd.merge(feature, 放款后账单median,how='left', on = "用户标识")
feature['放款后上期还款金额median与放款后上期账单金额median差值']=feature['放款后上期还款金额median']-feature['放款后上期账单金额median']
feature['放款后信用卡额度median与放款后本期账单余额median差值']=feature['放款后信用卡额度median']-feature['放款后本期账单余额median']
feature['放款后可用余额median与放款后预借现金额度median差值']=feature['放款后可用余额median']-feature['放款后预借现金额度median']
feature['放款后本期账单最低还款额median与放款后循环利息median之和']=feature['放款后本期账单最低还款额median']+feature['放款后循环利息median']

放款后账单std=gb.std()
放款后账单std.columns = ['用户标识', '放款后上期账单金额std', '放款后上期还款金额std','放款后信用卡额度std','放款后本期账单余额std',
                     '放款后本期账单最低还款额std','放款后消费笔数std','放款后本期账单金额std','放款后调整金额std','放款后循环利息std',
                     '放款后可用余额std','放款后预借现金额度std']
feature=pd.merge(feature, 放款后账单std,how='left', on = "用户标识")
feature['放款后上期还款金额std与放款后上期账单金额std差值']=feature['放款后上期还款金额std']-feature['放款后上期账单金额std']
feature['放款后信用卡额度std与放款后本期账单余额std差值']=feature['放款后信用卡额度std']-feature['放款后本期账单余额std']
feature['放款后可用余额std与放款后预借现金额度std差值']=feature['放款后可用余额std']-feature['放款后预借现金额度std']
feature['放款后本期账单最低还款额std与放款后循环利息std之和']=feature['放款后本期账单最低还款额std']+feature['放款后循环利息std']

放款后账单var=gb.var()
放款后账单var.columns = ['用户标识', '放款后上期账单金额var', '放款后上期还款金额var','放款后信用卡额度var','放款后本期账单余额var',
                     '放款后本期账单最低还款额var','放款后消费笔数var','放款后本期账单金额var','放款后调整金额var','放款后循环利息var',
                     '放款后可用余额var','放款后预借现金额度var']
feature=pd.merge(feature, 放款后账单var,how='left', on = "用户标识")
feature['放款后上期还款金额var与放款后上期账单金额var差值']=feature['放款后上期还款金额var']-feature['放款后上期账单金额var']
feature['放款后信用卡额度var与放款后本期账单余额var差值']=feature['放款后信用卡额度var']-feature['放款后本期账单余额var']
feature['放款后可用余额var与放款后预借现金额度var差值']=feature['放款后可用余额var']-feature['放款后预借现金额度var']
feature['放款后本期账单最低还款额var与放款后循环利息var之和']=feature['放款后本期账单最低还款额var']+feature['放款后循环利息var']

feature#55596 rows × 358 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,放款后消费笔数var,放款后本期账单金额var,放款后调整金额var,放款后循环利息var,放款后可用余额var,放款后预借现金额度var,放款后上期还款金额var与放款后上期账单金额var差值,放款后信用卡额度var与放款后本期账单余额var差值,放款后可用余额var与放款后预借现金额度var差值,放款后本期账单最低还款额var与放款后循环利息var之和
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,136.760811,135.406663,139.365030,176.806257,88.329982,24.0,136.072147,0.000000,...,6.935897,80.368279,0.000000,0.000000,0.000000,111.787677,-3.856274,-1.446660,-111.787677,97.265261
2,3,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,144.813292,105.148045,338.422883,372.743693,346.818948,2.0,158.766677,0.000000,...,0.000000,76.342017,0.000000,0.000000,0.000000,88.052869,-28.108086,78.213260,-88.052869,0.710071
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,296.822014,289.286843,348.958827,297.547743,291.448253,56.0,214.111454,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,68458,113.912497,499.590074,545.060225,44.272435,247.793789,45.0,372.624755,0.000000,...,1.628571,100.994502,0.000000,0.000000,90.206048,95.046013,-43.447010,-19.027304,-4.839965,15.992041
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,68458,203.990621,277.888959,286.764829,244.116769,238.446171,1.0,186.410678,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,68458,1229.021694,1323.621599,1774.741754,1444.023419,497.256464,73.0,1494.956298,0.000000,...,0.000000,1.989399,0.000000,0.000000,0.000000,94.593163,-75.243209,-94.789435,-94.593163,44.807352


In [15]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#放款后账单去重统计特征
#d=训练信用卡账单表[(训练信用卡账单表['时间']>0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d[(d['时间']>d['放款时间'])].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后放款后账单sum=gb.sum()
去重后放款后账单sum.columns = ['用户标识', '去重后放款后上期账单金额sum', '去重后放款后上期还款金额sum','去重后放款后信用卡额度sum'
                       ,'去重后放款后本期账单余额sum','去重后放款后本期账单最低还款额sum','去重后放款后消费笔数sum'
                       ,'去重后放款后本期账单金额sum','去重后放款后调整金额sum','去重后放款后循环利息sum','去重后放款后可用余额sum'
                       ,'去重后放款后预借现金额度sum']
feature=pd.merge(feature, 去重后放款后账单sum,how='left', on = "用户标识")
feature['去重后放款后上期还款金额sum与放款后上期账单金额sum差值']=feature['去重后放款后上期还款金额sum']-feature['去重后放款后上期账单金额sum']
feature['去重后放款后信用卡额度sum与放款后本期账单余额sum差值']=feature['去重后放款后信用卡额度sum']-feature['去重后放款后本期账单余额sum']
feature['去重后放款后可用余额sum与放款后预借现金额度sum差值']=feature['去重后放款后可用余额sum']-feature['去重后放款后预借现金额度sum']
feature['去重后放款后本期账单最低还款额sum与放款后循环利息sum之和']=feature['去重后放款后本期账单最低还款额sum']+feature['去重后放款后循环利息sum']

去重后放款后账单count=gb.count()
去重后放款后账单count.columns = ['用户标识', '去重后放款后上期账单金额count', '去重后放款后上期还款金额count','去重后放款后信用卡额度count'
                         ,'去重后放款后本期账单余额count','去重后放款后本期账单最低还款额count','去重后放款后消费笔数count'
                         ,'去重后放款后本期账单金额count','去重后放款后调整金额count','去重后放款后循环利息count'
                         ,'去重后放款后可用余额count','去重后放款后预借现金额度count']
feature=pd.merge(feature, 去重后放款后账单count,how='left', on = "用户标识")

去重后放款后账单max=gb.max()
去重后放款后账单max.columns = ['用户标识', '去重后放款后上期账单金额max', '去重后放款后上期还款金额max','去重后放款后信用卡额度max'
                       ,'去重后放款后本期账单余额max','去重后放款后本期账单最低还款额max','去重后放款后消费笔数max'
                       ,'去重后放款后本期账单金额max','去重后放款后调整金额max','去重后放款后循环利息max'
                       ,'去重后放款后可用余额max','去重后放款后预借现金额度max']
feature=pd.merge(feature, 去重后放款后账单max,how='left', on = "用户标识")
feature['去重后放款后上期还款金额max与放款后上期账单金额max差值']=feature['去重后放款后上期还款金额max']-feature['去重后放款后上期账单金额max']
feature['去重后放款后信用卡额度max与放款后本期账单余额max差值']=feature['去重后放款后信用卡额度max']-feature['去重后放款后本期账单余额max']
feature['去重后放款后可用余额max与放款后预借现金额度max差值']=feature['去重后放款后可用余额max']-feature['去重后放款后预借现金额度max']
feature['去重后放款后本期账单最低还款额max与放款后循环利息max之和']=feature['去重后放款后本期账单最低还款额max']+feature['去重后放款后循环利息max']

去重后放款后账单min=gb.min()
去重后放款后账单min.columns = ['用户标识', '去重后放款后上期账单金额min', '去重后放款后上期还款金额min','去重后放款后信用卡额度min'
                       ,'去重后放款后本期账单余额min','去重后放款后本期账单最低还款额min','去重后放款后消费笔数min'
                       ,'去重后放款后本期账单金额min','去重后放款后调整金额min','去重后放款后循环利息min'
                       ,'去重后放款后可用余额min','去重后放款后预借现金额度min']
feature=pd.merge(feature, 去重后放款后账单min,how='left', on = "用户标识")
feature['去重后放款后上期还款金额min与放款后上期账单金额min差值']=feature['去重后放款后上期还款金额min']-feature['去重后放款后上期账单金额min']
feature['去重后放款后信用卡额度min与放款后本期账单余额min差值']=feature['去重后放款后信用卡额度min']-feature['去重后放款后本期账单余额min']
feature['去重后放款后可用余额min与放款后预借现金额度min差值']=feature['去重后放款后可用余额min']-feature['去重后放款后预借现金额度min']
feature['去重后放款后本期账单最低还款额min与放款后循环利息min之和']=feature['去重后放款后本期账单最低还款额min']+feature['去重后放款后循环利息min']

去重后放款后账单mean=gb.mean()
去重后放款后账单mean.columns = ['用户标识', '去重后放款后上期账单金额mean', '去重后放款后上期还款金额mean','去重后放款后信用卡额度mean'
                        ,'去重后放款后本期账单余额mean','去重后放款后本期账单最低还款额mean','去重后放款后消费笔数mean'
                        ,'去重后放款后本期账单金额mean','去重后放款后调整金额mean','去重后放款后循环利息mean'
                        ,'去重后放款后可用余额mean','去重后放款后预借现金额度mean']
feature=pd.merge(feature, 去重后放款后账单mean,how='left', on = "用户标识")
feature['去重后放款后上期还款金额mean与放款后上期账单金额mean差值']=feature['去重后放款后上期还款金额mean']-feature['去重后放款后上期账单金额mean']
feature['去重后放款后信用卡额度mean与放款后本期账单余额mean差值']=feature['去重后放款后信用卡额度mean']-feature['去重后放款后本期账单余额mean']
feature['去重后放款后可用余额mean与放款后预借现金额度mean差值']=feature['去重后放款后可用余额mean']-feature['去重后放款后预借现金额度mean']
feature['去重后放款后本期账单最低还款额mean与放款后循环利息mean之和']=feature['去重后放款后本期账单最低还款额mean']+feature['去重后放款后循环利息mean']


去重后放款后账单median=gb.median()
去重后放款后账单median.columns = ['用户标识', '去重后放款后上期账单金额median', '去重后放款后上期还款金额median'
                          ,'去重后放款后信用卡额度median','去重后放款后本期账单余额median','去重后放款后本期账单最低还款额median'
                          ,'去重后放款后消费笔数median','去重后放款后本期账单金额median','去重后放款后调整金额median'
                          ,'去重后放款后循环利息median','去重后放款后可用余额median','去重后放款后预借现金额度median']
feature=pd.merge(feature, 去重后放款后账单median,how='left', on = "用户标识")
feature['去重后放款后上期还款金额median与放款后上期账单金额median差值']=feature['去重后放款后上期还款金额median']-feature['去重后放款后上期账单金额median']
feature['去重后放款后信用卡额度median与放款后本期账单余额median差值']=feature['去重后放款后信用卡额度median']-feature['去重后放款后本期账单余额median']
feature['去重后放款后可用余额median与放款后预借现金额度median差值']=feature['去重后放款后可用余额median']-feature['去重后放款后预借现金额度median']
feature['去重后放款后本期账单最低还款额median与放款后循环利息median之和']=feature['去重后放款后本期账单最低还款额median']+feature['去重后放款后循环利息median']


去重后放款后账单std=gb.std()
去重后放款后账单std.columns = ['用户标识', '去重后放款后上期账单金额std', '去重后放款后上期还款金额std','去重后放款后信用卡额度std'
                       ,'去重后放款后本期账单余额std','去重后放款后本期账单最低还款额std','去重后放款后消费笔数std'
                       ,'去重后放款后本期账单金额std','去重后放款后调整金额std','去重后放款后循环利息std'
                       ,'去重后放款后可用余额std','去重后放款后预借现金额度std']
feature=pd.merge(feature, 去重后放款后账单std,how='left', on = "用户标识")
feature['去重后放款后上期还款金额std与放款后上期账单金额std差值']=feature['去重后放款后上期还款金额std']-feature['去重后放款后上期账单金额std']
feature['去重后放款后信用卡额度std与放款后本期账单余额std差值']=feature['去重后放款后信用卡额度std']-feature['去重后放款后本期账单余额std']
feature['去重后放款后可用余额std与放款后预借现金额度std差值']=feature['去重后放款后可用余额std']-feature['去重后放款后预借现金额度std']
feature['去重后放款后本期账单最低还款额std与放款后循环利息std之和']=feature['去重后放款后本期账单最低还款额std']+feature['去重后放款后循环利息std']

去重后放款后账单var=gb.var()
去重后放款后账单var.columns = ['用户标识', '去重后放款后上期账单金额var', '去重后放款后上期还款金额var','去重后放款后信用卡额度var'
                       ,'去重后放款后本期账单余额var','去重后放款后本期账单最低还款额var','去重后放款后消费笔数var'
                       ,'去重后放款后本期账单金额var','去重后放款后调整金额var','去重后放款后循环利息var','去重后放款后可用余额var','去重后放款后预借现金额度var']
feature=pd.merge(feature, 去重后放款后账单var,how='left', on = "用户标识")
feature['去重后放款后上期还款金额var与放款后上期账单金额var差值']=feature['去重后放款后上期还款金额var']-feature['去重后放款后上期账单金额var']
feature['去重后放款后信用卡额度var与放款后本期账单余额var差值']=feature['去重后放款后信用卡额度var']-feature['去重后放款后本期账单余额var']
feature['去重后放款后可用余额var与放款后预借现金额度var差值']=feature['去重后放款后可用余额var']-feature['去重后放款后预借现金额度var']
feature['去重后放款后本期账单最低还款额var与放款后循环利息var之和']=feature['去重后放款后本期账单最低还款额var']+feature['去重后放款后循环利息var']

feature#55596 rows × 474 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,去重后放款后消费笔数var,去重后放款后本期账单金额var,去重后放款后调整金额var,去重后放款后循环利息var,去重后放款后可用余额var,去重后放款后预借现金额度var,去重后放款后上期还款金额var与放款后上期账单金额var差值,去重后放款后信用卡额度var与放款后本期账单余额var差值,去重后放款后可用余额var与放款后预借现金额度var差值,去重后放款后本期账单最低还款额var与放款后循环利息var之和
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,136.760811,135.406663,139.365030,176.806257,88.329982,24.0,136.072147,0.000000,...,6.935897,80.368279,0.000000,0.000000,0.000000,111.787677,-3.856274,-1.446660,-111.787677,97.265261
2,3,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,144.813292,105.148045,338.422883,372.743693,346.818948,2.0,158.766677,0.000000,...,0.000000,76.342017,0.000000,0.000000,0.000000,88.052869,-28.108086,78.213260,-88.052869,0.710071
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,296.822014,289.286843,348.958827,297.547743,291.448253,56.0,214.111454,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,68458,113.912497,499.590074,545.060225,44.272435,247.793789,45.0,372.624755,0.000000,...,1.820261,104.861753,0.000000,0.000000,75.871383,81.545337,-0.213149,-0.228701,-5.673954,0.722224
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,68458,203.990621,277.888959,286.764829,244.116769,238.446171,1.0,186.410678,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,68458,1229.021694,1323.621599,1774.741754,1444.023419,497.256464,73.0,1494.956298,0.000000,...,0.000000,1.837810,0.000000,0.000000,0.000000,96.627967,-54.399432,-94.405218,-96.627967,68.483343


In [17]:
feature.to_csv("../feature/用户账单表特征训练表20170203_时间已知.csv",index=None,encoding="gb2312")#55596 rows × 474 columns

时间未知：

In [18]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=训练信用卡账单表[(训练信用卡账单表['时间']==0)]
feature=训练放款时间表[['用户标识']]
###补充特征，增加 时间未知的各种统计信息sum count max min mean std var等
gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

时间未知账单sum=gb.sum()
时间未知账单sum.columns = ['用户标识', '时间未知上期账单金额sum', '时间未知上期还款金额sum','时间未知信用卡额度sum','时间未知本期账单余额sum',
                     '时间未知本期账单最低还款额sum','时间未知消费笔数sum','时间未知本期账单金额sum','时间未知调整金额sum','时间未知循环利息sum',
                     '时间未知可用余额sum','时间未知预借现金额度sum','时间未知还款状态sum']
feature=pd.merge(feature, 时间未知账单sum,how='left', on = "用户标识")
feature['时间未知上期还款金额sum与时间未知上期账单金额sum差值']=feature['时间未知上期还款金额sum']-feature['时间未知上期账单金额sum']
feature['时间未知信用卡额度sum与时间未知本期账单余额sum差值']=feature['时间未知信用卡额度sum']-feature['时间未知本期账单余额sum']
feature['时间未知可用余额sum与时间未知预借现金额度sum差值']=feature['时间未知可用余额sum']-feature['时间未知预借现金额度sum']
feature['时间未知本期账单最低还款额sum与时间未知循环利息sum之和']=feature['时间未知本期账单最低还款额sum']+feature['时间未知循环利息sum']

时间未知账单count=gb.count()
时间未知账单count.columns = ['用户标识', '时间未知上期账单金额count', '时间未知上期还款金额count','时间未知信用卡额度count','时间未知本期账单余额count',
                     '时间未知本期账单最低还款额count','时间未知消费笔数count','时间未知本期账单金额count','时间未知调整金额count','时间未知循环利息count',
                     '时间未知可用余额count','时间未知预借现金额度count','时间未知还款状态count']
feature=pd.merge(feature, 时间未知账单count,how='left', on = "用户标识")

时间未知账单max=gb.max()
时间未知账单max.columns = ['用户标识', '时间未知上期账单金额max', '时间未知上期还款金额max','时间未知信用卡额度max','时间未知本期账单余额max',
                     '时间未知本期账单最低还款额max','时间未知消费笔数max','时间未知本期账单金额max','时间未知调整金额max','时间未知循环利息max',
                     '时间未知可用余额max','时间未知预借现金额度max','时间未知还款状态max']
feature=pd.merge(feature, 时间未知账单max,how='left', on = "用户标识")
feature['时间未知上期还款金额max与时间未知上期账单金额max差值']=feature['时间未知上期还款金额max']-feature['时间未知上期账单金额max']
feature['时间未知信用卡额度max与时间未知本期账单余额max差值']=feature['时间未知信用卡额度max']-feature['时间未知本期账单余额max']
feature['时间未知可用余额max与时间未知预借现金额度max差值']=feature['时间未知可用余额max']-feature['时间未知预借现金额度max']
feature['时间未知本期账单最低还款额max与时间未知循环利息max之和']=feature['时间未知本期账单最低还款额max']+feature['时间未知循环利息max']

时间未知账单min=gb.min()
时间未知账单min.columns = ['用户标识', '时间未知上期账单金额min', '时间未知上期还款金额min','时间未知信用卡额度min','时间未知本期账单余额min',
                     '时间未知本期账单最低还款额min','时间未知消费笔数min','时间未知本期账单金额min','时间未知调整金额min','时间未知循环利息min',
                     '时间未知可用余额min','时间未知预借现金额度min','时间未知还款状态min']
feature=pd.merge(feature, 时间未知账单min,how='left', on = "用户标识")
feature['时间未知上期还款金额min与时间未知上期账单金额min差值']=feature['时间未知上期还款金额min']-feature['时间未知上期账单金额min']
feature['时间未知信用卡额度min与时间未知本期账单余额min差值']=feature['时间未知信用卡额度min']-feature['时间未知本期账单余额min']
feature['时间未知可用余额min与时间未知预借现金额度min差值']=feature['时间未知可用余额min']-feature['时间未知预借现金额度min']
feature['时间未知本期账单最低还款额min与时间未知循环利息min之和']=feature['时间未知本期账单最低还款额min']+feature['时间未知循环利息min']

时间未知账单mean=gb.mean()
时间未知账单mean.columns = ['用户标识', '时间未知上期账单金额mean', '时间未知上期还款金额mean','时间未知信用卡额度mean','时间未知本期账单余额mean',
                     '时间未知本期账单最低还款额mean','时间未知消费笔数mean','时间未知本期账单金额mean','时间未知调整金额mean','时间未知循环利息mean',
                     '时间未知可用余额mean','时间未知预借现金额度mean','时间未知还款状态mean']
feature=pd.merge(feature, 时间未知账单mean,how='left', on = "用户标识")
feature['时间未知上期还款金额mean与时间未知上期账单金额mean差值']=feature['时间未知上期还款金额mean']-feature['时间未知上期账单金额mean']
feature['时间未知信用卡额度mean与时间未知本期账单余额mean差值']=feature['时间未知信用卡额度mean']-feature['时间未知本期账单余额mean']
feature['时间未知可用余额mean与时间未知预借现金额度mean差值']=feature['时间未知可用余额mean']-feature['时间未知预借现金额度mean']
feature['时间未知本期账单最低还款额mean与时间未知循环利息mean之和']=feature['时间未知本期账单最低还款额mean']+feature['时间未知循环利息mean']

时间未知账单median=gb.median()
时间未知账单median.columns = ['用户标识', '时间未知上期账单金额median', '时间未知上期还款金额median','时间未知信用卡额度median','时间未知本期账单余额median',
                  '时间未知本期账单最低还款额median','时间未知消费笔数median','时间未知本期账单金额median','时间未知调整金额median',
                  '时间未知循环利息median','时间未知可用余额median','时间未知预借现金额度median','时间未知还款状态median']
feature=pd.merge(feature, 时间未知账单median,how='left', on = "用户标识")
feature['时间未知上期还款金额median与时间未知上期账单金额median差值']=feature['时间未知上期还款金额median']-feature['时间未知上期账单金额median']
feature['时间未知信用卡额度median与时间未知本期账单余额median差值']=feature['时间未知信用卡额度median']-feature['时间未知本期账单余额median']
feature['时间未知可用余额median与时间未知预借现金额度median差值']=feature['时间未知可用余额median']-feature['时间未知预借现金额度median']
feature['时间未知本期账单最低还款额median与时间未知循环利息median之和']=feature['时间未知本期账单最低还款额median']+feature['时间未知循环利息median']

时间未知账单std=gb.std()
时间未知账单std.columns = ['用户标识', '时间未知上期账单金额std', '时间未知上期还款金额std','时间未知信用卡额度std','时间未知本期账单余额std',
                     '时间未知本期账单最低还款额std','时间未知消费笔数std','时间未知本期账单金额std','时间未知调整金额std','时间未知循环利息std',
                     '时间未知可用余额std','时间未知预借现金额度std','时间未知还款状态std']
feature=pd.merge(feature, 时间未知账单std,how='left', on = "用户标识")
feature['时间未知上期还款金额std与时间未知上期账单金额std差值']=feature['时间未知上期还款金额std']-feature['时间未知上期账单金额std']
feature['时间未知信用卡额度std与时间未知本期账单余额std差值']=feature['时间未知信用卡额度std']-feature['时间未知本期账单余额std']
feature['时间未知可用余额std与时间未知预借现金额度std差值']=feature['时间未知可用余额std']-feature['时间未知预借现金额度std']
feature['时间未知本期账单最低还款额std与时间未知循环利息std之和']=feature['时间未知本期账单最低还款额std']+feature['时间未知循环利息std']

时间未知账单var=gb.var()
时间未知账单var.columns = ['用户标识', '时间未知上期账单金额var', '时间未知上期还款金额var','时间未知信用卡额度var','时间未知本期账单余额var',
                     '时间未知本期账单最低还款额var','时间未知消费笔数var','时间未知本期账单金额var','时间未知调整金额var','时间未知循环利息var',
                     '时间未知可用余额var','时间未知预借现金额度var','时间未知还款状态var']
feature=pd.merge(feature, 时间未知账单var,how='left', on = "用户标识")
feature['时间未知上期还款金额var与时间未知上期账单金额var差值']=feature['时间未知上期还款金额var']-feature['时间未知上期账单金额var']
feature['时间未知信用卡额度var与时间未知本期账单余额var差值']=feature['时间未知信用卡额度var']-feature['时间未知本期账单余额var']
feature['时间未知可用余额var与时间未知预借现金额度var差值']=feature['时间未知可用余额var']-feature['时间未知预借现金额度var']
feature['时间未知本期账单最低还款额var与时间未知循环利息var之和']=feature['时间未知本期账单最低还款额var']+feature['时间未知循环利息var']
feature#55596 rows × 125 columns

,用户标识,时间未知上期账单金额sum,时间未知上期还款金额sum,时间未知信用卡额度sum,时间未知本期账单余额sum,时间未知本期账单最低还款额sum,时间未知消费笔数sum,时间未知本期账单金额sum,时间未知调整金额sum,时间未知循环利息sum,...,时间未知本期账单金额var,时间未知调整金额var,时间未知循环利息var,时间未知可用余额var,时间未知预借现金额度var,时间未知还款状态var,时间未知上期还款金额var与时间未知上期账单金额var差值,时间未知信用卡额度var与时间未知本期账单余额var差值,时间未知可用余额var与时间未知预借现金额度var差值,时间未知本期账单最低还款额var与时间未知循环利息var之和
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,17.389606,17.389606,19.460445,16.656269,15.366101,1.0,16.656269,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,36.733542,37.160348,36.723666,36.696541,32.091301,4.0,37.162823,0.000000,0.000000,...,1.617681,0.000000,0.000000,0.000000,0.000000,0.0,1.674981,-0.001036,0.000000,0.001036
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,41.358453,37.079520,41.328836,41.272092,36.948737,0.0,35.914512,0.000000,0.000000,...,0.623708,0.000000,0.000000,0.000000,0.000000,0.0,0.000862,-0.003286,0.000000,0.003046
5,6,506.823422,439.973993,514.888537,507.847365,463.574687,66.0,308.885682,0.000000,0.000000,...,89.694709,0.000000,0.000000,16.020165,90.934794,0.0,28.513411,-0.018584,-74.914629,0.318808
6,7,589.363120,1164.064195,1225.482562,154.489156,537.914467,191.0,695.115043,0.000000,0.000000,...,106.046084,0.000000,0.000000,76.097935,95.485061,0.0,-214.501286,-325.925132,-19.387126,97.599102
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,117.113699,119.382678,121.578561,140.095475,128.940904,5.0,61.999518,0.000000,0.000000,...,114.448784,0.000000,0.000000,0.000000,98.493371,0.0,3.002471,37.685688,-98.493371,43.767593
9,10,708.570234,442.284608,1118.667180,879.184114,844.514820,44.0,547.791118,0.000000,0.000000,...,86.498570,0.000000,0.000000,0.000000,35.073672,0.0,123.653371,-73.536408,-35.073672,35.668346


去重（时间未知）：

In [19]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#时间未知账单去重统计特征
#d=训练信用卡账单表[(训练信用卡账单表['时间']==0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
              ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
              ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后时间未知账单sum=gb.sum()
去重后时间未知账单sum.columns = ['用户标识', '去重后时间未知上期账单金额sum', '去重后时间未知上期还款金额sum','去重后时间未知信用卡额度sum'
                       ,'去重后时间未知本期账单余额sum','去重后时间未知本期账单最低还款额sum','去重后时间未知消费笔数sum'
                       ,'去重后时间未知本期账单金额sum','去重后时间未知调整金额sum','去重后时间未知循环利息sum','去重后时间未知可用余额sum'
                       ,'去重后时间未知预借现金额度sum']
feature=pd.merge(feature, 去重后时间未知账单sum,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额sum与时间未知上期账单金额sum差值']=feature['去重后时间未知上期还款金额sum']-feature['去重后时间未知上期账单金额sum']
feature['去重后时间未知信用卡额度sum与时间未知本期账单余额sum差值']=feature['去重后时间未知信用卡额度sum']-feature['去重后时间未知本期账单余额sum']
feature['去重后时间未知可用余额sum与时间未知预借现金额度sum差值']=feature['去重后时间未知可用余额sum']-feature['去重后时间未知预借现金额度sum']
feature['去重后时间未知本期账单最低还款额sum与时间未知循环利息sum之和']=feature['去重后时间未知本期账单最低还款额sum']+feature['去重后时间未知循环利息sum']

去重后时间未知账单count=gb.count()
去重后时间未知账单count.columns = ['用户标识', '去重后时间未知上期账单金额count', '去重后时间未知上期还款金额count','去重后时间未知信用卡额度count'
                         ,'去重后时间未知本期账单余额count','去重后时间未知本期账单最低还款额count','去重后时间未知消费笔数count'
                         ,'去重后时间未知本期账单金额count','去重后时间未知调整金额count','去重后时间未知循环利息count'
                         ,'去重后时间未知可用余额count','去重后时间未知预借现金额度count']
feature=pd.merge(feature, 去重后时间未知账单count,how='left', on = "用户标识")

去重后时间未知账单max=gb.max()
去重后时间未知账单max.columns = ['用户标识', '去重后时间未知上期账单金额max', '去重后时间未知上期还款金额max','去重后时间未知信用卡额度max'
                       ,'去重后时间未知本期账单余额max','去重后时间未知本期账单最低还款额max','去重后时间未知消费笔数max'
                       ,'去重后时间未知本期账单金额max','去重后时间未知调整金额max','去重后时间未知循环利息max'
                       ,'去重后时间未知可用余额max','去重后时间未知预借现金额度max']
feature=pd.merge(feature, 去重后时间未知账单max,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额max与时间未知上期账单金额max差值']=feature['去重后时间未知上期还款金额max']-feature['去重后时间未知上期账单金额max']
feature['去重后时间未知信用卡额度max与时间未知本期账单余额max差值']=feature['去重后时间未知信用卡额度max']-feature['去重后时间未知本期账单余额max']
feature['去重后时间未知可用余额max与时间未知预借现金额度max差值']=feature['去重后时间未知可用余额max']-feature['去重后时间未知预借现金额度max']
feature['去重后时间未知本期账单最低还款额max与时间未知循环利息max之和']=feature['去重后时间未知本期账单最低还款额max']+feature['去重后时间未知循环利息max']

去重后时间未知账单min=gb.min()
去重后时间未知账单min.columns = ['用户标识', '去重后时间未知上期账单金额min', '去重后时间未知上期还款金额min','去重后时间未知信用卡额度min'
                       ,'去重后时间未知本期账单余额min','去重后时间未知本期账单最低还款额min','去重后时间未知消费笔数min'
                       ,'去重后时间未知本期账单金额min','去重后时间未知调整金额min','去重后时间未知循环利息min'
                       ,'去重后时间未知可用余额min','去重后时间未知预借现金额度min']
feature=pd.merge(feature, 去重后时间未知账单min,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额min与时间未知上期账单金额min差值']=feature['去重后时间未知上期还款金额min']-feature['去重后时间未知上期账单金额min']
feature['去重后时间未知信用卡额度min与时间未知本期账单余额min差值']=feature['去重后时间未知信用卡额度min']-feature['去重后时间未知本期账单余额min']
feature['去重后时间未知可用余额min与时间未知预借现金额度min差值']=feature['去重后时间未知可用余额min']-feature['去重后时间未知预借现金额度min']
feature['去重后时间未知本期账单最低还款额min与时间未知循环利息min之和']=feature['去重后时间未知本期账单最低还款额min']+feature['去重后时间未知循环利息min']

去重后时间未知账单mean=gb.mean()
去重后时间未知账单mean.columns = ['用户标识', '去重后时间未知上期账单金额mean', '去重后时间未知上期还款金额mean','去重后时间未知信用卡额度mean'
                        ,'去重后时间未知本期账单余额mean','去重后时间未知本期账单最低还款额mean','去重后时间未知消费笔数mean'
                        ,'去重后时间未知本期账单金额mean','去重后时间未知调整金额mean','去重后时间未知循环利息mean'
                        ,'去重后时间未知可用余额mean','去重后时间未知预借现金额度mean']
feature=pd.merge(feature, 去重后时间未知账单mean,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额mean与时间未知上期账单金额mean差值']=feature['去重后时间未知上期还款金额mean']-feature['去重后时间未知上期账单金额mean']
feature['去重后时间未知信用卡额度mean与时间未知本期账单余额mean差值']=feature['去重后时间未知信用卡额度mean']-feature['去重后时间未知本期账单余额mean']
feature['去重后时间未知可用余额mean与时间未知预借现金额度mean差值']=feature['去重后时间未知可用余额mean']-feature['去重后时间未知预借现金额度mean']
feature['去重后时间未知本期账单最低还款额mean与时间未知循环利息mean之和']=feature['去重后时间未知本期账单最低还款额mean']+feature['去重后时间未知循环利息mean']


去重后时间未知账单median=gb.median()
去重后时间未知账单median.columns = ['用户标识', '去重后时间未知上期账单金额median', '去重后时间未知上期还款金额median'
                          ,'去重后时间未知信用卡额度median','去重后时间未知本期账单余额median','去重后时间未知本期账单最低还款额median'
                          ,'去重后时间未知消费笔数median','去重后时间未知本期账单金额median','去重后时间未知调整金额median'
                          ,'去重后时间未知循环利息median','去重后时间未知可用余额median','去重后时间未知预借现金额度median']
feature=pd.merge(feature, 去重后时间未知账单median,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额median与时间未知上期账单金额median差值']=feature['去重后时间未知上期还款金额median']-feature['去重后时间未知上期账单金额median']
feature['去重后时间未知信用卡额度median与时间未知本期账单余额median差值']=feature['去重后时间未知信用卡额度median']-feature['去重后时间未知本期账单余额median']
feature['去重后时间未知可用余额median与时间未知预借现金额度median差值']=feature['去重后时间未知可用余额median']-feature['去重后时间未知预借现金额度median']
feature['去重后时间未知本期账单最低还款额median与时间未知循环利息median之和']=feature['去重后时间未知本期账单最低还款额median']+feature['去重后时间未知循环利息median']


去重后时间未知账单std=gb.std()
去重后时间未知账单std.columns = ['用户标识', '去重后时间未知上期账单金额std', '去重后时间未知上期还款金额std','去重后时间未知信用卡额度std'
                       ,'去重后时间未知本期账单余额std','去重后时间未知本期账单最低还款额std','去重后时间未知消费笔数std'
                       ,'去重后时间未知本期账单金额std','去重后时间未知调整金额std','去重后时间未知循环利息std'
                       ,'去重后时间未知可用余额std','去重后时间未知预借现金额度std']
feature=pd.merge(feature, 去重后时间未知账单std,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额std与时间未知上期账单金额std差值']=feature['去重后时间未知上期还款金额std']-feature['去重后时间未知上期账单金额std']
feature['去重后时间未知信用卡额度std与时间未知本期账单余额std差值']=feature['去重后时间未知信用卡额度std']-feature['去重后时间未知本期账单余额std']
feature['去重后时间未知可用余额std与时间未知预借现金额度std差值']=feature['去重后时间未知可用余额std']-feature['去重后时间未知预借现金额度std']
feature['去重后时间未知本期账单最低还款额std与时间未知循环利息std之和']=feature['去重后时间未知本期账单最低还款额std']+feature['去重后时间未知循环利息std']

去重后时间未知账单var=gb.var()
去重后时间未知账单var.columns = ['用户标识', '去重后时间未知上期账单金额var', '去重后时间未知上期还款金额var','去重后时间未知信用卡额度var'
                       ,'去重后时间未知本期账单余额var','去重后时间未知本期账单最低还款额var','去重后时间未知消费笔数var'
                       ,'去重后时间未知本期账单金额var','去重后时间未知调整金额var','去重后时间未知循环利息var','去重后时间未知可用余额var','去重后时间未知预借现金额度var']
feature=pd.merge(feature, 去重后时间未知账单var,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额var与时间未知上期账单金额var差值']=feature['去重后时间未知上期还款金额var']-feature['去重后时间未知上期账单金额var']
feature['去重后时间未知信用卡额度var与时间未知本期账单余额var差值']=feature['去重后时间未知信用卡额度var']-feature['去重后时间未知本期账单余额var']
feature['去重后时间未知可用余额var与时间未知预借现金额度var差值']=feature['去重后时间未知可用余额var']-feature['去重后时间未知预借现金额度var']
feature['去重后时间未知本期账单最低还款额var与时间未知循环利息var之和']=feature['去重后时间未知本期账单最低还款额var']+feature['去重后时间未知循环利息var']

feature#55596 rows × 241 columns

,用户标识,时间未知上期账单金额sum,时间未知上期还款金额sum,时间未知信用卡额度sum,时间未知本期账单余额sum,时间未知本期账单最低还款额sum,时间未知消费笔数sum,时间未知本期账单金额sum,时间未知调整金额sum,时间未知循环利息sum,...,去重后时间未知消费笔数var,去重后时间未知本期账单金额var,去重后时间未知调整金额var,去重后时间未知循环利息var,去重后时间未知可用余额var,去重后时间未知预借现金额度var,去重后时间未知上期还款金额var与时间未知上期账单金额var差值,去重后时间未知信用卡额度var与时间未知本期账单余额var差值,去重后时间未知可用余额var与时间未知预借现金额度var差值,去重后时间未知本期账单最低还款额var与时间未知循环利息var之和
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,17.389606,17.389606,19.460445,16.656269,15.366101,1.0,16.656269,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,36.733542,37.160348,36.723666,36.696541,32.091301,4.0,37.162823,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,41.358453,37.079520,41.328836,41.272092,36.948737,0.0,35.914512,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,506.823422,439.973993,514.888537,507.847365,463.574687,66.0,308.885682,0.000000,0.000000,...,12.333333,132.885929,0.000000,0.000000,133.501373,136.301610,0.111811,-0.176690,-2.800237,0.507759
6,7,589.363120,1164.064195,1225.482562,154.489156,537.914467,191.0,695.115043,0.000000,0.000000,...,33.200000,138.656786,0.000000,0.000000,86.193261,116.012715,-1.577225,0.533268,-29.819454,7.155619
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,117.113699,119.382678,121.578561,140.095475,128.940904,5.0,61.999518,0.000000,0.000000,...,1.000000,148.423646,0.000000,0.000000,0.000000,126.236307,5.861038,138.741063,-126.236307,1.702088
9,10,708.570234,442.284608,1118.667180,879.184114,844.514820,44.0,547.791118,0.000000,0.000000,...,3.000000,0.031660,0.000000,0.000000,0.000000,123.882009,130.227193,-0.239858,-123.882009,0.194184


In [20]:
feature.to_csv("../feature/用户账单表特征训练表20170203_时间未知.csv",index=None,encoding="gb2312")#55596 rows × 241columns 2017.02.03

整体（不区分放款前放款后）：

In [21]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=训练信用卡账单表
feature=训练放款时间表[['用户标识']]
###补充特征，增加 整体的各种统计信息sum count max min mean std var等
gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

整体账单sum=gb.sum()
整体账单sum.columns = ['用户标识', '整体上期账单金额sum', '整体上期还款金额sum','整体信用卡额度sum','整体本期账单余额sum',
                     '整体本期账单最低还款额sum','整体消费笔数sum','整体本期账单金额sum','整体调整金额sum','整体循环利息sum',
                     '整体可用余额sum','整体预借现金额度sum','整体还款状态sum']
feature=pd.merge(feature, 整体账单sum,how='left', on = "用户标识")
feature['整体上期还款金额sum与整体上期账单金额sum差值']=feature['整体上期还款金额sum']-feature['整体上期账单金额sum']
feature['整体信用卡额度sum与整体本期账单余额sum差值']=feature['整体信用卡额度sum']-feature['整体本期账单余额sum']
feature['整体可用余额sum与整体预借现金额度sum差值']=feature['整体可用余额sum']-feature['整体预借现金额度sum']
feature['整体本期账单最低还款额sum与整体循环利息sum之和']=feature['整体本期账单最低还款额sum']+feature['整体循环利息sum']

整体账单count=gb.count()
整体账单count.columns = ['用户标识', '整体上期账单金额count', '整体上期还款金额count','整体信用卡额度count','整体本期账单余额count',
                     '整体本期账单最低还款额count','整体消费笔数count','整体本期账单金额count','整体调整金额count','整体循环利息count',
                     '整体可用余额count','整体预借现金额度count','整体还款状态count']
feature=pd.merge(feature, 整体账单count,how='left', on = "用户标识")

整体账单max=gb.max()
整体账单max.columns = ['用户标识', '整体上期账单金额max', '整体上期还款金额max','整体信用卡额度max','整体本期账单余额max',
                     '整体本期账单最低还款额max','整体消费笔数max','整体本期账单金额max','整体调整金额max','整体循环利息max',
                     '整体可用余额max','整体预借现金额度max','整体还款状态max']
feature=pd.merge(feature, 整体账单max,how='left', on = "用户标识")
feature['整体上期还款金额max与整体上期账单金额max差值']=feature['整体上期还款金额max']-feature['整体上期账单金额max']
feature['整体信用卡额度max与整体本期账单余额max差值']=feature['整体信用卡额度max']-feature['整体本期账单余额max']
feature['整体可用余额max与整体预借现金额度max差值']=feature['整体可用余额max']-feature['整体预借现金额度max']
feature['整体本期账单最低还款额max与整体循环利息max之和']=feature['整体本期账单最低还款额max']+feature['整体循环利息max']

整体账单min=gb.min()
整体账单min.columns = ['用户标识', '整体上期账单金额min', '整体上期还款金额min','整体信用卡额度min','整体本期账单余额min',
                     '整体本期账单最低还款额min','整体消费笔数min','整体本期账单金额min','整体调整金额min','整体循环利息min',
                     '整体可用余额min','整体预借现金额度min','整体还款状态min']
feature=pd.merge(feature, 整体账单min,how='left', on = "用户标识")
feature['整体上期还款金额min与整体上期账单金额min差值']=feature['整体上期还款金额min']-feature['整体上期账单金额min']
feature['整体信用卡额度min与整体本期账单余额min差值']=feature['整体信用卡额度min']-feature['整体本期账单余额min']
feature['整体可用余额min与整体预借现金额度min差值']=feature['整体可用余额min']-feature['整体预借现金额度min']
feature['整体本期账单最低还款额min与整体循环利息min之和']=feature['整体本期账单最低还款额min']+feature['整体循环利息min']

整体账单mean=gb.mean()
整体账单mean.columns = ['用户标识', '整体上期账单金额mean', '整体上期还款金额mean','整体信用卡额度mean','整体本期账单余额mean',
                     '整体本期账单最低还款额mean','整体消费笔数mean','整体本期账单金额mean','整体调整金额mean','整体循环利息mean',
                     '整体可用余额mean','整体预借现金额度mean','整体还款状态mean']
feature=pd.merge(feature, 整体账单mean,how='left', on = "用户标识")
feature['整体上期还款金额mean与整体上期账单金额mean差值']=feature['整体上期还款金额mean']-feature['整体上期账单金额mean']
feature['整体信用卡额度mean与整体本期账单余额mean差值']=feature['整体信用卡额度mean']-feature['整体本期账单余额mean']
feature['整体可用余额mean与整体预借现金额度mean差值']=feature['整体可用余额mean']-feature['整体预借现金额度mean']
feature['整体本期账单最低还款额mean与整体循环利息mean之和']=feature['整体本期账单最低还款额mean']+feature['整体循环利息mean']

整体账单median=gb.median()
整体账单median.columns = ['用户标识', '整体上期账单金额median', '整体上期还款金额median','整体信用卡额度median','整体本期账单余额median',
                  '整体本期账单最低还款额median','整体消费笔数median','整体本期账单金额median','整体调整金额median',
                  '整体循环利息median','整体可用余额median','整体预借现金额度median','整体还款状态median']
feature=pd.merge(feature, 整体账单median,how='left', on = "用户标识")
feature['整体上期还款金额median与整体上期账单金额median差值']=feature['整体上期还款金额median']-feature['整体上期账单金额median']
feature['整体信用卡额度median与整体本期账单余额median差值']=feature['整体信用卡额度median']-feature['整体本期账单余额median']
feature['整体可用余额median与整体预借现金额度median差值']=feature['整体可用余额median']-feature['整体预借现金额度median']
feature['整体本期账单最低还款额median与整体循环利息median之和']=feature['整体本期账单最低还款额median']+feature['整体循环利息median']

整体账单std=gb.std()
整体账单std.columns = ['用户标识', '整体上期账单金额std', '整体上期还款金额std','整体信用卡额度std','整体本期账单余额std',
                     '整体本期账单最低还款额std','整体消费笔数std','整体本期账单金额std','整体调整金额std','整体循环利息std',
                     '整体可用余额std','整体预借现金额度std','整体还款状态std']
feature=pd.merge(feature, 整体账单std,how='left', on = "用户标识")
feature['整体上期还款金额std与整体上期账单金额std差值']=feature['整体上期还款金额std']-feature['整体上期账单金额std']
feature['整体信用卡额度std与整体本期账单余额std差值']=feature['整体信用卡额度std']-feature['整体本期账单余额std']
feature['整体可用余额std与整体预借现金额度std差值']=feature['整体可用余额std']-feature['整体预借现金额度std']
feature['整体本期账单最低还款额std与整体循环利息std之和']=feature['整体本期账单最低还款额std']+feature['整体循环利息std']

整体账单var=gb.var()
整体账单var.columns = ['用户标识', '整体上期账单金额var', '整体上期还款金额var','整体信用卡额度var','整体本期账单余额var',
                     '整体本期账单最低还款额var','整体消费笔数var','整体本期账单金额var','整体调整金额var','整体循环利息var',
                     '整体可用余额var','整体预借现金额度var','整体还款状态var']
feature=pd.merge(feature, 整体账单var,how='left', on = "用户标识")
feature['整体上期还款金额var与整体上期账单金额var差值']=feature['整体上期还款金额var']-feature['整体上期账单金额var']
feature['整体信用卡额度var与整体本期账单余额var差值']=feature['整体信用卡额度var']-feature['整体本期账单余额var']
feature['整体可用余额var与整体预借现金额度var差值']=feature['整体可用余额var']-feature['整体预借现金额度var']
feature['整体本期账单最低还款额var与整体循环利息var之和']=feature['整体本期账单最低还款额var']+feature['整体循环利息var']
feature#55596 rows × 125 columns

,用户标识,整体上期账单金额sum,整体上期还款金额sum,整体信用卡额度sum,整体本期账单余额sum,整体本期账单最低还款额sum,整体消费笔数sum,整体本期账单金额sum,整体调整金额sum,整体循环利息sum,...,整体本期账单金额var,整体调整金额var,整体循环利息var,整体可用余额var,整体预借现金额度var,整体还款状态var,整体上期还款金额var与整体上期账单金额var差值,整体信用卡额度var与整体本期账单余额var差值,整体可用余额var与整体预借现金额度var差值,整体本期账单最低还款额var与整体循环利息var之和
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,360.734470,351.387237,428.696405,457.057503,301.508459,58.0,352.352004,0.000000,0.000000,...,71.470209,0.000000,0.000000,0.000000,98.469100,0.0,-2.824033,32.567287,-98.469100,145.318143
2,3,36.733542,37.160348,36.723666,36.696541,32.091301,4.0,37.162823,0.000000,0.000000,...,1.617681,0.000000,0.000000,0.000000,0.000000,0.0,1.674981,-0.001036,0.000000,0.001036
3,4,261.650351,178.153143,577.987348,678.506333,628.451974,2.0,263.290061,0.000000,0.000000,...,78.000994,0.000000,0.000000,18.103521,87.803786,0.0,-20.167102,62.381889,-69.700264,0.954005
4,5,41.358453,37.079520,41.328836,41.272092,36.948737,0.0,35.914512,0.000000,0.000000,...,0.623708,0.000000,0.000000,0.000000,0.000000,0.0,0.000862,-0.003286,0.000000,0.003046
5,6,803.645436,729.260836,863.847364,805.395108,755.022940,122.0,522.997136,0.000000,0.000000,...,88.701035,0.000000,0.000000,61.293908,87.555156,0.0,-1.668253,-34.844525,-26.261248,8.503540
6,7,1070.586607,2069.566733,2202.883453,594.655816,1141.937370,254.0,1329.699230,0.000000,0.000000,...,99.792420,0.000000,0.000000,77.403051,93.603181,0.0,-209.314033,-313.874177,-16.200130,87.745275
7,8,0.000000,70.682736,84.279532,84.273040,75.062704,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,9,321.104320,397.271637,408.343390,384.212244,367.387075,6.0,248.410196,0.000000,0.000000,...,111.478320,0.000000,0.000000,0.000000,99.600406,0.0,-115.955752,-48.420293,-99.600406,30.329771
9,10,2814.411514,2740.659992,3934.936295,3175.096997,2172.459178,117.0,3039.801281,0.000000,0.000000,...,56.759883,0.000000,0.000000,5.224857,80.594896,0.0,13.512057,-74.138396,-75.370039,152.754479


In [22]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#整体账单去重统计特征
#d=训练信用卡账单表[(训练信用卡账单表['时间']==0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
              ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
              ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后整体账单sum=gb.sum()
去重后整体账单sum.columns = ['用户标识', '去重后整体上期账单金额sum', '去重后整体上期还款金额sum','去重后整体信用卡额度sum'
                       ,'去重后整体本期账单余额sum','去重后整体本期账单最低还款额sum','去重后整体消费笔数sum'
                       ,'去重后整体本期账单金额sum','去重后整体调整金额sum','去重后整体循环利息sum','去重后整体可用余额sum'
                       ,'去重后整体预借现金额度sum']
feature=pd.merge(feature, 去重后整体账单sum,how='left', on = "用户标识")
feature['去重后整体上期还款金额sum与整体上期账单金额sum差值']=feature['去重后整体上期还款金额sum']-feature['去重后整体上期账单金额sum']
feature['去重后整体信用卡额度sum与整体本期账单余额sum差值']=feature['去重后整体信用卡额度sum']-feature['去重后整体本期账单余额sum']
feature['去重后整体可用余额sum与整体预借现金额度sum差值']=feature['去重后整体可用余额sum']-feature['去重后整体预借现金额度sum']
feature['去重后整体本期账单最低还款额sum与整体循环利息sum之和']=feature['去重后整体本期账单最低还款额sum']+feature['去重后整体循环利息sum']

去重后整体账单count=gb.count()
去重后整体账单count.columns = ['用户标识', '去重后整体上期账单金额count', '去重后整体上期还款金额count','去重后整体信用卡额度count'
                         ,'去重后整体本期账单余额count','去重后整体本期账单最低还款额count','去重后整体消费笔数count'
                         ,'去重后整体本期账单金额count','去重后整体调整金额count','去重后整体循环利息count'
                         ,'去重后整体可用余额count','去重后整体预借现金额度count']
feature=pd.merge(feature, 去重后整体账单count,how='left', on = "用户标识")

去重后整体账单max=gb.max()
去重后整体账单max.columns = ['用户标识', '去重后整体上期账单金额max', '去重后整体上期还款金额max','去重后整体信用卡额度max'
                       ,'去重后整体本期账单余额max','去重后整体本期账单最低还款额max','去重后整体消费笔数max'
                       ,'去重后整体本期账单金额max','去重后整体调整金额max','去重后整体循环利息max'
                       ,'去重后整体可用余额max','去重后整体预借现金额度max']
feature=pd.merge(feature, 去重后整体账单max,how='left', on = "用户标识")
feature['去重后整体上期还款金额max与整体上期账单金额max差值']=feature['去重后整体上期还款金额max']-feature['去重后整体上期账单金额max']
feature['去重后整体信用卡额度max与整体本期账单余额max差值']=feature['去重后整体信用卡额度max']-feature['去重后整体本期账单余额max']
feature['去重后整体可用余额max与整体预借现金额度max差值']=feature['去重后整体可用余额max']-feature['去重后整体预借现金额度max']
feature['去重后整体本期账单最低还款额max与整体循环利息max之和']=feature['去重后整体本期账单最低还款额max']+feature['去重后整体循环利息max']

去重后整体账单min=gb.min()
去重后整体账单min.columns = ['用户标识', '去重后整体上期账单金额min', '去重后整体上期还款金额min','去重后整体信用卡额度min'
                       ,'去重后整体本期账单余额min','去重后整体本期账单最低还款额min','去重后整体消费笔数min'
                       ,'去重后整体本期账单金额min','去重后整体调整金额min','去重后整体循环利息min'
                       ,'去重后整体可用余额min','去重后整体预借现金额度min']
feature=pd.merge(feature, 去重后整体账单min,how='left', on = "用户标识")
feature['去重后整体上期还款金额min与整体上期账单金额min差值']=feature['去重后整体上期还款金额min']-feature['去重后整体上期账单金额min']
feature['去重后整体信用卡额度min与整体本期账单余额min差值']=feature['去重后整体信用卡额度min']-feature['去重后整体本期账单余额min']
feature['去重后整体可用余额min与整体预借现金额度min差值']=feature['去重后整体可用余额min']-feature['去重后整体预借现金额度min']
feature['去重后整体本期账单最低还款额min与整体循环利息min之和']=feature['去重后整体本期账单最低还款额min']+feature['去重后整体循环利息min']

去重后整体账单mean=gb.mean()
去重后整体账单mean.columns = ['用户标识', '去重后整体上期账单金额mean', '去重后整体上期还款金额mean','去重后整体信用卡额度mean'
                        ,'去重后整体本期账单余额mean','去重后整体本期账单最低还款额mean','去重后整体消费笔数mean'
                        ,'去重后整体本期账单金额mean','去重后整体调整金额mean','去重后整体循环利息mean'
                        ,'去重后整体可用余额mean','去重后整体预借现金额度mean']
feature=pd.merge(feature, 去重后整体账单mean,how='left', on = "用户标识")
feature['去重后整体上期还款金额mean与整体上期账单金额mean差值']=feature['去重后整体上期还款金额mean']-feature['去重后整体上期账单金额mean']
feature['去重后整体信用卡额度mean与整体本期账单余额mean差值']=feature['去重后整体信用卡额度mean']-feature['去重后整体本期账单余额mean']
feature['去重后整体可用余额mean与整体预借现金额度mean差值']=feature['去重后整体可用余额mean']-feature['去重后整体预借现金额度mean']
feature['去重后整体本期账单最低还款额mean与整体循环利息mean之和']=feature['去重后整体本期账单最低还款额mean']+feature['去重后整体循环利息mean']


去重后整体账单median=gb.median()
去重后整体账单median.columns = ['用户标识', '去重后整体上期账单金额median', '去重后整体上期还款金额median'
                          ,'去重后整体信用卡额度median','去重后整体本期账单余额median','去重后整体本期账单最低还款额median'
                          ,'去重后整体消费笔数median','去重后整体本期账单金额median','去重后整体调整金额median'
                          ,'去重后整体循环利息median','去重后整体可用余额median','去重后整体预借现金额度median']
feature=pd.merge(feature, 去重后整体账单median,how='left', on = "用户标识")
feature['去重后整体上期还款金额median与整体上期账单金额median差值']=feature['去重后整体上期还款金额median']-feature['去重后整体上期账单金额median']
feature['去重后整体信用卡额度median与整体本期账单余额median差值']=feature['去重后整体信用卡额度median']-feature['去重后整体本期账单余额median']
feature['去重后整体可用余额median与整体预借现金额度median差值']=feature['去重后整体可用余额median']-feature['去重后整体预借现金额度median']
feature['去重后整体本期账单最低还款额median与整体循环利息median之和']=feature['去重后整体本期账单最低还款额median']+feature['去重后整体循环利息median']


去重后整体账单std=gb.std()
去重后整体账单std.columns = ['用户标识', '去重后整体上期账单金额std', '去重后整体上期还款金额std','去重后整体信用卡额度std'
                       ,'去重后整体本期账单余额std','去重后整体本期账单最低还款额std','去重后整体消费笔数std'
                       ,'去重后整体本期账单金额std','去重后整体调整金额std','去重后整体循环利息std'
                       ,'去重后整体可用余额std','去重后整体预借现金额度std']
feature=pd.merge(feature, 去重后整体账单std,how='left', on = "用户标识")
feature['去重后整体上期还款金额std与整体上期账单金额std差值']=feature['去重后整体上期还款金额std']-feature['去重后整体上期账单金额std']
feature['去重后整体信用卡额度std与整体本期账单余额std差值']=feature['去重后整体信用卡额度std']-feature['去重后整体本期账单余额std']
feature['去重后整体可用余额std与整体预借现金额度std差值']=feature['去重后整体可用余额std']-feature['去重后整体预借现金额度std']
feature['去重后整体本期账单最低还款额std与整体循环利息std之和']=feature['去重后整体本期账单最低还款额std']+feature['去重后整体循环利息std']

去重后整体账单var=gb.var()
去重后整体账单var.columns = ['用户标识', '去重后整体上期账单金额var', '去重后整体上期还款金额var','去重后整体信用卡额度var'
                       ,'去重后整体本期账单余额var','去重后整体本期账单最低还款额var','去重后整体消费笔数var'
                       ,'去重后整体本期账单金额var','去重后整体调整金额var','去重后整体循环利息var','去重后整体可用余额var','去重后整体预借现金额度var']
feature=pd.merge(feature, 去重后整体账单var,how='left', on = "用户标识")
feature['去重后整体上期还款金额var与整体上期账单金额var差值']=feature['去重后整体上期还款金额var']-feature['去重后整体上期账单金额var']
feature['去重后整体信用卡额度var与整体本期账单余额var差值']=feature['去重后整体信用卡额度var']-feature['去重后整体本期账单余额var']
feature['去重后整体可用余额var与整体预借现金额度var差值']=feature['去重后整体可用余额var']-feature['去重后整体预借现金额度var']
feature['去重后整体本期账单最低还款额var与整体循环利息var之和']=feature['去重后整体本期账单最低还款额var']+feature['去重后整体循环利息var']

feature#55596 rows × 241 columns

,用户标识,整体上期账单金额sum,整体上期还款金额sum,整体信用卡额度sum,整体本期账单余额sum,整体本期账单最低还款额sum,整体消费笔数sum,整体本期账单金额sum,整体调整金额sum,整体循环利息sum,...,去重后整体消费笔数var,去重后整体本期账单金额var,去重后整体调整金额var,去重后整体循环利息var,去重后整体可用余额var,去重后整体预借现金额度var,去重后整体上期还款金额var与整体上期账单金额var差值,去重后整体信用卡额度var与整体本期账单余额var差值,去重后整体可用余额var与整体预借现金额度var差值,去重后整体本期账单最低还款额var与整体循环利息var之和
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,360.734470,351.387237,428.696405,457.057503,301.508459,58.0,352.352004,0.000000,0.000000,...,7.079051,71.470209,0.000000,0.000000,0.000000,98.469100,-2.824033,32.567287,-98.469100,145.318143
2,3,36.733542,37.160348,36.723666,36.696541,32.091301,4.0,37.162823,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,261.650351,178.153143,577.987348,678.506333,628.451974,2.0,263.290061,0.000000,0.000000,...,0.111111,78.000994,0.000000,0.000000,18.103521,87.803786,-20.167102,62.381889,-69.700264,0.954005
4,5,41.358453,37.079520,41.328836,41.272092,36.948737,0.0,35.914512,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,803.645436,729.260836,863.847364,805.395108,755.022940,122.0,522.997136,0.000000,0.000000,...,9.378947,92.131896,0.000000,0.000000,117.697134,88.352303,-35.159449,-72.294406,29.344831,17.623604
6,7,1070.586607,2069.566733,2202.883453,594.655816,1141.937370,254.0,1329.699230,0.000000,0.000000,...,7.418357,97.932595,0.000000,0.000000,67.875596,78.664177,-198.968749,-232.767048,-10.788580,56.628128
7,8,0.000000,70.682736,84.279532,84.273040,75.062704,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,321.104320,397.271637,408.343390,384.212244,367.387075,6.0,248.410196,0.000000,0.000000,...,0.316176,111.034824,0.000000,0.000000,0.000000,99.776144,-155.005745,-65.653716,-99.776144,21.106390
9,10,2814.411514,2740.659992,3934.936295,3175.096997,2172.459178,117.0,3039.801281,0.000000,0.000000,...,1.315217,30.777128,0.000000,0.000000,11.223393,88.561135,-40.982678,-79.587599,-77.337742,172.023065


In [23]:
feature.to_csv("../feature/用户账单表特征训练表20170203_整体.csv",index=None,encoding="gb2312")#55596 rows × 241columns 2017.02.03

# 用户账单表初级特征：
55596 rows × 57 columns

In [106]:
#删除指定行ok
#x=d[(d['标签']==0)].index.tolist()
#x=d.drop(x,axis=0)

#==========================================特征工程===============================================#
d=训练信用卡账单表
feature=训练放款时间表
#----------------------------------------放款前特征统计------------------------------------------#

#统计放款前用户上期账单金额值总额以及用户账单金额为负数的情况统计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款前账单金额统计' : 'sum'})
x['放款前账单金额为负数']=x1
x['放款前账单金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

#统计放款前用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款前还款金额统计' : 'sum'})
x['放款前还款金额为负数']=x1
x['放款前还款金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")
feature['放款前账单还款差额']=feature['放款前账单金额统计']-feature['放款前还款金额统计']

#删除0和负等异常值
d1=d[(d['上期账单金额']<=0)].index.tolist()
d=d.drop(d1,axis=0)
d2=d[(d['上期还款金额']<=0)].index.tolist()
d=d.drop(d2,axis=0)
#删除0和负等异常值后的d共1625621行

#按用户标识\时间\银行标识汇总统计

gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
x1=gb['上期账单金额'].agg({'放款前该用户该银行上月账单金额总计' : 'sum','放款前该用户该银行上月账单金额最大值' : 'max'})
x2=gb['上期还款金额'].agg({'放款前该用户该银行上月还款金额总计' : 'sum','放款前该用户该银行还款金额最大值' : 'max'})
x3=gb['消费笔数'].agg({'用户放款前消费笔数最大值' : 'max'})
x4=gb['循环利息'].agg({'用户放款前循环利息最大值' : 'max'})

gb1=x1.groupby(["用户标识"],as_index=False)
gb2=x2.groupby(["用户标识"],as_index=False)
gb3=x3.groupby(["用户标识"],as_index=False)
gb4=x4.groupby(["用户标识"],as_index=False)

x11=gb1['放款前该用户该银行上月账单金额总计'].agg({'放款前该用户账单金额汇总(去重)' : 'sum','放款前该用户账单数(去重)' : 'count'})
x12=gb1['放款前该用户该银行上月账单金额最大值'].agg({'放款前该用户账单金额最大值汇总(去重)' : 'sum'})

x21=gb2['放款前该用户该银行上月还款金额总计'].agg({'放款前该用户账单还款金额汇总(去重)' : 'sum'})
x22=gb2['放款前该用户该银行还款金额最大值'].agg({'放款前该用户账单还款金额最大值汇总(去重)' : 'sum'})

x31=gb3['用户放款前消费笔数最大值'].agg({'用户放款前消费笔数(去重)' : 'sum'})
x41=gb4['用户放款前循环利息最大值'].agg({'用户放款前循环利息(去重)' : 'sum'})

feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature=pd.merge(feature, x12,how='left', on = "用户标识")
feature=pd.merge(feature, x21,how='left', on = "用户标识")
feature=pd.merge(feature, x22,how='left', on = "用户标识")
feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x41,how='left', on = "用户标识")

x=pd.merge(x1, x2,how='inner')
gb3=x[(x['放款前该用户该银行上月账单金额最大值']>x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb4=x[(x['放款前该用户该银行上月账单金额最大值']==x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb5=x[(x['放款前该用户该银行上月账单金额最大值']<x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

x31=gb3['用户标识'].agg({'放款前账单大于还款计数(去重)' : 'count'})
x32=gb4['用户标识'].agg({'放款前账单等于还款计数(去重)' : 'count'})
x33=gb5['用户标识'].agg({'放款前账单小于还款计数(去重)' : 'count'})

feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x32,how='left', on = "用户标识")
feature=pd.merge(feature, x33,how='left', on = "用户标识")

feature['放款前账单汇总还款差额(去重)']=feature['放款前该用户账单金额汇总(去重)']-feature['放款前该用户账单还款金额汇总(去重)']
feature['放款前账单最大值还款差额(去重)']=feature['放款前该用户账单金额最大值汇总(去重)']-feature['放款前该用户账单还款金额最大值汇总(去重)']

#统计放款前用户消费笔数，循环利息总计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['消费笔数'].agg({'用户放款前消费笔数' : 'sum'})
x2=gb['循环利息'].agg({'用户放款前循环利息' : 'sum'})
x3=gb['信用卡额度'].agg({'用户放款前信用卡额度最大值' : 'max'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

#----------------------------------------放款后特征统计------------------------------------------#
d=训练信用卡账单表
#统计放款后用户上期账单金额值总额以及用户账单金额为负数的情况统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款后账单金额统计' : 'sum'})
x['放款后账单金额为负数']=x1
x['放款后账单金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

#统计放款后用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款后还款金额统计' : 'sum'})
x['放款后还款金额为负数']=x1
x['放款后还款金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

feature['放款后账单还款差额']=feature['放款后账单金额统计']-feature['放款后还款金额统计']

#删除0和负等异常值
d1=d[(d['上期账单金额']<=0)].index.tolist()
d=d.drop(d1,axis=0)
d2=d[(d['上期还款金额']<=0)].index.tolist()
d=d.drop(d2,axis=0)
#删除0和负等异常值后的d共1625621行

#按用户标识\时间\银行标识汇总统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
x1=gb['上期账单金额'].agg({'放款后该用户该银行上月账单金额总计' : 'sum','放款后该用户该银行上月账单金额最大值' : 'max'})
x2=gb['上期还款金额'].agg({'放款后该用户该银行上月还款金额总计' : 'sum','放款后该用户该银行还款金额最大值' : 'max'})
x3=gb['消费笔数'].agg({'用户放款后消费笔数最大值' : 'max'})
x4=gb['循环利息'].agg({'用户放款后循环利息最大值' : 'max'})

gb1=x1.groupby(["用户标识"],as_index=False)
gb2=x2.groupby(["用户标识"],as_index=False)
gb3=x3.groupby(["用户标识"],as_index=False)
gb4=x4.groupby(["用户标识"],as_index=False)

x11=gb1['放款后该用户该银行上月账单金额总计'].agg({'放款后该用户账单金额汇总(去重)' : 'sum','放款后该用户账单数(去重)' : 'count'})
x12=gb1['放款后该用户该银行上月账单金额最大值'].agg({'放款后该用户账单金额最大值汇总(去重)' : 'sum'})

x21=gb2['放款后该用户该银行上月还款金额总计'].agg({'放款后该用户账单还款金额汇总(去重)' : 'sum'})
x22=gb2['放款后该用户该银行还款金额最大值'].agg({'放款后该用户账单还款金额最大值汇总(去重)' : 'sum'})

x31=gb3['用户放款后消费笔数最大值'].agg({'用户放款后消费笔数(去重)' : 'sum'})
x41=gb4['用户放款后循环利息最大值'].agg({'用户放款后循环利息(去重)' : 'sum'})

feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature=pd.merge(feature, x12,how='left', on = "用户标识")
feature=pd.merge(feature, x21,how='left', on = "用户标识")
feature=pd.merge(feature, x22,how='left', on = "用户标识")
feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x41,how='left', on = "用户标识")

x=pd.merge(x1, x2,how='inner')
gb3=x[(x['放款后该用户该银行上月账单金额最大值']>x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb4=x[(x['放款后该用户该银行上月账单金额最大值']==x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb5=x[(x['放款后该用户该银行上月账单金额最大值']<x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

x31=gb3['用户标识'].agg({'放款后账单大于还款计数(去重)' : 'count'})
x32=gb4['用户标识'].agg({'放款后账单等于还款计数(去重)' : 'count'})
x33=gb5['用户标识'].agg({'放款后账单小于还款计数(去重)' : 'count'})

feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x32,how='left', on = "用户标识")
feature=pd.merge(feature, x33,how='left', on = "用户标识")

feature['放款后账单汇总还款差额(去重)']=feature['放款后该用户账单金额汇总(去重)']-feature['放款后该用户账单还款金额汇总(去重)']
feature['放款后账单最大值还款差额(去重)']=feature['放款后该用户账单金额最大值汇总(去重)']-feature['放款后该用户账单还款金额最大值汇总(去重)']

#统计放款前用户消费笔数，循环利息总计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['消费笔数'].agg({'用户放款后消费笔数' : 'sum'})
x2=gb['循环利息'].agg({'用户放款后循环利息' : 'sum'})
x3=gb['信用卡额度'].agg({'用户放款后信用卡额度最大值' : 'max'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

#----------------------------------------总体特征统计------------------------------------------#
d=训练信用卡账单表
#爆卡指的是本期账单余额大于信用卡额度
gb=d[(d['信用卡额度']<d['本期账单余额'])].groupby(["用户标识"],as_index=False)
x1=gb['时间'].apply(lambda x:np.unique(x).size)
x=gb['时间'].agg({'爆卡次数' : 'count'})
x['爆卡次数(去重)']=x1
feature=pd.merge(feature, x,how='left', on = "用户标识")

#用户持卡数
gb=d.groupby(["用户标识"],as_index=False)
x=gb['银行标识'].apply(lambda x:np.unique(x).size)
x1=gb['银行标识'].agg({'用户银行卡账单计数' : 'count'})
x1['用户持卡数']=x
feature=pd.merge(feature,x1,how='left', on = "用户标识")

#----------------------------------------老段子的特征------------------------------------------#
d=训练信用卡账单表

#老段子的特征...神了个奇
t1=d[(d['时间']>d['放款时间'])].groupby("用户标识",as_index=False)
t2=d[(d['时间']>d['放款时间']+1)].groupby("用户标识",as_index=False)
t3=d[(d['时间']>d['放款时间']+2)].groupby("用户标识",as_index=False)

x=t1['时间'].apply(lambda x:np.unique(x).size)
x1=t1['时间'].agg({'老段子特征1' : 'count'})
x1['x1']=x

x=t2['时间'].apply(lambda x:np.unique(x).size)
x2=t2['时间'].agg({'老段子特征2' : 'count'})
x2['x2']=x

x=t3['时间'].apply(lambda x:np.unique(x).size)
x3=t3['时间'].agg({'老段子特征3' : 'count'})
x3['x3']=x

t=feature[['用户标识']]
t=pd.merge(t,x1,how='left',on = "用户标识")
t=pd.merge(t,x2,how='left',on = "用户标识")
t=pd.merge(t,x3,how='left',on = "用户标识")
t=t[['用户标识','x1','x2','x3','老段子特征1','老段子特征2','老段子特征3']]

feature=pd.merge(feature, t,how='left', on = "用户标识")

feature['老段子特征x']=(feature['x1']+1)*(feature['x2']+1)*(feature['x3']+1)
#from sklearn.preprocessing import MinMaxScaler
#feature.老段子特征x = MinMaxScaler().fit_transform(feature.老段子特征x)
feature
feature.to_csv("../feature/用户账单表初级特征训练表_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前账单金额统计,放款前账单金额为负数,放款前账单金额为零,放款前还款金额统计,放款前还款金额为负数,放款前还款金额为零,放款前账单还款差额,放款前该用户账单数(去重),...,爆卡次数(去重),用户银行卡账单计数,用户持卡数,x1,x2,x3,老段子特征1,老段子特征2,老段子特征3,老段子特征x
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,154.150417,0.0,2.0,152.796269,0.0,2.0,1.354148,8.0,...,3.0,23.0,3.0,13.0,13.0,13.0,13.0,13.0,13.0,2744.0
2,3,68458,36.733542,0.0,0.0,37.160348,0.0,0.0,-0.426806,1.0,...,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,144.813292,0.0,12.0,105.148045,0.0,14.0,39.665247,6.0,...,14.0,36.0,3.0,14.0,14.0,14.0,16.0,16.0,16.0,3375.0
4,5,68458,41.358453,0.0,0.0,37.079520,0.0,0.0,4.278933,1.0,...,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,803.645436,1.0,0.0,729.260836,0.0,4.0,74.384600,17.0,...,1.0,42.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,68458,703.275617,22.0,10.0,1663.654269,0.0,6.0,-960.378652,17.0,...,NaN,111.0,6.0,18.0,18.0,18.0,21.0,21.0,21.0,6859.0
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,1.0,2.0,2.0,2.0,4.0,4.0,4.0,27.0
8,9,68458,321.104320,2.0,2.0,397.271637,0.0,2.0,-76.167317,14.0,...,4.0,22.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,68458,1937.591928,9.0,32.0,1765.906207,12.0,27.0,171.685721,49.0,...,5.0,200.0,4.0,25.0,25.0,25.0,52.0,52.0,52.0,17576.0


In [107]:
feature.to_csv("../feature/用户账单表初级特征训练表_20170119_A.csv",index=None,encoding="gb2312")

**银行流水记录特征提取:**  
55596 rows × 26 columns

In [36]:
#银行流水记录特征提取

银行流水记录=pd.read_csv("../train/bank_detail_train.txt").rename(index=str,
                                                        columns={"uid": "用户标识","timespan": "流水时间",
                                                                 "type":"交易类型","amount":"交易金额","markup":"工资收入标记"})    
银行流水记录['流水时间']=银行流水记录['流水时间']//86400
银行流水记录.shape#(6070197, 5)
#6031424 rows × 5 columns

(6070197, 5)

In [38]:
银行流水记录[(银行流水记录['流水时间']==0)]#流水时间未知的样本较少，暂时不做处理了

,用户标识,流水时间,交易类型,交易金额,工资收入标记
2869,590,0,0,16.963005,0
2870,590,0,0,15.888777,0
2871,590,0,0,18.485067,0
2872,590,0,0,17.944179,0
2873,590,0,0,18.463221,0
2874,590,0,0,16.496051,0
2875,590,0,0,14.438709,0
2876,590,0,0,18.263467,0
2877,590,0,0,17.639162,0
2878,590,0,0,18.308174,0


In [113]:
#银行流水记录特征提取

#银行流水记录=pd.read_csv("../train/bank_detail_train.txt").rename(index=str,
#                                                        columns={"uid": "用户标识","timespan": "流水时间",
#                                                                 "type":"交易类型","amount":"交易金额","markup":"工资收入标记"})    
#银行流水记录['流水时间']=银行流水记录['流水时间']//86400
#训练银行流水记录表 = pd.merge(银行流水记录,训练放款时间表,how='left',on = "用户标识")
#训练银行流水记录表#6070197行

#训练放款时间表 = pd.read_csv("../train/loan_time_train.txt",header=None,names=['用户标识','放款时间'])
#训练放款时间表['放款时间']=训练放款时间表['放款时间']//86400

#==========================================特征工程===============================================#
feature=训练放款时间表
d=训练银行流水记录表
#----------------------------------------放款前特征统计------------------------------------------#
t=d[(d['流水时间']<=d['放款时间'])]#5684742
gb1=t[(t['交易类型']==0)].groupby(["用户标识"],as_index=False)#收入统计
gb2=t[(t['交易类型']==1)].groupby(["用户标识"],as_index=False)#支出统计
gb3=t[(t['工资收入标记']==1)].groupby(["用户标识"],as_index=False)#工资收入统计
x1=gb1['交易金额'].agg({'放款前用户收入笔数' : 'count','放款前用户收入总计':'sum'})
x2=gb2['交易金额'].agg({'放款前用户支出笔数' : 'count','放款前用户支出总计':'sum'})
x3=gb3['交易金额'].agg({'放款前用户工资收入笔数' : 'count','放款前用户工资收入总计':'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

feature['放款前用户收入支出笔数差值']=feature['放款前用户收入笔数']-feature['放款前用户支出笔数']
feature['放款前用户收入支出总计差值']=feature['放款前用户收入总计']-feature['放款前用户支出总计']
feature['放款前用户非工资收入笔数']=feature['放款前用户收入笔数']-feature['放款前用户工资收入笔数']
feature['放款前用户非工资收入总计']=feature['放款前用户收入总计']-feature['放款前用户工资收入总计']
feature['放款前工资收入笔数乘以差值']=feature['放款前用户工资收入笔数']*feature['放款前用户收入支出笔数差值']
feature['放款前工资收入总计乘以差值']=feature['放款前用户工资收入总计']*feature['放款前用户收入支出总计差值']

#----------------------------------------放款后特征统计------------------------------------------#
t=d[(d['流水时间']>d['放款时间'])]#5684742
gb1=t[(t['交易类型']==0)].groupby(["用户标识"],as_index=False)#收入统计
gb2=t[(t['交易类型']==1)].groupby(["用户标识"],as_index=False)#支出统计
gb3=t[(t['工资收入标记']==1)].groupby(["用户标识"],as_index=False)#工资收入统计
x1=gb1['交易金额'].agg({'放款后用户收入笔数' : 'count','放款后用户收入总计':'sum'})
x2=gb2['交易金额'].agg({'放款后用户支出笔数' : 'count','放款后用户支出总计':'sum'})
x3=gb3['交易金额'].agg({'放款后用户工资收入笔数' : 'count','放款后用户工资收入总计':'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

feature['放款后用户收入支出笔数差值']=feature['放款后用户收入笔数']-feature['放款后用户支出笔数']
feature['放款后用户收入支出总计差值']=feature['放款后用户收入总计']-feature['放款后用户支出总计']
feature['放款后用户非工资收入笔数']=feature['放款后用户收入笔数']-feature['放款后用户工资收入笔数']
feature['放款后用户非工资收入总计']=feature['放款后用户收入总计']-feature['放款后用户工资收入总计']
feature['放款后工资收入笔数乘以差值']=feature['放款后用户工资收入笔数']*feature['放款后用户收入支出笔数差值']
feature['放款后工资收入总计乘以差值']=feature['放款后用户工资收入总计']*feature['放款后用户收入支出总计差值']
feature
feature.to_csv("../feature/用户银行流水记录训练表_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前用户收入笔数,放款前用户收入总计,放款前用户支出总计,放款前用户支出笔数,放款前用户工资收入总计,放款前用户工资收入笔数,放款前用户收入支出笔数差值,放款前用户收入支出总计差值,...,放款后用户支出笔数,放款后用户支出总计,放款后用户工资收入笔数,放款后用户工资收入总计,放款后用户收入支出笔数差值,放款后用户收入支出总计差值,放款后用户非工资收入笔数,放款后用户非工资收入总计,放款后工资收入笔数乘以差值,放款后工资收入总计乘以差值
0,1,68458,35.0,480.692762,676.025269,51.0,NaN,NaN,-16.0,-195.332507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,68458,172.0,2278.873446,4985.957607,507.0,NaN,NaN,-335.0,-2707.084161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,68458,96.0,1164.342384,2129.425722,195.0,NaN,NaN,-99.0,-965.083338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,68458,13.0,186.833251,141.544665,11.0,NaN,NaN,2.0,45.288586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,68458,71.0,895.089300,1268.661289,106.0,NaN,NaN,-35.0,-373.571989,...,77.0,981.631241,NaN,NaN,-7.0,-83.078408,NaN,NaN,NaN,NaN


In [114]:
feature.to_csv("../feature/用户银行流水记录训练表_20170119_A.csv",index=None,encoding="gb2312")

In [121]:
#d= pd.merge(用户浏览行为, 训练放款时间表,how='left', on = "用户标识")#22919547 rows × 5 columns
#用户浏览行为#22919547 rows × 4 columns
#d['放款时间']

**用户浏览行为：**  
55596 rows × 40 columns

In [39]:
用户浏览行为 = pd.read_csv("../train/browse_history_train.txt",header=None,
                    names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
用户浏览行为['浏览时间']=用户浏览行为['浏览时间']//86400
用户浏览行为.shape

(22919547, 4)

In [45]:
用户浏览行为[(用户浏览行为['浏览时间']==0)].shape#浏览时间未知的样本为0，不用特殊处理

(0, 4)

In [124]:
feature=训练放款时间表
d= pd.merge(用户浏览行为, 训练放款时间表,how='left', on = "用户标识")

#----------------------------------------放款前特征统计------------------------------------------#
#统计放款前用户浏览子行为总数以及浏览行为数据总和
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览行为数据'].agg({'放款前浏览行为数据sum' : 'sum','放款前浏览行为数据max' : 'max','放款前浏览行为数据mean' : 'mean'
                    ,'放款前浏览行为数据min' : 'min','放款前浏览行为数据std' : 'std','放款前浏览行为数据var' : 'var'})
xx=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
x2=gb['浏览子行为编号'].agg({'放款前浏览子行为编号count' : 'count'})
x2['放款前浏览子行为编号计数（去重）']=xx

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")

#统计放款前用户浏览子行为个类别统计信息
d=pd.get_dummies(d,columns=d[['浏览子行为编号']])#22919547 rows × 14 columns
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览子行为编号_1'].agg({'放款前浏览子行为编号_1' : 'sum'})
x2=gb['浏览子行为编号_2'].agg({'放款前浏览子行为编号_2' : 'sum'})
x3=gb['浏览子行为编号_3'].agg({'放款前浏览子行为编号_3' : 'sum'})
x4=gb['浏览子行为编号_4'].agg({'放款前浏览子行为编号_4' : 'sum'})
x5=gb['浏览子行为编号_5'].agg({'放款前浏览子行为编号_5' : 'sum'})
x6=gb['浏览子行为编号_6'].agg({'放款前浏览子行为编号_6' : 'sum'})
x7=gb['浏览子行为编号_7'].agg({'放款前浏览子行为编号_7' : 'sum'})
x8=gb['浏览子行为编号_8'].agg({'放款前浏览子行为编号_8' : 'sum'})
x9=gb['浏览子行为编号_9'].agg({'放款前浏览子行为编号_9' : 'sum'})
x10=gb['浏览子行为编号_10'].agg({'放款前浏览子行为编号_10' : 'sum'})
x11=gb['浏览子行为编号_11'].agg({'放款前浏览子行为编号_11' : 'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")
feature=pd.merge(feature, x4,how='left', on = "用户标识")
feature=pd.merge(feature, x5,how='left', on = "用户标识")
feature=pd.merge(feature, x6,how='left', on = "用户标识")
feature=pd.merge(feature, x7,how='left', on = "用户标识")
feature=pd.merge(feature, x8,how='left', on = "用户标识")
feature=pd.merge(feature, x9,how='left', on = "用户标识")
feature=pd.merge(feature, x10,how='left', on = "用户标识")
feature=pd.merge(feature, x11,how='left', on = "用户标识")

d= pd.merge(用户浏览行为, 训练放款时间表,how='left', on = "用户标识")#22919547 rows × 5 columns
#----------------------------------------放款后特征统计------------------------------------------#
#统计放款后用户浏览子行为总数以及浏览行为数据总和
gb=d[(d['浏览时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览行为数据'].agg({'放款后浏览行为数据sum' : 'sum','放款后浏览行为数据max' : 'max','放款后浏览行为数据mean' : 'mean'
                     ,'放款后浏览行为数据min' : 'min','放款后浏览行为数据std' : 'std','放款后浏览行为数据var' : 'var'})
xx=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
x2=gb['浏览子行为编号'].agg({'放款后浏览子行为编号count' : 'count'})
x2['放款后浏览子行为编号计数（去重）']=xx

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")

#统计放款前用户浏览子行为个类别统计信息
d=pd.get_dummies(d,columns=d[['浏览子行为编号']])#22919547 rows × 14 columns
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览子行为编号_1'].agg({'放款后浏览子行为编号_1' : 'sum'})
x2=gb['浏览子行为编号_2'].agg({'放款后浏览子行为编号_2' : 'sum'})
x3=gb['浏览子行为编号_3'].agg({'放款后浏览子行为编号_3' : 'sum'})
x4=gb['浏览子行为编号_4'].agg({'放款后浏览子行为编号_4' : 'sum'})
x5=gb['浏览子行为编号_5'].agg({'放款后浏览子行为编号_5' : 'sum'})
x6=gb['浏览子行为编号_6'].agg({'放款后浏览子行为编号_6' : 'sum'})
x7=gb['浏览子行为编号_7'].agg({'放款后浏览子行为编号_7' : 'sum'})
x8=gb['浏览子行为编号_8'].agg({'放款后浏览子行为编号_8' : 'sum'})
x9=gb['浏览子行为编号_9'].agg({'放款后浏览子行为编号_9' : 'sum'})
x10=gb['浏览子行为编号_10'].agg({'放款后浏览子行为编号_10' : 'sum'})
x11=gb['浏览子行为编号_11'].agg({'放款后浏览子行为编号_11' : 'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")
feature=pd.merge(feature, x4,how='left', on = "用户标识")
feature=pd.merge(feature, x5,how='left', on = "用户标识")
feature=pd.merge(feature, x6,how='left', on = "用户标识")
feature=pd.merge(feature, x7,how='left', on = "用户标识")
feature=pd.merge(feature, x8,how='left', on = "用户标识")
feature=pd.merge(feature, x9,how='left', on = "用户标识")
feature=pd.merge(feature, x10,how='left', on = "用户标识")
feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature
feature.to_csv("../feature/用户浏览行为_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前浏览行为数据var,放款前浏览行为数据mean,放款前浏览行为数据std,放款前浏览行为数据min,放款前浏览行为数据max,放款前浏览行为数据sum,放款前浏览子行为编号count,放款前浏览子行为编号计数（去重）,...,放款后浏览子行为编号_2,放款后浏览子行为编号_3,放款后浏览子行为编号_4,放款后浏览子行为编号_5,放款后浏览子行为编号_6,放款后浏览子行为编号_7,放款后浏览子行为编号_8,放款后浏览子行为编号_9,放款后浏览子行为编号_10,放款后浏览子行为编号_11
0,1,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,68458,3390.796753,116.091954,58.230548,7.0,213.0,151500.0,1305.0,8.0,...,0.0,90.0,150.0,105.0,135.0,105.0,105.0,0.0,75.0,0.0
2,3,68458,2536.790245,112.824561,50.366559,35.0,190.0,38586.0,342.0,6.0,...,0.0,0.0,48.0,30.0,24.0,42.0,0.0,0.0,24.0,0.0
3,4,68458,2942.068840,106.494505,54.240841,35.0,213.0,38764.0,364.0,8.0,...,0.0,4.0,64.0,36.0,32.0,36.0,4.0,0.0,24.0,0.0
4,5,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,68458,0.000000,110.000000,0.000000,110.0,110.0,1650.0,15.0,1.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0
6,7,68458,3021.163081,111.527778,54.965108,15.0,213.0,112420.0,1008.0,7.0,...,0.0,0.0,140.0,98.0,98.0,112.0,14.0,0.0,84.0,0.0
7,8,68458,2859.874409,106.128713,53.477794,7.0,213.0,117909.0,1111.0,8.0,...,0.0,22.0,154.0,66.0,99.0,132.0,11.0,0.0,132.0,0.0
8,9,68458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,68458,3376.252939,113.718750,58.105533,31.0,213.0,36390.0,320.0,8.0,...,0.0,20.0,40.0,20.0,20.0,30.0,40.0,0.0,20.0,0.0


In [125]:
feature.to_csv("../feature/用户浏览行为训练表_20170119_A.csv",index=None,encoding="gb2312")

# 测试集：

In [2]:
测试放款时间表 = pd.read_csv("test/loan_time_test.txt",header=None,names=['用户标识','放款时间'])
测试放款时间表['放款时间']=测试放款时间表['放款时间']//86400

测试用户表 = pd.read_csv("test/user_info_test.txt",header=None,
                    names=['用户标识','用户性别','用户职业','用户教育程度',
                           '用户婚姻状态', '用户户口类型'])

测试信用卡账单表=pd.read_csv("test/bill_detail_test.txt",header=None,
                    names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态'])
测试信用卡账单表['时间']=测试信用卡账单表['时间']//86400

测试信用卡账单表 = pd.merge(测试信用卡账单表, 测试放款时间表,how='inner', on = '用户标识')

测试用户浏览行为 = pd.read_csv("../test/browse_history_test.txt",header=None,
                       names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
测试用户浏览行为['浏览时间']=测试用户浏览行为['浏览时间']//86400

测试银行流水记录=pd.read_csv("../test/bank_detail_test.txt",header=None,
                     names=['用户标识','流水时间','交易类型','交易金额','工资收入标记'])

测试银行流水记录['流水时间']=测试银行流水记录['流水时间']//86400

测试表 = pd.read_csv("test/usersID_test.txt",header=None,
                    names=['用户标识','标签'])

测试表 = pd.merge(测试表,测试用户表,how='inner',on = "用户标识")
测试表 = pd.merge(测试表,测试放款时间表,how='inner',on = "用户标识")

In [151]:
测试用户表 = pd.read_csv("../test/user_info_test.txt",header=None,
                    names=['用户标识','用户性别','用户职业','用户教育程度','用户婚姻状态', '用户户口类型'])

测试表 = pd.read_csv("../test/usersID_test.txt",header=None,names=['用户标识','标签'])

测试表 = pd.merge(测试表,测试用户表,how='inner',on = "用户标识")
测试表.to_csv("../feature/测试表_20170119_A.csv",index=None,encoding="gb2312")

In [24]:
测试信用卡账单表=pd.read_csv("../test/bill_detail_test.txt",header=None,
                    names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态'])
测试信用卡账单表['时间']=测试信用卡账单表['时间']//86400

测试信用卡账单表 = pd.merge(测试信用卡账单表, 测试放款时间表,how='inner', on = '用户标识')



# **测试集集特征构造:**

In [98]:
#feature=测试放款时间表
feature.shape

(13899, 466)

# 补充特征：共13889rows × 466 columns
**放款前：**

In [25]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=测试信用卡账单表[(测试信用卡账单表['时间']>0)]
feature=测试放款时间表
###补充特征，增加 放款前的各种统计信息sum count max min mean std var等
gb=d[(d['时间']<=d['放款时间'])].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

放款前账单sum=gb.sum()
放款前账单sum.columns = ['用户标识', '放款前上期账单金额sum', '放款前上期还款金额sum','放款前信用卡额度sum','放款前本期账单余额sum',
                     '放款前本期账单最低还款额sum','放款前消费笔数sum','放款前本期账单金额sum','放款前调整金额sum','放款前循环利息sum',
                     '放款前可用余额sum','放款前预借现金额度sum','放款前还款状态sum']
feature=pd.merge(feature, 放款前账单sum,how='left', on = "用户标识")
feature['放款前上期还款金额sum与放款前上期账单金额sum差值']=feature['放款前上期还款金额sum']-feature['放款前上期账单金额sum']
feature['放款前信用卡额度sum与放款前本期账单余额sum差值']=feature['放款前信用卡额度sum']-feature['放款前本期账单余额sum']
feature['放款前可用余额sum与放款前预借现金额度sum差值']=feature['放款前可用余额sum']-feature['放款前预借现金额度sum']
feature['放款前本期账单最低还款额sum与放款前循环利息sum之和']=feature['放款前本期账单最低还款额sum']+feature['放款前循环利息sum']

放款前账单count=gb.count()
放款前账单count.columns = ['用户标识', '放款前上期账单金额count', '放款前上期还款金额count','放款前信用卡额度count','放款前本期账单余额count',
                     '放款前本期账单最低还款额count','放款前消费笔数count','放款前本期账单金额count','放款前调整金额count','放款前循环利息count',
                     '放款前可用余额count','放款前预借现金额度count','放款前还款状态count']
feature=pd.merge(feature, 放款前账单count,how='left', on = "用户标识")

放款前账单max=gb.max()
放款前账单max.columns = ['用户标识', '放款前上期账单金额max', '放款前上期还款金额max','放款前信用卡额度max','放款前本期账单余额max',
                     '放款前本期账单最低还款额max','放款前消费笔数max','放款前本期账单金额max','放款前调整金额max','放款前循环利息max',
                     '放款前可用余额max','放款前预借现金额度max','放款前还款状态max']
feature=pd.merge(feature, 放款前账单max,how='left', on = "用户标识")
feature['放款前上期还款金额max与放款前上期账单金额max差值']=feature['放款前上期还款金额max']-feature['放款前上期账单金额max']
feature['放款前信用卡额度max与放款前本期账单余额max差值']=feature['放款前信用卡额度max']-feature['放款前本期账单余额max']
feature['放款前可用余额max与放款前预借现金额度max差值']=feature['放款前可用余额max']-feature['放款前预借现金额度max']
feature['放款前本期账单最低还款额max与放款前循环利息max之和']=feature['放款前本期账单最低还款额max']+feature['放款前循环利息max']

放款前账单min=gb.min()
放款前账单min.columns = ['用户标识', '放款前上期账单金额min', '放款前上期还款金额min','放款前信用卡额度min','放款前本期账单余额min',
                     '放款前本期账单最低还款额min','放款前消费笔数min','放款前本期账单金额min','放款前调整金额min','放款前循环利息min',
                     '放款前可用余额min','放款前预借现金额度min','放款前还款状态min']
feature=pd.merge(feature, 放款前账单min,how='left', on = "用户标识")
feature['放款前上期还款金额min与放款前上期账单金额min差值']=feature['放款前上期还款金额min']-feature['放款前上期账单金额min']
feature['放款前信用卡额度min与放款前本期账单余额min差值']=feature['放款前信用卡额度min']-feature['放款前本期账单余额min']
feature['放款前可用余额min与放款前预借现金额度min差值']=feature['放款前可用余额min']-feature['放款前预借现金额度min']
feature['放款前本期账单最低还款额min与放款前循环利息min之和']=feature['放款前本期账单最低还款额min']+feature['放款前循环利息min']

放款前账单mean=gb.mean()
放款前账单mean.columns = ['用户标识', '放款前上期账单金额mean', '放款前上期还款金额mean','放款前信用卡额度mean','放款前本期账单余额mean',
                     '放款前本期账单最低还款额mean','放款前消费笔数mean','放款前本期账单金额mean','放款前调整金额mean','放款前循环利息mean',
                     '放款前可用余额mean','放款前预借现金额度mean','放款前还款状态mean']
feature=pd.merge(feature, 放款前账单mean,how='left', on = "用户标识")
feature['放款前上期还款金额mean与放款前上期账单金额mean差值']=feature['放款前上期还款金额mean']-feature['放款前上期账单金额mean']
feature['放款前信用卡额度mean与放款前本期账单余额mean差值']=feature['放款前信用卡额度mean']-feature['放款前本期账单余额mean']
feature['放款前可用余额mean与放款前预借现金额度mean差值']=feature['放款前可用余额mean']-feature['放款前预借现金额度mean']
feature['放款前本期账单最低还款额mean与放款前循环利息mean之和']=feature['放款前本期账单最低还款额mean']+feature['放款前循环利息mean']

放款前账单median=gb.median()
放款前账单median.columns = ['用户标识', '放款前上期账单金额median', '放款前上期还款金额median','放款前信用卡额度median','放款前本期账单余额median',
                  '放款前本期账单最低还款额median','放款前消费笔数median','放款前本期账单金额median','放款前调整金额median',
                  '放款前循环利息median','放款前可用余额median','放款前预借现金额度median','放款前还款状态median']
feature=pd.merge(feature, 放款前账单median,how='left', on = "用户标识")
feature['放款前上期还款金额median与放款前上期账单金额median差值']=feature['放款前上期还款金额median']-feature['放款前上期账单金额median']
feature['放款前信用卡额度median与放款前本期账单余额median差值']=feature['放款前信用卡额度median']-feature['放款前本期账单余额median']
feature['放款前可用余额median与放款前预借现金额度median差值']=feature['放款前可用余额median']-feature['放款前预借现金额度median']
feature['放款前本期账单最低还款额median与放款前循环利息median之和']=feature['放款前本期账单最低还款额median']+feature['放款前循环利息median']

放款前账单std=gb.std()
放款前账单std.columns = ['用户标识', '放款前上期账单金额std', '放款前上期还款金额std','放款前信用卡额度std','放款前本期账单余额std',
                     '放款前本期账单最低还款额std','放款前消费笔数std','放款前本期账单金额std','放款前调整金额std','放款前循环利息std',
                     '放款前可用余额std','放款前预借现金额度std','放款前还款状态std']
feature=pd.merge(feature, 放款前账单std,how='left', on = "用户标识")
feature['放款前上期还款金额std与放款前上期账单金额std差值']=feature['放款前上期还款金额std']-feature['放款前上期账单金额std']
feature['放款前信用卡额度std与放款前本期账单余额std差值']=feature['放款前信用卡额度std']-feature['放款前本期账单余额std']
feature['放款前可用余额std与放款前预借现金额度std差值']=feature['放款前可用余额std']-feature['放款前预借现金额度std']
feature['放款前本期账单最低还款额std与放款前循环利息std之和']=feature['放款前本期账单最低还款额std']+feature['放款前循环利息std']

放款前账单var=gb.var()
放款前账单var.columns = ['用户标识', '放款前上期账单金额var', '放款前上期还款金额var','放款前信用卡额度var','放款前本期账单余额var',
                     '放款前本期账单最低还款额var','放款前消费笔数var','放款前本期账单金额var','放款前调整金额var','放款前循环利息var',
                     '放款前可用余额var','放款前预借现金额度var','放款前还款状态var']
feature=pd.merge(feature, 放款前账单var,how='left', on = "用户标识")
feature['放款前上期还款金额var与放款前上期账单金额var差值']=feature['放款前上期还款金额var']-feature['放款前上期账单金额var']
feature['放款前信用卡额度var与放款前本期账单余额var差值']=feature['放款前信用卡额度var']-feature['放款前本期账单余额var']
feature['放款前可用余额var与放款前预借现金额度var差值']=feature['放款前可用余额var']-feature['放款前预借现金额度var']
feature['放款前本期账单最低还款额var与放款前循环利息var之和']=feature['放款前本期账单最低还款额var']+feature['放款前循环利息var']
feature#13899 rows × 126 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,放款前本期账单金额var,放款前调整金额var,放款前循环利息var,放款前可用余额var,放款前预借现金额度var,放款前还款状态var,放款前上期还款金额var与放款前上期账单金额var差值,放款前信用卡额度var与放款前本期账单余额var差值,放款前可用余额var与放款前预借现金额度var差值,放款前本期账单最低还款额var与放款前循环利息var之和
0,55597,68658,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,...,85.711795,0.000000,5.228635,44.787550,71.670644,0.0,-10.628049,69.578198,-26.883094,29.069828
1,55598,68658,964.349747,1021.231984,1118.829861,1188.218789,1089.335474,19.0,1012.540985,0.000000,...,72.882029,0.000000,16.798085,0.000000,97.678443,0.0,-19.391125,1.683402,-97.678443,35.364825
2,55599,68658,107.295517,128.703402,155.916992,169.127169,153.632826,12.0,143.969095,0.000000,...,53.334709,0.000000,0.000000,0.000000,0.000000,0.0,-186.074377,60.988772,0.000000,0.384156
3,55600,68658,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,...,61.999657,0.000000,0.000000,0.000000,60.814804,0.0,2.585524,64.296676,-60.814804,0.754121
4,55601,68658,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.798782,0.000000,1.256742
5,55602,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,...,0.488110,0.000000,0.000000,0.000000,0.000000,0.0,0.354719,0.001154,0.000000,0.071379
7,55604,68658,0.000000,0.000000,79.885084,79.683276,69.086642,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.160151
8,55605,68658,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-2.111352,0.000000,0.077874
9,55606,68658,448.534726,413.873936,282.539107,568.833762,511.378893,50.0,463.153045,0.000000,...,37.419907,0.000000,70.604261,0.000000,75.831101,0.0,-10.773106,105.331785,-75.831101,71.173066


**去重：**

In [26]:
#放款前账单去重统计特征
#d=测试信用卡账单表[(测试信用卡账单表['时间']>0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d[(d['时间']<=d['放款时间'])].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后放款前账单sum=gb.sum()
去重后放款前账单sum.columns = ['用户标识', '去重后放款前上期账单金额sum', '去重后放款前上期还款金额sum','去重后放款前信用卡额度sum'
                       ,'去重后放款前本期账单余额sum','去重后放款前本期账单最低还款额sum','去重后放款前消费笔数sum'
                       ,'去重后放款前本期账单金额sum','去重后放款前调整金额sum','去重后放款前循环利息sum','去重后放款前可用余额sum'
                       ,'去重后放款前预借现金额度sum']
feature=pd.merge(feature, 去重后放款前账单sum,how='left', on = "用户标识")
feature['去重后放款前上期还款金额sum与放款前上期账单金额sum差值']=feature['去重后放款前上期还款金额sum']-feature['去重后放款前上期账单金额sum']
feature['去重后放款前信用卡额度sum与放款前本期账单余额sum差值']=feature['去重后放款前信用卡额度sum']-feature['去重后放款前本期账单余额sum']
feature['去重后放款前可用余额sum与放款前预借现金额度sum差值']=feature['去重后放款前可用余额sum']-feature['去重后放款前预借现金额度sum']
feature['去重后放款前本期账单最低还款额sum与放款前循环利息sum之和']=feature['去重后放款前本期账单最低还款额sum']+feature['去重后放款前循环利息sum']

去重后放款前账单count=gb.count()
去重后放款前账单count.columns = ['用户标识', '去重后放款前上期账单金额count', '去重后放款前上期还款金额count','去重后放款前信用卡额度count'
                         ,'去重后放款前本期账单余额count','去重后放款前本期账单最低还款额count','去重后放款前消费笔数count'
                         ,'去重后放款前本期账单金额count','去重后放款前调整金额count','去重后放款前循环利息count'
                         ,'去重后放款前可用余额count','去重后放款前预借现金额度count']
feature=pd.merge(feature, 去重后放款前账单count,how='left', on = "用户标识")

去重后放款前账单max=gb.max()
去重后放款前账单max.columns = ['用户标识', '去重后放款前上期账单金额max', '去重后放款前上期还款金额max','去重后放款前信用卡额度max'
                       ,'去重后放款前本期账单余额max','去重后放款前本期账单最低还款额max','去重后放款前消费笔数max'
                       ,'去重后放款前本期账单金额max','去重后放款前调整金额max','去重后放款前循环利息max'
                       ,'去重后放款前可用余额max','去重后放款前预借现金额度max']
feature=pd.merge(feature, 去重后放款前账单max,how='left', on = "用户标识")
feature['去重后放款前上期还款金额max与放款前上期账单金额max差值']=feature['去重后放款前上期还款金额max']-feature['去重后放款前上期账单金额max']
feature['去重后放款前信用卡额度max与放款前本期账单余额max差值']=feature['去重后放款前信用卡额度max']-feature['去重后放款前本期账单余额max']
feature['去重后放款前可用余额max与放款前预借现金额度max差值']=feature['去重后放款前可用余额max']-feature['去重后放款前预借现金额度max']
feature['去重后放款前本期账单最低还款额max与放款前循环利息max之和']=feature['去重后放款前本期账单最低还款额max']+feature['去重后放款前循环利息max']

去重后放款前账单min=gb.min()
去重后放款前账单min.columns = ['用户标识', '去重后放款前上期账单金额min', '去重后放款前上期还款金额min','去重后放款前信用卡额度min'
                       ,'去重后放款前本期账单余额min','去重后放款前本期账单最低还款额min','去重后放款前消费笔数min'
                       ,'去重后放款前本期账单金额min','去重后放款前调整金额min','去重后放款前循环利息min'
                       ,'去重后放款前可用余额min','去重后放款前预借现金额度min']
feature=pd.merge(feature, 去重后放款前账单min,how='left', on = "用户标识")
feature['去重后放款前上期还款金额min与放款前上期账单金额min差值']=feature['去重后放款前上期还款金额min']-feature['去重后放款前上期账单金额min']
feature['去重后放款前信用卡额度min与放款前本期账单余额min差值']=feature['去重后放款前信用卡额度min']-feature['去重后放款前本期账单余额min']
feature['去重后放款前可用余额min与放款前预借现金额度min差值']=feature['去重后放款前可用余额min']-feature['去重后放款前预借现金额度min']
feature['去重后放款前本期账单最低还款额min与放款前循环利息min之和']=feature['去重后放款前本期账单最低还款额min']+feature['去重后放款前循环利息min']

去重后放款前账单mean=gb.mean()
去重后放款前账单mean.columns = ['用户标识', '去重后放款前上期账单金额mean', '去重后放款前上期还款金额mean','去重后放款前信用卡额度mean'
                        ,'去重后放款前本期账单余额mean','去重后放款前本期账单最低还款额mean','去重后放款前消费笔数mean'
                        ,'去重后放款前本期账单金额mean','去重后放款前调整金额mean','去重后放款前循环利息mean'
                        ,'去重后放款前可用余额mean','去重后放款前预借现金额度mean']
feature=pd.merge(feature, 去重后放款前账单mean,how='left', on = "用户标识")
feature['去重后放款前上期还款金额mean与放款前上期账单金额mean差值']=feature['去重后放款前上期还款金额mean']-feature['去重后放款前上期账单金额mean']
feature['去重后放款前信用卡额度mean与放款前本期账单余额mean差值']=feature['去重后放款前信用卡额度mean']-feature['去重后放款前本期账单余额mean']
feature['去重后放款前可用余额mean与放款前预借现金额度mean差值']=feature['去重后放款前可用余额mean']-feature['去重后放款前预借现金额度mean']
feature['去重后放款前本期账单最低还款额mean与放款前循环利息mean之和']=feature['去重后放款前本期账单最低还款额mean']+feature['去重后放款前循环利息mean']


去重后放款前账单median=gb.median()
去重后放款前账单median.columns = ['用户标识', '去重后放款前上期账单金额median', '去重后放款前上期还款金额median'
                          ,'去重后放款前信用卡额度median','去重后放款前本期账单余额median','去重后放款前本期账单最低还款额median'
                          ,'去重后放款前消费笔数median','去重后放款前本期账单金额median','去重后放款前调整金额median'
                          ,'去重后放款前循环利息median','去重后放款前可用余额median','去重后放款前预借现金额度median']
feature=pd.merge(feature, 去重后放款前账单median,how='left', on = "用户标识")
feature['去重后放款前上期还款金额median与放款前上期账单金额median差值']=feature['去重后放款前上期还款金额median']-feature['去重后放款前上期账单金额median']
feature['去重后放款前信用卡额度median与放款前本期账单余额median差值']=feature['去重后放款前信用卡额度median']-feature['去重后放款前本期账单余额median']
feature['去重后放款前可用余额median与放款前预借现金额度median差值']=feature['去重后放款前可用余额median']-feature['去重后放款前预借现金额度median']
feature['去重后放款前本期账单最低还款额median与放款前循环利息median之和']=feature['去重后放款前本期账单最低还款额median']+feature['去重后放款前循环利息median']


去重后放款前账单std=gb.std()
去重后放款前账单std.columns = ['用户标识', '去重后放款前上期账单金额std', '去重后放款前上期还款金额std','去重后放款前信用卡额度std'
                       ,'去重后放款前本期账单余额std','去重后放款前本期账单最低还款额std','去重后放款前消费笔数std'
                       ,'去重后放款前本期账单金额std','去重后放款前调整金额std','去重后放款前循环利息std'
                       ,'去重后放款前可用余额std','去重后放款前预借现金额度std']
feature=pd.merge(feature, 去重后放款前账单std,how='left', on = "用户标识")
feature['去重后放款前上期还款金额std与放款前上期账单金额std差值']=feature['去重后放款前上期还款金额std']-feature['去重后放款前上期账单金额std']
feature['去重后放款前信用卡额度std与放款前本期账单余额std差值']=feature['去重后放款前信用卡额度std']-feature['去重后放款前本期账单余额std']
feature['去重后放款前可用余额std与放款前预借现金额度std差值']=feature['去重后放款前可用余额std']-feature['去重后放款前预借现金额度std']
feature['去重后放款前本期账单最低还款额std与放款前循环利息std之和']=feature['去重后放款前本期账单最低还款额std']+feature['去重后放款前循环利息std']

去重后放款前账单var=gb.var()
去重后放款前账单var.columns = ['用户标识', '去重后放款前上期账单金额var', '去重后放款前上期还款金额var','去重后放款前信用卡额度var'
                       ,'去重后放款前本期账单余额var','去重后放款前本期账单最低还款额var','去重后放款前消费笔数var'
                       ,'去重后放款前本期账单金额var','去重后放款前调整金额var','去重后放款前循环利息var','去重后放款前可用余额var','去重后放款前预借现金额度var']
feature=pd.merge(feature, 去重后放款前账单var,how='left', on = "用户标识")
feature['去重后放款前上期还款金额var与放款前上期账单金额var差值']=feature['去重后放款前上期还款金额var']-feature['去重后放款前上期账单金额var']
feature['去重后放款前信用卡额度var与放款前本期账单余额var差值']=feature['去重后放款前信用卡额度var']-feature['去重后放款前本期账单余额var']
feature['去重后放款前可用余额var与放款前预借现金额度var差值']=feature['去重后放款前可用余额var']-feature['去重后放款前预借现金额度var']
feature['去重后放款前本期账单最低还款额var与放款前循环利息var之和']=feature['去重后放款前本期账单最低还款额var']+feature['去重后放款前循环利息var']

feature#13899 rows × 242 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,去重后放款前消费笔数var,去重后放款前本期账单金额var,去重后放款前调整金额var,去重后放款前循环利息var,去重后放款前可用余额var,去重后放款前预借现金额度var,去重后放款前上期还款金额var与放款前上期账单金额var差值,去重后放款前信用卡额度var与放款前本期账单余额var差值,去重后放款前可用余额var与放款前预借现金额度var差值,去重后放款前本期账单最低还款额var与放款前循环利息var之和
0,55597,68658,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,...,1.958567,82.424643,0.000000,5.881882,49.455701,76.543896,-6.552769,77.029255,-27.088195,18.311860
1,55598,68658,964.349747,1021.231984,1118.829861,1188.218789,1089.335474,19.0,1012.540985,0.000000,...,0.669697,72.882029,0.000000,16.798085,0.000000,97.678443,-19.391125,1.683402,-97.678443,35.364825
2,55599,68658,107.295517,128.703402,155.916992,169.127169,153.632826,12.0,143.969095,0.000000,...,1.142857,53.334709,0.000000,0.000000,0.000000,0.000000,-186.074377,60.988772,0.000000,0.384156
3,55600,68658,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,...,0.000000,61.999657,0.000000,0.000000,0.000000,60.814804,2.585524,64.296676,-60.814804,0.754121
4,55601,68658,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.798782,0.000000,1.256742
5,55602,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,...,0.000000,0.488110,0.000000,0.000000,0.000000,0.000000,0.354719,0.001154,0.000000,0.071379
7,55604,68658,0.000000,0.000000,79.885084,79.683276,69.086642,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240226
8,55605,68658,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.111352,0.000000,0.077874
9,55606,68658,448.534726,413.873936,282.539107,568.833762,511.378893,50.0,463.153045,0.000000,...,3.676667,40.917643,0.000000,69.849566,0.000000,81.897589,-11.648373,104.272737,-81.897589,70.383042


# 放款后：

In [27]:
#d=测试信用卡账单表[(测试信用卡账单表['时间']>0)]
feature_beifen=feature
###补充特征，增加 放款后的各种统计信息sum count max min mean std var等
gb=d[(d['时间']>d['放款时间'])].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

放款后账单sum=gb.sum()
放款后账单sum.columns = ['用户标识', '放款后上期账单金额sum', '放款后上期还款金额sum','放款后信用卡额度sum','放款后本期账单余额sum',
                     '放款后本期账单最低还款额sum','放款后消费笔数sum','放款后本期账单金额sum','放款后调整金额sum','放款后循环利息sum',
                     '放款后可用余额sum','放款后预借现金额度sum']
feature=pd.merge(feature, 放款后账单sum,how='left', on = "用户标识")
feature['放款后上期还款金额sum与放款后上期账单金额sum差值']=feature['放款后上期还款金额sum']-feature['放款后上期账单金额sum']
feature['放款后信用卡额度sum与放款后本期账单余额sum差值']=feature['放款后信用卡额度sum']-feature['放款后本期账单余额sum']
feature['放款后可用余额sum与放款后预借现金额度sum差值']=feature['放款后可用余额sum']-feature['放款后预借现金额度sum']
feature['放款后本期账单最低还款额sum与放款后循环利息sum之和']=feature['放款后本期账单最低还款额sum']+feature['放款后循环利息sum']

放款后账单count=gb.count()
放款后账单count.columns = ['用户标识', '放款后上期账单金额count', '放款后上期还款金额count','放款后信用卡额度count','放款后本期账单余额count',
                     '放款后本期账单最低还款额count','放款后消费笔数count','放款后本期账单金额count','放款后调整金额count','放款后循环利息count',
                     '放款后可用余额count','放款后预借现金额度count']
feature=pd.merge(feature, 放款后账单count,how='left', on = "用户标识")

放款后账单max=gb.max()
放款后账单max.columns = ['用户标识', '放款后上期账单金额max', '放款后上期还款金额max','放款后信用卡额度max','放款后本期账单余额max',
                     '放款后本期账单最低还款额max','放款后消费笔数max','放款后本期账单金额max','放款后调整金额max','放款后循环利息max',
                     '放款后可用余额max','放款后预借现金额度max']
feature=pd.merge(feature, 放款后账单max,how='left', on = "用户标识")
feature['放款后上期还款金额max与放款后上期账单金额max差值']=feature['放款后上期还款金额max']-feature['放款后上期账单金额max']
feature['放款后信用卡额度max与放款后本期账单余额max差值']=feature['放款后信用卡额度max']-feature['放款后本期账单余额max']
feature['放款后可用余额max与放款后预借现金额度max差值']=feature['放款后可用余额max']-feature['放款后预借现金额度max']
feature['放款后本期账单最低还款额max与放款后循环利息max之和']=feature['放款后本期账单最低还款额max']+feature['放款后循环利息max']

放款后账单min=gb.min()
放款后账单min.columns = ['用户标识', '放款后上期账单金额min', '放款后上期还款金额min','放款后信用卡额度min','放款后本期账单余额min',
                     '放款后本期账单最低还款额min','放款后消费笔数min','放款后本期账单金额min','放款后调整金额min','放款后循环利息min',
                     '放款后可用余额min','放款后预借现金额度min']
feature=pd.merge(feature, 放款后账单min,how='left', on = "用户标识")
feature['放款后上期还款金额min与放款后上期账单金额min差值']=feature['放款后上期还款金额min']-feature['放款后上期账单金额min']
feature['放款后信用卡额度min与放款后本期账单余额min差值']=feature['放款后信用卡额度min']-feature['放款后本期账单余额min']
feature['放款后可用余额min与放款后预借现金额度min差值']=feature['放款后可用余额min']-feature['放款后预借现金额度min']
feature['放款后本期账单最低还款额min与放款后循环利息min之和']=feature['放款后本期账单最低还款额min']+feature['放款后循环利息min']

放款后账单mean=gb.mean()
放款后账单mean.columns = ['用户标识', '放款后上期账单金额mean', '放款后上期还款金额mean','放款后信用卡额度mean','放款后本期账单余额mean',
                     '放款后本期账单最低还款额mean','放款后消费笔数mean','放款后本期账单金额mean','放款后调整金额mean','放款后循环利息mean',
                     '放款后可用余额mean','放款后预借现金额度mean']
feature=pd.merge(feature, 放款后账单mean,how='left', on = "用户标识")
feature['放款后上期还款金额mean与放款后上期账单金额mean差值']=feature['放款后上期还款金额mean']-feature['放款后上期账单金额mean']
feature['放款后信用卡额度mean与放款后本期账单余额mean差值']=feature['放款后信用卡额度mean']-feature['放款后本期账单余额mean']
feature['放款后可用余额mean与放款后预借现金额度mean差值']=feature['放款后可用余额mean']-feature['放款后预借现金额度mean']
feature['放款后本期账单最低还款额mean与放款后循环利息mean之和']=feature['放款后本期账单最低还款额mean']+feature['放款后循环利息mean']

放款后账单median=gb.median()
放款后账单median.columns = ['用户标识', '放款后上期账单金额median', '放款后上期还款金额median','放款后信用卡额度median','放款后本期账单余额median',
                  '放款后本期账单最低还款额median','放款后消费笔数median','放款后本期账单金额median','放款后调整金额median',
                  '放款后循环利息median','放款后可用余额median','放款后预借现金额度median']
feature=pd.merge(feature, 放款后账单median,how='left', on = "用户标识")
feature['放款后上期还款金额median与放款后上期账单金额median差值']=feature['放款后上期还款金额median']-feature['放款后上期账单金额median']
feature['放款后信用卡额度median与放款后本期账单余额median差值']=feature['放款后信用卡额度median']-feature['放款后本期账单余额median']
feature['放款后可用余额median与放款后预借现金额度median差值']=feature['放款后可用余额median']-feature['放款后预借现金额度median']
feature['放款后本期账单最低还款额median与放款后循环利息median之和']=feature['放款后本期账单最低还款额median']+feature['放款后循环利息median']

放款后账单std=gb.std()
放款后账单std.columns = ['用户标识', '放款后上期账单金额std', '放款后上期还款金额std','放款后信用卡额度std','放款后本期账单余额std',
                     '放款后本期账单最低还款额std','放款后消费笔数std','放款后本期账单金额std','放款后调整金额std','放款后循环利息std',
                     '放款后可用余额std','放款后预借现金额度std']
feature=pd.merge(feature, 放款后账单std,how='left', on = "用户标识")
feature['放款后上期还款金额std与放款后上期账单金额std差值']=feature['放款后上期还款金额std']-feature['放款后上期账单金额std']
feature['放款后信用卡额度std与放款后本期账单余额std差值']=feature['放款后信用卡额度std']-feature['放款后本期账单余额std']
feature['放款后可用余额std与放款后预借现金额度std差值']=feature['放款后可用余额std']-feature['放款后预借现金额度std']
feature['放款后本期账单最低还款额std与放款后循环利息std之和']=feature['放款后本期账单最低还款额std']+feature['放款后循环利息std']

放款后账单var=gb.var()
放款后账单var.columns = ['用户标识', '放款后上期账单金额var', '放款后上期还款金额var','放款后信用卡额度var','放款后本期账单余额var',
                     '放款后本期账单最低还款额var','放款后消费笔数var','放款后本期账单金额var','放款后调整金额var','放款后循环利息var',
                     '放款后可用余额var','放款后预借现金额度var']
feature=pd.merge(feature, 放款后账单var,how='left', on = "用户标识")
feature['放款后上期还款金额var与放款后上期账单金额var差值']=feature['放款后上期还款金额var']-feature['放款后上期账单金额var']
feature['放款后信用卡额度var与放款后本期账单余额var差值']=feature['放款后信用卡额度var']-feature['放款后本期账单余额var']
feature['放款后可用余额var与放款后预借现金额度var差值']=feature['放款后可用余额var']-feature['放款后预借现金额度var']
feature['放款后本期账单最低还款额var与放款后循环利息var之和']=feature['放款后本期账单最低还款额var']+feature['放款后循环利息var']

feature#13899 rows × 358 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,放款后消费笔数var,放款后本期账单金额var,放款后调整金额var,放款后循环利息var,放款后可用余额var,放款后预借现金额度var,放款后上期还款金额var与放款后上期账单金额var差值,放款后信用卡额度var与放款后本期账单余额var差值,放款后可用余额var与放款后预借现金额度var差值,放款后本期账单最低还款额var与放款后循环利息var之和
0,55597,68658,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,68658,964.349747,1021.231984,1118.829861,1188.218789,1089.335474,19.0,1012.540985,0.000000,...,0.250000,113.220132,0.000000,0.000000,0.000000,94.677230,8.326361,-231.130008,-94.677230,78.520088
2,55599,68658,107.295517,128.703402,155.916992,169.127169,153.632826,12.0,143.969095,0.000000,...,2.000000,1.185942,0.000000,0.000000,0.000000,192.366188,0.240555,0.762156,-192.366188,1.946025
3,55600,68658,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,68658,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,68658,0.000000,0.000000,79.885084,79.683276,69.086642,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.318556,0.000000,0.117114
8,55605,68658,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,68658,448.534726,413.873936,282.539107,568.833762,511.378893,50.0,463.153045,0.000000,...,2.250000,0.664839,0.000000,68.450033,0.000000,97.497130,94.798637,97.630746,-97.497130,71.144908


**去重，放款后：**

In [28]:
#放款后账单去重统计特征
#d=测试信用卡账单表[(测试信用卡账单表['时间']>0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d[(d['时间']>d['放款时间'])].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后放款后账单sum=gb.sum()
去重后放款后账单sum.columns = ['用户标识', '去重后放款后上期账单金额sum', '去重后放款后上期还款金额sum','去重后放款后信用卡额度sum'
                       ,'去重后放款后本期账单余额sum','去重后放款后本期账单最低还款额sum','去重后放款后消费笔数sum'
                       ,'去重后放款后本期账单金额sum','去重后放款后调整金额sum','去重后放款后循环利息sum','去重后放款后可用余额sum'
                       ,'去重后放款后预借现金额度sum']
feature=pd.merge(feature, 去重后放款后账单sum,how='left', on = "用户标识")
feature['去重后放款后上期还款金额sum与放款后上期账单金额sum差值']=feature['去重后放款后上期还款金额sum']-feature['去重后放款后上期账单金额sum']
feature['去重后放款后信用卡额度sum与放款后本期账单余额sum差值']=feature['去重后放款后信用卡额度sum']-feature['去重后放款后本期账单余额sum']
feature['去重后放款后可用余额sum与放款后预借现金额度sum差值']=feature['去重后放款后可用余额sum']-feature['去重后放款后预借现金额度sum']
feature['去重后放款后本期账单最低还款额sum与放款后循环利息sum之和']=feature['去重后放款后本期账单最低还款额sum']+feature['去重后放款后循环利息sum']

去重后放款后账单count=gb.count()
去重后放款后账单count.columns = ['用户标识', '去重后放款后上期账单金额count', '去重后放款后上期还款金额count','去重后放款后信用卡额度count'
                         ,'去重后放款后本期账单余额count','去重后放款后本期账单最低还款额count','去重后放款后消费笔数count'
                         ,'去重后放款后本期账单金额count','去重后放款后调整金额count','去重后放款后循环利息count'
                         ,'去重后放款后可用余额count','去重后放款后预借现金额度count']
feature=pd.merge(feature, 去重后放款后账单count,how='left', on = "用户标识")

去重后放款后账单max=gb.max()
去重后放款后账单max.columns = ['用户标识', '去重后放款后上期账单金额max', '去重后放款后上期还款金额max','去重后放款后信用卡额度max'
                       ,'去重后放款后本期账单余额max','去重后放款后本期账单最低还款额max','去重后放款后消费笔数max'
                       ,'去重后放款后本期账单金额max','去重后放款后调整金额max','去重后放款后循环利息max'
                       ,'去重后放款后可用余额max','去重后放款后预借现金额度max']
feature=pd.merge(feature, 去重后放款后账单max,how='left', on = "用户标识")
feature['去重后放款后上期还款金额max与放款后上期账单金额max差值']=feature['去重后放款后上期还款金额max']-feature['去重后放款后上期账单金额max']
feature['去重后放款后信用卡额度max与放款后本期账单余额max差值']=feature['去重后放款后信用卡额度max']-feature['去重后放款后本期账单余额max']
feature['去重后放款后可用余额max与放款后预借现金额度max差值']=feature['去重后放款后可用余额max']-feature['去重后放款后预借现金额度max']
feature['去重后放款后本期账单最低还款额max与放款后循环利息max之和']=feature['去重后放款后本期账单最低还款额max']+feature['去重后放款后循环利息max']

去重后放款后账单min=gb.min()
去重后放款后账单min.columns = ['用户标识', '去重后放款后上期账单金额min', '去重后放款后上期还款金额min','去重后放款后信用卡额度min'
                       ,'去重后放款后本期账单余额min','去重后放款后本期账单最低还款额min','去重后放款后消费笔数min'
                       ,'去重后放款后本期账单金额min','去重后放款后调整金额min','去重后放款后循环利息min'
                       ,'去重后放款后可用余额min','去重后放款后预借现金额度min']
feature=pd.merge(feature, 去重后放款后账单min,how='left', on = "用户标识")
feature['去重后放款后上期还款金额min与放款后上期账单金额min差值']=feature['去重后放款后上期还款金额min']-feature['去重后放款后上期账单金额min']
feature['去重后放款后信用卡额度min与放款后本期账单余额min差值']=feature['去重后放款后信用卡额度min']-feature['去重后放款后本期账单余额min']
feature['去重后放款后可用余额min与放款后预借现金额度min差值']=feature['去重后放款后可用余额min']-feature['去重后放款后预借现金额度min']
feature['去重后放款后本期账单最低还款额min与放款后循环利息min之和']=feature['去重后放款后本期账单最低还款额min']+feature['去重后放款后循环利息min']

去重后放款后账单mean=gb.mean()
去重后放款后账单mean.columns = ['用户标识', '去重后放款后上期账单金额mean', '去重后放款后上期还款金额mean','去重后放款后信用卡额度mean'
                        ,'去重后放款后本期账单余额mean','去重后放款后本期账单最低还款额mean','去重后放款后消费笔数mean'
                        ,'去重后放款后本期账单金额mean','去重后放款后调整金额mean','去重后放款后循环利息mean'
                        ,'去重后放款后可用余额mean','去重后放款后预借现金额度mean']
feature=pd.merge(feature, 去重后放款后账单mean,how='left', on = "用户标识")
feature['去重后放款后上期还款金额mean与放款后上期账单金额mean差值']=feature['去重后放款后上期还款金额mean']-feature['去重后放款后上期账单金额mean']
feature['去重后放款后信用卡额度mean与放款后本期账单余额mean差值']=feature['去重后放款后信用卡额度mean']-feature['去重后放款后本期账单余额mean']
feature['去重后放款后可用余额mean与放款后预借现金额度mean差值']=feature['去重后放款后可用余额mean']-feature['去重后放款后预借现金额度mean']
feature['去重后放款后本期账单最低还款额mean与放款后循环利息mean之和']=feature['去重后放款后本期账单最低还款额mean']+feature['去重后放款后循环利息mean']


去重后放款后账单median=gb.median()
去重后放款后账单median.columns = ['用户标识', '去重后放款后上期账单金额median', '去重后放款后上期还款金额median'
                          ,'去重后放款后信用卡额度median','去重后放款后本期账单余额median','去重后放款后本期账单最低还款额median'
                          ,'去重后放款后消费笔数median','去重后放款后本期账单金额median','去重后放款后调整金额median'
                          ,'去重后放款后循环利息median','去重后放款后可用余额median','去重后放款后预借现金额度median']
feature=pd.merge(feature, 去重后放款后账单median,how='left', on = "用户标识")
feature['去重后放款后上期还款金额median与放款后上期账单金额median差值']=feature['去重后放款后上期还款金额median']-feature['去重后放款后上期账单金额median']
feature['去重后放款后信用卡额度median与放款后本期账单余额median差值']=feature['去重后放款后信用卡额度median']-feature['去重后放款后本期账单余额median']
feature['去重后放款后可用余额median与放款后预借现金额度median差值']=feature['去重后放款后可用余额median']-feature['去重后放款后预借现金额度median']
feature['去重后放款后本期账单最低还款额median与放款后循环利息median之和']=feature['去重后放款后本期账单最低还款额median']+feature['去重后放款后循环利息median']


去重后放款后账单std=gb.std()
去重后放款后账单std.columns = ['用户标识', '去重后放款后上期账单金额std', '去重后放款后上期还款金额std','去重后放款后信用卡额度std'
                       ,'去重后放款后本期账单余额std','去重后放款后本期账单最低还款额std','去重后放款后消费笔数std'
                       ,'去重后放款后本期账单金额std','去重后放款后调整金额std','去重后放款后循环利息std'
                       ,'去重后放款后可用余额std','去重后放款后预借现金额度std']
feature=pd.merge(feature, 去重后放款后账单std,how='left', on = "用户标识")
feature['去重后放款后上期还款金额std与放款后上期账单金额std差值']=feature['去重后放款后上期还款金额std']-feature['去重后放款后上期账单金额std']
feature['去重后放款后信用卡额度std与放款后本期账单余额std差值']=feature['去重后放款后信用卡额度std']-feature['去重后放款后本期账单余额std']
feature['去重后放款后可用余额std与放款后预借现金额度std差值']=feature['去重后放款后可用余额std']-feature['去重后放款后预借现金额度std']
feature['去重后放款后本期账单最低还款额std与放款后循环利息std之和']=feature['去重后放款后本期账单最低还款额std']+feature['去重后放款后循环利息std']

去重后放款后账单var=gb.var()
去重后放款后账单var.columns = ['用户标识', '去重后放款后上期账单金额var', '去重后放款后上期还款金额var','去重后放款后信用卡额度var'
                       ,'去重后放款后本期账单余额var','去重后放款后本期账单最低还款额var','去重后放款后消费笔数var'
                       ,'去重后放款后本期账单金额var','去重后放款后调整金额var','去重后放款后循环利息var','去重后放款后可用余额var','去重后放款后预借现金额度var']
feature=pd.merge(feature, 去重后放款后账单var,how='left', on = "用户标识")
feature['去重后放款后上期还款金额var与放款后上期账单金额var差值']=feature['去重后放款后上期还款金额var']-feature['去重后放款后上期账单金额var']
feature['去重后放款后信用卡额度var与放款后本期账单余额var差值']=feature['去重后放款后信用卡额度var']-feature['去重后放款后本期账单余额var']
feature['去重后放款后可用余额var与放款后预借现金额度var差值']=feature['去重后放款后可用余额var']-feature['去重后放款后预借现金额度var']
feature['去重后放款后本期账单最低还款额var与放款后循环利息var之和']=feature['去重后放款后本期账单最低还款额var']+feature['去重后放款后循环利息var']

feature#13899 rows × 474 columns

,用户标识,放款时间,放款前上期账单金额sum,放款前上期还款金额sum,放款前信用卡额度sum,放款前本期账单余额sum,放款前本期账单最低还款额sum,放款前消费笔数sum,放款前本期账单金额sum,放款前调整金额sum,...,去重后放款后消费笔数var,去重后放款后本期账单金额var,去重后放款后调整金额var,去重后放款后循环利息var,去重后放款后可用余额var,去重后放款后预借现金额度var,去重后放款后上期还款金额var与放款后上期账单金额var差值,去重后放款后信用卡额度var与放款后本期账单余额var差值,去重后放款后可用余额var与放款后预借现金额度var差值,去重后放款后本期账单最低还款额var与放款后循环利息var之和
0,55597,68658,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,68658,964.349747,1021.231984,1118.829861,1188.218789,1089.335474,19.0,1012.540985,0.000000,...,0.250000,113.220132,0.0,0.000000,0.000000,94.677230,8.326361,-231.130008,-94.677230,78.520088
2,55599,68658,107.295517,128.703402,155.916992,169.127169,153.632826,12.0,143.969095,0.000000,...,2.000000,1.185942,0.0,0.000000,0.000000,192.366188,0.240555,0.762156,-192.366188,1.946025
3,55600,68658,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,68658,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,68658,0.000000,0.000000,79.885084,79.683276,69.086642,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.318556,0.000000,0.117114
8,55605,68658,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,68658,448.534726,413.873936,282.539107,568.833762,511.378893,50.0,463.153045,0.000000,...,2.250000,0.664839,0.0,68.450033,0.000000,97.497130,94.798637,97.630746,-97.497130,71.144908


In [29]:
feature.to_csv("../feature/用户账单表特征测试表20170203_时间已知.csv",index=None,encoding="gb2312")

时间未知：

In [30]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=测试信用卡账单表[(测试信用卡账单表['时间']==0)]
feature=测试放款时间表[['用户标识']]
###补充特征，增加 时间未知的各种统计信息sum count max min mean std var等
gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

时间未知账单sum=gb.sum()
时间未知账单sum.columns = ['用户标识', '时间未知上期账单金额sum', '时间未知上期还款金额sum','时间未知信用卡额度sum','时间未知本期账单余额sum',
                     '时间未知本期账单最低还款额sum','时间未知消费笔数sum','时间未知本期账单金额sum','时间未知调整金额sum','时间未知循环利息sum',
                     '时间未知可用余额sum','时间未知预借现金额度sum','时间未知还款状态sum']
feature=pd.merge(feature, 时间未知账单sum,how='left', on = "用户标识")
feature['时间未知上期还款金额sum与时间未知上期账单金额sum差值']=feature['时间未知上期还款金额sum']-feature['时间未知上期账单金额sum']
feature['时间未知信用卡额度sum与时间未知本期账单余额sum差值']=feature['时间未知信用卡额度sum']-feature['时间未知本期账单余额sum']
feature['时间未知可用余额sum与时间未知预借现金额度sum差值']=feature['时间未知可用余额sum']-feature['时间未知预借现金额度sum']
feature['时间未知本期账单最低还款额sum与时间未知循环利息sum之和']=feature['时间未知本期账单最低还款额sum']+feature['时间未知循环利息sum']

时间未知账单count=gb.count()
时间未知账单count.columns = ['用户标识', '时间未知上期账单金额count', '时间未知上期还款金额count','时间未知信用卡额度count','时间未知本期账单余额count',
                     '时间未知本期账单最低还款额count','时间未知消费笔数count','时间未知本期账单金额count','时间未知调整金额count','时间未知循环利息count',
                     '时间未知可用余额count','时间未知预借现金额度count','时间未知还款状态count']
feature=pd.merge(feature, 时间未知账单count,how='left', on = "用户标识")

时间未知账单max=gb.max()
时间未知账单max.columns = ['用户标识', '时间未知上期账单金额max', '时间未知上期还款金额max','时间未知信用卡额度max','时间未知本期账单余额max',
                     '时间未知本期账单最低还款额max','时间未知消费笔数max','时间未知本期账单金额max','时间未知调整金额max','时间未知循环利息max',
                     '时间未知可用余额max','时间未知预借现金额度max','时间未知还款状态max']
feature=pd.merge(feature, 时间未知账单max,how='left', on = "用户标识")
feature['时间未知上期还款金额max与时间未知上期账单金额max差值']=feature['时间未知上期还款金额max']-feature['时间未知上期账单金额max']
feature['时间未知信用卡额度max与时间未知本期账单余额max差值']=feature['时间未知信用卡额度max']-feature['时间未知本期账单余额max']
feature['时间未知可用余额max与时间未知预借现金额度max差值']=feature['时间未知可用余额max']-feature['时间未知预借现金额度max']
feature['时间未知本期账单最低还款额max与时间未知循环利息max之和']=feature['时间未知本期账单最低还款额max']+feature['时间未知循环利息max']

时间未知账单min=gb.min()
时间未知账单min.columns = ['用户标识', '时间未知上期账单金额min', '时间未知上期还款金额min','时间未知信用卡额度min','时间未知本期账单余额min',
                     '时间未知本期账单最低还款额min','时间未知消费笔数min','时间未知本期账单金额min','时间未知调整金额min','时间未知循环利息min',
                     '时间未知可用余额min','时间未知预借现金额度min','时间未知还款状态min']
feature=pd.merge(feature, 时间未知账单min,how='left', on = "用户标识")
feature['时间未知上期还款金额min与时间未知上期账单金额min差值']=feature['时间未知上期还款金额min']-feature['时间未知上期账单金额min']
feature['时间未知信用卡额度min与时间未知本期账单余额min差值']=feature['时间未知信用卡额度min']-feature['时间未知本期账单余额min']
feature['时间未知可用余额min与时间未知预借现金额度min差值']=feature['时间未知可用余额min']-feature['时间未知预借现金额度min']
feature['时间未知本期账单最低还款额min与时间未知循环利息min之和']=feature['时间未知本期账单最低还款额min']+feature['时间未知循环利息min']

时间未知账单mean=gb.mean()
时间未知账单mean.columns = ['用户标识', '时间未知上期账单金额mean', '时间未知上期还款金额mean','时间未知信用卡额度mean','时间未知本期账单余额mean',
                     '时间未知本期账单最低还款额mean','时间未知消费笔数mean','时间未知本期账单金额mean','时间未知调整金额mean','时间未知循环利息mean',
                     '时间未知可用余额mean','时间未知预借现金额度mean','时间未知还款状态mean']
feature=pd.merge(feature, 时间未知账单mean,how='left', on = "用户标识")
feature['时间未知上期还款金额mean与时间未知上期账单金额mean差值']=feature['时间未知上期还款金额mean']-feature['时间未知上期账单金额mean']
feature['时间未知信用卡额度mean与时间未知本期账单余额mean差值']=feature['时间未知信用卡额度mean']-feature['时间未知本期账单余额mean']
feature['时间未知可用余额mean与时间未知预借现金额度mean差值']=feature['时间未知可用余额mean']-feature['时间未知预借现金额度mean']
feature['时间未知本期账单最低还款额mean与时间未知循环利息mean之和']=feature['时间未知本期账单最低还款额mean']+feature['时间未知循环利息mean']

时间未知账单median=gb.median()
时间未知账单median.columns = ['用户标识', '时间未知上期账单金额median', '时间未知上期还款金额median','时间未知信用卡额度median','时间未知本期账单余额median',
                  '时间未知本期账单最低还款额median','时间未知消费笔数median','时间未知本期账单金额median','时间未知调整金额median',
                  '时间未知循环利息median','时间未知可用余额median','时间未知预借现金额度median','时间未知还款状态median']
feature=pd.merge(feature, 时间未知账单median,how='left', on = "用户标识")
feature['时间未知上期还款金额median与时间未知上期账单金额median差值']=feature['时间未知上期还款金额median']-feature['时间未知上期账单金额median']
feature['时间未知信用卡额度median与时间未知本期账单余额median差值']=feature['时间未知信用卡额度median']-feature['时间未知本期账单余额median']
feature['时间未知可用余额median与时间未知预借现金额度median差值']=feature['时间未知可用余额median']-feature['时间未知预借现金额度median']
feature['时间未知本期账单最低还款额median与时间未知循环利息median之和']=feature['时间未知本期账单最低还款额median']+feature['时间未知循环利息median']

时间未知账单std=gb.std()
时间未知账单std.columns = ['用户标识', '时间未知上期账单金额std', '时间未知上期还款金额std','时间未知信用卡额度std','时间未知本期账单余额std',
                     '时间未知本期账单最低还款额std','时间未知消费笔数std','时间未知本期账单金额std','时间未知调整金额std','时间未知循环利息std',
                     '时间未知可用余额std','时间未知预借现金额度std','时间未知还款状态std']
feature=pd.merge(feature, 时间未知账单std,how='left', on = "用户标识")
feature['时间未知上期还款金额std与时间未知上期账单金额std差值']=feature['时间未知上期还款金额std']-feature['时间未知上期账单金额std']
feature['时间未知信用卡额度std与时间未知本期账单余额std差值']=feature['时间未知信用卡额度std']-feature['时间未知本期账单余额std']
feature['时间未知可用余额std与时间未知预借现金额度std差值']=feature['时间未知可用余额std']-feature['时间未知预借现金额度std']
feature['时间未知本期账单最低还款额std与时间未知循环利息std之和']=feature['时间未知本期账单最低还款额std']+feature['时间未知循环利息std']

时间未知账单var=gb.var()
时间未知账单var.columns = ['用户标识', '时间未知上期账单金额var', '时间未知上期还款金额var','时间未知信用卡额度var','时间未知本期账单余额var',
                     '时间未知本期账单最低还款额var','时间未知消费笔数var','时间未知本期账单金额var','时间未知调整金额var','时间未知循环利息var',
                     '时间未知可用余额var','时间未知预借现金额度var','时间未知还款状态var']
feature=pd.merge(feature, 时间未知账单var,how='left', on = "用户标识")
feature['时间未知上期还款金额var与时间未知上期账单金额var差值']=feature['时间未知上期还款金额var']-feature['时间未知上期账单金额var']
feature['时间未知信用卡额度var与时间未知本期账单余额var差值']=feature['时间未知信用卡额度var']-feature['时间未知本期账单余额var']
feature['时间未知可用余额var与时间未知预借现金额度var差值']=feature['时间未知可用余额var']-feature['时间未知预借现金额度var']
feature['时间未知本期账单最低还款额var与时间未知循环利息var之和']=feature['时间未知本期账单最低还款额var']+feature['时间未知循环利息var']
feature#13899 rows × 125 columns

,用户标识,时间未知上期账单金额sum,时间未知上期还款金额sum,时间未知信用卡额度sum,时间未知本期账单余额sum,时间未知本期账单最低还款额sum,时间未知消费笔数sum,时间未知本期账单金额sum,时间未知调整金额sum,时间未知循环利息sum,...,时间未知本期账单金额var,时间未知调整金额var,时间未知循环利息var,时间未知可用余额var,时间未知预借现金额度var,时间未知还款状态var,时间未知上期还款金额var与时间未知上期账单金额var差值,时间未知信用卡额度var与时间未知本期账单余额var差值,时间未知可用余额var与时间未知预借现金额度var差值,时间未知本期账单最低还款额var与时间未知循环利息var之和
0,55597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55599,21.295669,19.060047,22.273856,21.402271,19.099685,1.0,19.778889,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,55605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,266.704161,242.393146,81.765096,287.305203,259.526900,33.0,253.888238,0.0,164.940389,...,5.80032,0.0,59.795904,0.0,85.711763,0.0,-5.138015,91.643308,-85.711763,60.140462


In [31]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#时间未知账单去重统计特征
#d=测试信用卡账单表[(测试信用卡账单表['时间']==0)]
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
              ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
              ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后时间未知账单sum=gb.sum()
去重后时间未知账单sum.columns = ['用户标识', '去重后时间未知上期账单金额sum', '去重后时间未知上期还款金额sum','去重后时间未知信用卡额度sum'
                       ,'去重后时间未知本期账单余额sum','去重后时间未知本期账单最低还款额sum','去重后时间未知消费笔数sum'
                       ,'去重后时间未知本期账单金额sum','去重后时间未知调整金额sum','去重后时间未知循环利息sum','去重后时间未知可用余额sum'
                       ,'去重后时间未知预借现金额度sum']
feature=pd.merge(feature, 去重后时间未知账单sum,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额sum与时间未知上期账单金额sum差值']=feature['去重后时间未知上期还款金额sum']-feature['去重后时间未知上期账单金额sum']
feature['去重后时间未知信用卡额度sum与时间未知本期账单余额sum差值']=feature['去重后时间未知信用卡额度sum']-feature['去重后时间未知本期账单余额sum']
feature['去重后时间未知可用余额sum与时间未知预借现金额度sum差值']=feature['去重后时间未知可用余额sum']-feature['去重后时间未知预借现金额度sum']
feature['去重后时间未知本期账单最低还款额sum与时间未知循环利息sum之和']=feature['去重后时间未知本期账单最低还款额sum']+feature['去重后时间未知循环利息sum']

去重后时间未知账单count=gb.count()
去重后时间未知账单count.columns = ['用户标识', '去重后时间未知上期账单金额count', '去重后时间未知上期还款金额count','去重后时间未知信用卡额度count'
                         ,'去重后时间未知本期账单余额count','去重后时间未知本期账单最低还款额count','去重后时间未知消费笔数count'
                         ,'去重后时间未知本期账单金额count','去重后时间未知调整金额count','去重后时间未知循环利息count'
                         ,'去重后时间未知可用余额count','去重后时间未知预借现金额度count']
feature=pd.merge(feature, 去重后时间未知账单count,how='left', on = "用户标识")

去重后时间未知账单max=gb.max()
去重后时间未知账单max.columns = ['用户标识', '去重后时间未知上期账单金额max', '去重后时间未知上期还款金额max','去重后时间未知信用卡额度max'
                       ,'去重后时间未知本期账单余额max','去重后时间未知本期账单最低还款额max','去重后时间未知消费笔数max'
                       ,'去重后时间未知本期账单金额max','去重后时间未知调整金额max','去重后时间未知循环利息max'
                       ,'去重后时间未知可用余额max','去重后时间未知预借现金额度max']
feature=pd.merge(feature, 去重后时间未知账单max,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额max与时间未知上期账单金额max差值']=feature['去重后时间未知上期还款金额max']-feature['去重后时间未知上期账单金额max']
feature['去重后时间未知信用卡额度max与时间未知本期账单余额max差值']=feature['去重后时间未知信用卡额度max']-feature['去重后时间未知本期账单余额max']
feature['去重后时间未知可用余额max与时间未知预借现金额度max差值']=feature['去重后时间未知可用余额max']-feature['去重后时间未知预借现金额度max']
feature['去重后时间未知本期账单最低还款额max与时间未知循环利息max之和']=feature['去重后时间未知本期账单最低还款额max']+feature['去重后时间未知循环利息max']

去重后时间未知账单min=gb.min()
去重后时间未知账单min.columns = ['用户标识', '去重后时间未知上期账单金额min', '去重后时间未知上期还款金额min','去重后时间未知信用卡额度min'
                       ,'去重后时间未知本期账单余额min','去重后时间未知本期账单最低还款额min','去重后时间未知消费笔数min'
                       ,'去重后时间未知本期账单金额min','去重后时间未知调整金额min','去重后时间未知循环利息min'
                       ,'去重后时间未知可用余额min','去重后时间未知预借现金额度min']
feature=pd.merge(feature, 去重后时间未知账单min,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额min与时间未知上期账单金额min差值']=feature['去重后时间未知上期还款金额min']-feature['去重后时间未知上期账单金额min']
feature['去重后时间未知信用卡额度min与时间未知本期账单余额min差值']=feature['去重后时间未知信用卡额度min']-feature['去重后时间未知本期账单余额min']
feature['去重后时间未知可用余额min与时间未知预借现金额度min差值']=feature['去重后时间未知可用余额min']-feature['去重后时间未知预借现金额度min']
feature['去重后时间未知本期账单最低还款额min与时间未知循环利息min之和']=feature['去重后时间未知本期账单最低还款额min']+feature['去重后时间未知循环利息min']

去重后时间未知账单mean=gb.mean()
去重后时间未知账单mean.columns = ['用户标识', '去重后时间未知上期账单金额mean', '去重后时间未知上期还款金额mean','去重后时间未知信用卡额度mean'
                        ,'去重后时间未知本期账单余额mean','去重后时间未知本期账单最低还款额mean','去重后时间未知消费笔数mean'
                        ,'去重后时间未知本期账单金额mean','去重后时间未知调整金额mean','去重后时间未知循环利息mean'
                        ,'去重后时间未知可用余额mean','去重后时间未知预借现金额度mean']
feature=pd.merge(feature, 去重后时间未知账单mean,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额mean与时间未知上期账单金额mean差值']=feature['去重后时间未知上期还款金额mean']-feature['去重后时间未知上期账单金额mean']
feature['去重后时间未知信用卡额度mean与时间未知本期账单余额mean差值']=feature['去重后时间未知信用卡额度mean']-feature['去重后时间未知本期账单余额mean']
feature['去重后时间未知可用余额mean与时间未知预借现金额度mean差值']=feature['去重后时间未知可用余额mean']-feature['去重后时间未知预借现金额度mean']
feature['去重后时间未知本期账单最低还款额mean与时间未知循环利息mean之和']=feature['去重后时间未知本期账单最低还款额mean']+feature['去重后时间未知循环利息mean']


去重后时间未知账单median=gb.median()
去重后时间未知账单median.columns = ['用户标识', '去重后时间未知上期账单金额median', '去重后时间未知上期还款金额median'
                          ,'去重后时间未知信用卡额度median','去重后时间未知本期账单余额median','去重后时间未知本期账单最低还款额median'
                          ,'去重后时间未知消费笔数median','去重后时间未知本期账单金额median','去重后时间未知调整金额median'
                          ,'去重后时间未知循环利息median','去重后时间未知可用余额median','去重后时间未知预借现金额度median']
feature=pd.merge(feature, 去重后时间未知账单median,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额median与时间未知上期账单金额median差值']=feature['去重后时间未知上期还款金额median']-feature['去重后时间未知上期账单金额median']
feature['去重后时间未知信用卡额度median与时间未知本期账单余额median差值']=feature['去重后时间未知信用卡额度median']-feature['去重后时间未知本期账单余额median']
feature['去重后时间未知可用余额median与时间未知预借现金额度median差值']=feature['去重后时间未知可用余额median']-feature['去重后时间未知预借现金额度median']
feature['去重后时间未知本期账单最低还款额median与时间未知循环利息median之和']=feature['去重后时间未知本期账单最低还款额median']+feature['去重后时间未知循环利息median']


去重后时间未知账单std=gb.std()
去重后时间未知账单std.columns = ['用户标识', '去重后时间未知上期账单金额std', '去重后时间未知上期还款金额std','去重后时间未知信用卡额度std'
                       ,'去重后时间未知本期账单余额std','去重后时间未知本期账单最低还款额std','去重后时间未知消费笔数std'
                       ,'去重后时间未知本期账单金额std','去重后时间未知调整金额std','去重后时间未知循环利息std'
                       ,'去重后时间未知可用余额std','去重后时间未知预借现金额度std']
feature=pd.merge(feature, 去重后时间未知账单std,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额std与时间未知上期账单金额std差值']=feature['去重后时间未知上期还款金额std']-feature['去重后时间未知上期账单金额std']
feature['去重后时间未知信用卡额度std与时间未知本期账单余额std差值']=feature['去重后时间未知信用卡额度std']-feature['去重后时间未知本期账单余额std']
feature['去重后时间未知可用余额std与时间未知预借现金额度std差值']=feature['去重后时间未知可用余额std']-feature['去重后时间未知预借现金额度std']
feature['去重后时间未知本期账单最低还款额std与时间未知循环利息std之和']=feature['去重后时间未知本期账单最低还款额std']+feature['去重后时间未知循环利息std']

去重后时间未知账单var=gb.var()
去重后时间未知账单var.columns = ['用户标识', '去重后时间未知上期账单金额var', '去重后时间未知上期还款金额var','去重后时间未知信用卡额度var'
                       ,'去重后时间未知本期账单余额var','去重后时间未知本期账单最低还款额var','去重后时间未知消费笔数var'
                       ,'去重后时间未知本期账单金额var','去重后时间未知调整金额var','去重后时间未知循环利息var','去重后时间未知可用余额var','去重后时间未知预借现金额度var']
feature=pd.merge(feature, 去重后时间未知账单var,how='left', on = "用户标识")
feature['去重后时间未知上期还款金额var与时间未知上期账单金额var差值']=feature['去重后时间未知上期还款金额var']-feature['去重后时间未知上期账单金额var']
feature['去重后时间未知信用卡额度var与时间未知本期账单余额var差值']=feature['去重后时间未知信用卡额度var']-feature['去重后时间未知本期账单余额var']
feature['去重后时间未知可用余额var与时间未知预借现金额度var差值']=feature['去重后时间未知可用余额var']-feature['去重后时间未知预借现金额度var']
feature['去重后时间未知本期账单最低还款额var与时间未知循环利息var之和']=feature['去重后时间未知本期账单最低还款额var']+feature['去重后时间未知循环利息var']

feature#13899 rows × 241 columns

,用户标识,时间未知上期账单金额sum,时间未知上期还款金额sum,时间未知信用卡额度sum,时间未知本期账单余额sum,时间未知本期账单最低还款额sum,时间未知消费笔数sum,时间未知本期账单金额sum,时间未知调整金额sum,时间未知循环利息sum,...,去重后时间未知消费笔数var,去重后时间未知本期账单金额var,去重后时间未知调整金额var,去重后时间未知循环利息var,去重后时间未知可用余额var,去重后时间未知预借现金额度var,去重后时间未知上期还款金额var与时间未知上期账单金额var差值,去重后时间未知信用卡额度var与时间未知本期账单余额var差值,去重后时间未知可用余额var与时间未知预借现金额度var差值,去重后时间未知本期账单最低还款额var与时间未知循环利息var之和
0,55597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55599,21.295669,19.060047,22.273856,21.402271,19.099685,1.0,19.778889,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,55605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,266.704161,242.393146,81.765096,287.305203,259.526900,33.0,253.888238,0.0,164.940389,...,8.0,0.24926,0.0,136.955867,0.0,194.99426,0.159628,208.872067,-194.99426,137.287197


In [32]:
feature.to_csv("../feature/用户账单表特征测试表20170203_时间未知.csv",index=None,encoding="gb2312")#13899 rows × 241 columns 017.02.03

整体（不区分时间未知）：

In [33]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
d=测试信用卡账单表
feature=测试放款时间表[['用户标识']]
###补充特征，增加 整体的各种统计信息sum count max min mean std var等
gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

整体账单sum=gb.sum()
整体账单sum.columns = ['用户标识', '整体上期账单金额sum', '整体上期还款金额sum','整体信用卡额度sum','整体本期账单余额sum',
                     '整体本期账单最低还款额sum','整体消费笔数sum','整体本期账单金额sum','整体调整金额sum','整体循环利息sum',
                     '整体可用余额sum','整体预借现金额度sum','整体还款状态sum']
feature=pd.merge(feature, 整体账单sum,how='left', on = "用户标识")
feature['整体上期还款金额sum与整体上期账单金额sum差值']=feature['整体上期还款金额sum']-feature['整体上期账单金额sum']
feature['整体信用卡额度sum与整体本期账单余额sum差值']=feature['整体信用卡额度sum']-feature['整体本期账单余额sum']
feature['整体可用余额sum与整体预借现金额度sum差值']=feature['整体可用余额sum']-feature['整体预借现金额度sum']
feature['整体本期账单最低还款额sum与整体循环利息sum之和']=feature['整体本期账单最低还款额sum']+feature['整体循环利息sum']

整体账单count=gb.count()
整体账单count.columns = ['用户标识', '整体上期账单金额count', '整体上期还款金额count','整体信用卡额度count','整体本期账单余额count',
                     '整体本期账单最低还款额count','整体消费笔数count','整体本期账单金额count','整体调整金额count','整体循环利息count',
                     '整体可用余额count','整体预借现金额度count','整体还款状态count']
feature=pd.merge(feature, 整体账单count,how='left', on = "用户标识")

整体账单max=gb.max()
整体账单max.columns = ['用户标识', '整体上期账单金额max', '整体上期还款金额max','整体信用卡额度max','整体本期账单余额max',
                     '整体本期账单最低还款额max','整体消费笔数max','整体本期账单金额max','整体调整金额max','整体循环利息max',
                     '整体可用余额max','整体预借现金额度max','整体还款状态max']
feature=pd.merge(feature, 整体账单max,how='left', on = "用户标识")
feature['整体上期还款金额max与整体上期账单金额max差值']=feature['整体上期还款金额max']-feature['整体上期账单金额max']
feature['整体信用卡额度max与整体本期账单余额max差值']=feature['整体信用卡额度max']-feature['整体本期账单余额max']
feature['整体可用余额max与整体预借现金额度max差值']=feature['整体可用余额max']-feature['整体预借现金额度max']
feature['整体本期账单最低还款额max与整体循环利息max之和']=feature['整体本期账单最低还款额max']+feature['整体循环利息max']

整体账单min=gb.min()
整体账单min.columns = ['用户标识', '整体上期账单金额min', '整体上期还款金额min','整体信用卡额度min','整体本期账单余额min',
                     '整体本期账单最低还款额min','整体消费笔数min','整体本期账单金额min','整体调整金额min','整体循环利息min',
                     '整体可用余额min','整体预借现金额度min','整体还款状态min']
feature=pd.merge(feature, 整体账单min,how='left', on = "用户标识")
feature['整体上期还款金额min与整体上期账单金额min差值']=feature['整体上期还款金额min']-feature['整体上期账单金额min']
feature['整体信用卡额度min与整体本期账单余额min差值']=feature['整体信用卡额度min']-feature['整体本期账单余额min']
feature['整体可用余额min与整体预借现金额度min差值']=feature['整体可用余额min']-feature['整体预借现金额度min']
feature['整体本期账单最低还款额min与整体循环利息min之和']=feature['整体本期账单最低还款额min']+feature['整体循环利息min']

整体账单mean=gb.mean()
整体账单mean.columns = ['用户标识', '整体上期账单金额mean', '整体上期还款金额mean','整体信用卡额度mean','整体本期账单余额mean',
                     '整体本期账单最低还款额mean','整体消费笔数mean','整体本期账单金额mean','整体调整金额mean','整体循环利息mean',
                     '整体可用余额mean','整体预借现金额度mean','整体还款状态mean']
feature=pd.merge(feature, 整体账单mean,how='left', on = "用户标识")
feature['整体上期还款金额mean与整体上期账单金额mean差值']=feature['整体上期还款金额mean']-feature['整体上期账单金额mean']
feature['整体信用卡额度mean与整体本期账单余额mean差值']=feature['整体信用卡额度mean']-feature['整体本期账单余额mean']
feature['整体可用余额mean与整体预借现金额度mean差值']=feature['整体可用余额mean']-feature['整体预借现金额度mean']
feature['整体本期账单最低还款额mean与整体循环利息mean之和']=feature['整体本期账单最低还款额mean']+feature['整体循环利息mean']

整体账单median=gb.median()
整体账单median.columns = ['用户标识', '整体上期账单金额median', '整体上期还款金额median','整体信用卡额度median','整体本期账单余额median',
                  '整体本期账单最低还款额median','整体消费笔数median','整体本期账单金额median','整体调整金额median',
                  '整体循环利息median','整体可用余额median','整体预借现金额度median','整体还款状态median']
feature=pd.merge(feature, 整体账单median,how='left', on = "用户标识")
feature['整体上期还款金额median与整体上期账单金额median差值']=feature['整体上期还款金额median']-feature['整体上期账单金额median']
feature['整体信用卡额度median与整体本期账单余额median差值']=feature['整体信用卡额度median']-feature['整体本期账单余额median']
feature['整体可用余额median与整体预借现金额度median差值']=feature['整体可用余额median']-feature['整体预借现金额度median']
feature['整体本期账单最低还款额median与整体循环利息median之和']=feature['整体本期账单最低还款额median']+feature['整体循环利息median']

整体账单std=gb.std()
整体账单std.columns = ['用户标识', '整体上期账单金额std', '整体上期还款金额std','整体信用卡额度std','整体本期账单余额std',
                     '整体本期账单最低还款额std','整体消费笔数std','整体本期账单金额std','整体调整金额std','整体循环利息std',
                     '整体可用余额std','整体预借现金额度std','整体还款状态std']
feature=pd.merge(feature, 整体账单std,how='left', on = "用户标识")
feature['整体上期还款金额std与整体上期账单金额std差值']=feature['整体上期还款金额std']-feature['整体上期账单金额std']
feature['整体信用卡额度std与整体本期账单余额std差值']=feature['整体信用卡额度std']-feature['整体本期账单余额std']
feature['整体可用余额std与整体预借现金额度std差值']=feature['整体可用余额std']-feature['整体预借现金额度std']
feature['整体本期账单最低还款额std与整体循环利息std之和']=feature['整体本期账单最低还款额std']+feature['整体循环利息std']

整体账单var=gb.var()
整体账单var.columns = ['用户标识', '整体上期账单金额var', '整体上期还款金额var','整体信用卡额度var','整体本期账单余额var',
                     '整体本期账单最低还款额var','整体消费笔数var','整体本期账单金额var','整体调整金额var','整体循环利息var',
                     '整体可用余额var','整体预借现金额度var','整体还款状态var']
feature=pd.merge(feature, 整体账单var,how='left', on = "用户标识")
feature['整体上期还款金额var与整体上期账单金额var差值']=feature['整体上期还款金额var']-feature['整体上期账单金额var']
feature['整体信用卡额度var与整体本期账单余额var差值']=feature['整体信用卡额度var']-feature['整体本期账单余额var']
feature['整体可用余额var与整体预借现金额度var差值']=feature['整体可用余额var']-feature['整体预借现金额度var']
feature['整体本期账单最低还款额var与整体循环利息var之和']=feature['整体本期账单最低还款额var']+feature['整体循环利息var']
feature#13899 rows × 125 columns

,用户标识,整体上期账单金额sum,整体上期还款金额sum,整体信用卡额度sum,整体本期账单余额sum,整体本期账单最低还款额sum,整体消费笔数sum,整体本期账单金额sum,整体调整金额sum,整体循环利息sum,...,整体本期账单金额var,整体调整金额var,整体循环利息var,整体可用余额var,整体预借现金额度var,整体还款状态var,整体上期还款金额var与整体上期账单金额var差值,整体信用卡额度var与整体本期账单余额var差值,整体可用余额var与整体预借现金额度var差值,整体本期账单最低还款额var与整体循环利息var之和
0,55597,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,32.176404,...,85.711795,0.000000,5.228635,44.787550,71.670644,0.0,-10.628049,69.578198,-26.883094,29.069828
1,55598,1026.146745,1085.393735,1160.436328,1230.251143,1142.432029,20.0,1076.364351,0.000000,84.287950,...,73.600225,0.000000,15.913428,0.000000,97.042863,0.0,-17.925773,-9.188706,-97.042863,37.388331
2,55599,150.329730,169.513056,220.772447,232.686519,210.922443,15.0,205.915028,0.000000,0.000000,...,39.101444,0.000000,0.000000,0.000000,34.975671,0.0,-131.851617,43.814154,-34.975671,0.466037
3,55600,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,0.000000,...,61.999657,0.000000,0.000000,0.000000,60.814804,0.0,2.585524,64.296676,-60.814804,0.754121
4,55601,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.798782,0.000000,1.256742
5,55602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,0.000000,...,0.488110,0.000000,0.000000,0.000000,0.000000,0.0,0.354719,0.001154,0.000000,0.071379
7,55604,0.000000,0.000000,119.827626,117.673563,103.702249,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.492211,0.000000,0.119862
8,55605,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-2.111352,0.000000,0.077874
9,55606,795.977423,714.607595,424.513665,934.471787,841.882208,86.0,793.047971,0.000000,412.518219,...,25.000780,0.000000,69.057434,0.000000,76.867302,0.0,-2.923761,103.170418,-76.867302,69.723776


In [34]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#整体账单去重统计特征
#d=测试信用卡账单表
feature_beifen=feature
#按用户标识\时间\银行标识汇总统计（去重）
data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
              ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
              ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后整体账单sum=gb.sum()
去重后整体账单sum.columns = ['用户标识', '去重后整体上期账单金额sum', '去重后整体上期还款金额sum','去重后整体信用卡额度sum'
                       ,'去重后整体本期账单余额sum','去重后整体本期账单最低还款额sum','去重后整体消费笔数sum'
                       ,'去重后整体本期账单金额sum','去重后整体调整金额sum','去重后整体循环利息sum','去重后整体可用余额sum'
                       ,'去重后整体预借现金额度sum']
feature=pd.merge(feature, 去重后整体账单sum,how='left', on = "用户标识")
feature['去重后整体上期还款金额sum与整体上期账单金额sum差值']=feature['去重后整体上期还款金额sum']-feature['去重后整体上期账单金额sum']
feature['去重后整体信用卡额度sum与整体本期账单余额sum差值']=feature['去重后整体信用卡额度sum']-feature['去重后整体本期账单余额sum']
feature['去重后整体可用余额sum与整体预借现金额度sum差值']=feature['去重后整体可用余额sum']-feature['去重后整体预借现金额度sum']
feature['去重后整体本期账单最低还款额sum与整体循环利息sum之和']=feature['去重后整体本期账单最低还款额sum']+feature['去重后整体循环利息sum']

去重后整体账单count=gb.count()
去重后整体账单count.columns = ['用户标识', '去重后整体上期账单金额count', '去重后整体上期还款金额count','去重后整体信用卡额度count'
                         ,'去重后整体本期账单余额count','去重后整体本期账单最低还款额count','去重后整体消费笔数count'
                         ,'去重后整体本期账单金额count','去重后整体调整金额count','去重后整体循环利息count'
                         ,'去重后整体可用余额count','去重后整体预借现金额度count']
feature=pd.merge(feature, 去重后整体账单count,how='left', on = "用户标识")

去重后整体账单max=gb.max()
去重后整体账单max.columns = ['用户标识', '去重后整体上期账单金额max', '去重后整体上期还款金额max','去重后整体信用卡额度max'
                       ,'去重后整体本期账单余额max','去重后整体本期账单最低还款额max','去重后整体消费笔数max'
                       ,'去重后整体本期账单金额max','去重后整体调整金额max','去重后整体循环利息max'
                       ,'去重后整体可用余额max','去重后整体预借现金额度max']
feature=pd.merge(feature, 去重后整体账单max,how='left', on = "用户标识")
feature['去重后整体上期还款金额max与整体上期账单金额max差值']=feature['去重后整体上期还款金额max']-feature['去重后整体上期账单金额max']
feature['去重后整体信用卡额度max与整体本期账单余额max差值']=feature['去重后整体信用卡额度max']-feature['去重后整体本期账单余额max']
feature['去重后整体可用余额max与整体预借现金额度max差值']=feature['去重后整体可用余额max']-feature['去重后整体预借现金额度max']
feature['去重后整体本期账单最低还款额max与整体循环利息max之和']=feature['去重后整体本期账单最低还款额max']+feature['去重后整体循环利息max']

去重后整体账单min=gb.min()
去重后整体账单min.columns = ['用户标识', '去重后整体上期账单金额min', '去重后整体上期还款金额min','去重后整体信用卡额度min'
                       ,'去重后整体本期账单余额min','去重后整体本期账单最低还款额min','去重后整体消费笔数min'
                       ,'去重后整体本期账单金额min','去重后整体调整金额min','去重后整体循环利息min'
                       ,'去重后整体可用余额min','去重后整体预借现金额度min']
feature=pd.merge(feature, 去重后整体账单min,how='left', on = "用户标识")
feature['去重后整体上期还款金额min与整体上期账单金额min差值']=feature['去重后整体上期还款金额min']-feature['去重后整体上期账单金额min']
feature['去重后整体信用卡额度min与整体本期账单余额min差值']=feature['去重后整体信用卡额度min']-feature['去重后整体本期账单余额min']
feature['去重后整体可用余额min与整体预借现金额度min差值']=feature['去重后整体可用余额min']-feature['去重后整体预借现金额度min']
feature['去重后整体本期账单最低还款额min与整体循环利息min之和']=feature['去重后整体本期账单最低还款额min']+feature['去重后整体循环利息min']

去重后整体账单mean=gb.mean()
去重后整体账单mean.columns = ['用户标识', '去重后整体上期账单金额mean', '去重后整体上期还款金额mean','去重后整体信用卡额度mean'
                        ,'去重后整体本期账单余额mean','去重后整体本期账单最低还款额mean','去重后整体消费笔数mean'
                        ,'去重后整体本期账单金额mean','去重后整体调整金额mean','去重后整体循环利息mean'
                        ,'去重后整体可用余额mean','去重后整体预借现金额度mean']
feature=pd.merge(feature, 去重后整体账单mean,how='left', on = "用户标识")
feature['去重后整体上期还款金额mean与整体上期账单金额mean差值']=feature['去重后整体上期还款金额mean']-feature['去重后整体上期账单金额mean']
feature['去重后整体信用卡额度mean与整体本期账单余额mean差值']=feature['去重后整体信用卡额度mean']-feature['去重后整体本期账单余额mean']
feature['去重后整体可用余额mean与整体预借现金额度mean差值']=feature['去重后整体可用余额mean']-feature['去重后整体预借现金额度mean']
feature['去重后整体本期账单最低还款额mean与整体循环利息mean之和']=feature['去重后整体本期账单最低还款额mean']+feature['去重后整体循环利息mean']


去重后整体账单median=gb.median()
去重后整体账单median.columns = ['用户标识', '去重后整体上期账单金额median', '去重后整体上期还款金额median'
                          ,'去重后整体信用卡额度median','去重后整体本期账单余额median','去重后整体本期账单最低还款额median'
                          ,'去重后整体消费笔数median','去重后整体本期账单金额median','去重后整体调整金额median'
                          ,'去重后整体循环利息median','去重后整体可用余额median','去重后整体预借现金额度median']
feature=pd.merge(feature, 去重后整体账单median,how='left', on = "用户标识")
feature['去重后整体上期还款金额median与整体上期账单金额median差值']=feature['去重后整体上期还款金额median']-feature['去重后整体上期账单金额median']
feature['去重后整体信用卡额度median与整体本期账单余额median差值']=feature['去重后整体信用卡额度median']-feature['去重后整体本期账单余额median']
feature['去重后整体可用余额median与整体预借现金额度median差值']=feature['去重后整体可用余额median']-feature['去重后整体预借现金额度median']
feature['去重后整体本期账单最低还款额median与整体循环利息median之和']=feature['去重后整体本期账单最低还款额median']+feature['去重后整体循环利息median']


去重后整体账单std=gb.std()
去重后整体账单std.columns = ['用户标识', '去重后整体上期账单金额std', '去重后整体上期还款金额std','去重后整体信用卡额度std'
                       ,'去重后整体本期账单余额std','去重后整体本期账单最低还款额std','去重后整体消费笔数std'
                       ,'去重后整体本期账单金额std','去重后整体调整金额std','去重后整体循环利息std'
                       ,'去重后整体可用余额std','去重后整体预借现金额度std']
feature=pd.merge(feature, 去重后整体账单std,how='left', on = "用户标识")
feature['去重后整体上期还款金额std与整体上期账单金额std差值']=feature['去重后整体上期还款金额std']-feature['去重后整体上期账单金额std']
feature['去重后整体信用卡额度std与整体本期账单余额std差值']=feature['去重后整体信用卡额度std']-feature['去重后整体本期账单余额std']
feature['去重后整体可用余额std与整体预借现金额度std差值']=feature['去重后整体可用余额std']-feature['去重后整体预借现金额度std']
feature['去重后整体本期账单最低还款额std与整体循环利息std之和']=feature['去重后整体本期账单最低还款额std']+feature['去重后整体循环利息std']

去重后整体账单var=gb.var()
去重后整体账单var.columns = ['用户标识', '去重后整体上期账单金额var', '去重后整体上期还款金额var','去重后整体信用卡额度var'
                       ,'去重后整体本期账单余额var','去重后整体本期账单最低还款额var','去重后整体消费笔数var'
                       ,'去重后整体本期账单金额var','去重后整体调整金额var','去重后整体循环利息var','去重后整体可用余额var','去重后整体预借现金额度var']
feature=pd.merge(feature, 去重后整体账单var,how='left', on = "用户标识")
feature['去重后整体上期还款金额var与整体上期账单金额var差值']=feature['去重后整体上期还款金额var']-feature['去重后整体上期账单金额var']
feature['去重后整体信用卡额度var与整体本期账单余额var差值']=feature['去重后整体信用卡额度var']-feature['去重后整体本期账单余额var']
feature['去重后整体可用余额var与整体预借现金额度var差值']=feature['去重后整体可用余额var']-feature['去重后整体预借现金额度var']
feature['去重后整体本期账单最低还款额var与整体循环利息var之和']=feature['去重后整体本期账单最低还款额var']+feature['去重后整体循环利息var']

feature#13899 rows × 241 columns

,用户标识,整体上期账单金额sum,整体上期还款金额sum,整体信用卡额度sum,整体本期账单余额sum,整体本期账单最低还款额sum,整体消费笔数sum,整体本期账单金额sum,整体调整金额sum,整体循环利息sum,...,去重后整体消费笔数var,去重后整体本期账单金额var,去重后整体调整金额var,去重后整体循环利息var,去重后整体可用余额var,去重后整体预借现金额度var,去重后整体上期还款金额var与整体上期账单金额var差值,去重后整体信用卡额度var与整体本期账单余额var差值,去重后整体可用余额var与整体预借现金额度var差值,去重后整体本期账单最低还款额var与整体循环利息var之和
0,55597,1040.964295,849.318939,1109.390792,1827.865051,1667.506229,65.0,974.624595,0.000000,32.176404,...,1.958567,82.424643,0.000000,5.881882,49.455701,76.543896,-6.552769,77.029255,-27.088195,18.311860
1,55598,1026.146745,1085.393735,1160.436328,1230.251143,1142.432029,20.0,1076.364351,0.000000,84.287950,...,0.641822,73.600225,0.000000,15.913428,0.000000,97.042863,-17.925773,-9.188706,-97.042863,37.388331
2,55599,150.329730,169.513056,220.772447,232.686519,210.922443,15.0,205.915028,0.000000,0.000000,...,1.054545,39.101444,0.000000,0.000000,0.000000,34.975671,-131.851617,43.814154,-34.975671,0.466037
3,55600,151.853337,154.238780,179.741439,210.519668,195.046352,0.0,173.709292,0.000000,0.000000,...,0.000000,61.999657,0.000000,0.000000,0.000000,60.814804,2.585524,64.296676,-60.814804,0.754121
4,55601,0.000000,0.000000,0.000000,56.473385,52.439858,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.798782,0.000000,1.256742
5,55602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,190.600997,176.207679,190.996086,190.811063,170.535659,0.0,176.639140,0.000000,0.000000,...,0.000000,0.488110,0.000000,0.000000,0.000000,0.000000,0.354719,0.001154,0.000000,0.071379
7,55604,0.000000,0.000000,119.827626,117.673563,103.702249,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.725144,0.000000,0.119549
8,55605,0.000000,0.000000,0.000000,34.668544,31.474758,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.111352,0.000000,0.077874
9,55606,795.977423,714.607595,424.513665,934.471787,841.882208,86.0,793.047971,0.000000,412.518219,...,3.522581,34.361121,0.000000,68.540606,0.000000,83.029814,-2.380563,101.854794,-83.029814,69.292192


In [35]:
feature.to_csv("../feature/用户账单表特征测试表20170203_整体.csv",index=None,encoding="gb2312")#13899 rows × 241 columns 017.02.03

# 用户账单表初始特征:
13899 rows × 57 columns

In [108]:
#删除指定行ok
#x=d[(d['标签']==0)].index.tolist()
#x=d.drop(x,axis=0)

#==========================================特征工程===============================================#
d=测试信用卡账单表
feature=测试放款时间表
#----------------------------------------放款前特征统计------------------------------------------#

#统计放款前用户上期账单金额值总额以及用户账单金额为负数的情况统计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款前账单金额统计' : 'sum'})
x['放款前账单金额为负数']=x1
x['放款前账单金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

#统计放款前用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款前还款金额统计' : 'sum'})
x['放款前还款金额为负数']=x1
x['放款前还款金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")
feature['放款前账单还款差额']=feature['放款前账单金额统计']-feature['放款前还款金额统计']

#删除0和负等异常值
d1=d[(d['上期账单金额']<=0)].index.tolist()
d=d.drop(d1,axis=0)
d2=d[(d['上期还款金额']<=0)].index.tolist()
d=d.drop(d2,axis=0)
#删除0和负等异常值后的d共1625621行

#按用户标识\时间\银行标识汇总统计

gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
x1=gb['上期账单金额'].agg({'放款前该用户该银行上月账单金额总计' : 'sum','放款前该用户该银行上月账单金额最大值' : 'max'})
x2=gb['上期还款金额'].agg({'放款前该用户该银行上月还款金额总计' : 'sum','放款前该用户该银行还款金额最大值' : 'max'})
x3=gb['消费笔数'].agg({'用户放款前消费笔数最大值' : 'max'})
x4=gb['循环利息'].agg({'用户放款前循环利息最大值' : 'max'})

gb1=x1.groupby(["用户标识"],as_index=False)
gb2=x2.groupby(["用户标识"],as_index=False)
gb3=x3.groupby(["用户标识"],as_index=False)
gb4=x4.groupby(["用户标识"],as_index=False)

x11=gb1['放款前该用户该银行上月账单金额总计'].agg({'放款前该用户账单金额汇总(去重)' : 'sum','放款前该用户账单数(去重)' : 'count'})
x12=gb1['放款前该用户该银行上月账单金额最大值'].agg({'放款前该用户账单金额最大值汇总(去重)' : 'sum'})

x21=gb2['放款前该用户该银行上月还款金额总计'].agg({'放款前该用户账单还款金额汇总(去重)' : 'sum'})
x22=gb2['放款前该用户该银行还款金额最大值'].agg({'放款前该用户账单还款金额最大值汇总(去重)' : 'sum'})

x31=gb3['用户放款前消费笔数最大值'].agg({'用户放款前消费笔数(去重)' : 'sum'})
x41=gb4['用户放款前循环利息最大值'].agg({'用户放款前循环利息(去重)' : 'sum'})

feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature=pd.merge(feature, x12,how='left', on = "用户标识")
feature=pd.merge(feature, x21,how='left', on = "用户标识")
feature=pd.merge(feature, x22,how='left', on = "用户标识")
feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x41,how='left', on = "用户标识")

x=pd.merge(x1, x2,how='inner')
gb3=x[(x['放款前该用户该银行上月账单金额最大值']>x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb4=x[(x['放款前该用户该银行上月账单金额最大值']==x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb5=x[(x['放款前该用户该银行上月账单金额最大值']<x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

x31=gb3['用户标识'].agg({'放款前账单大于还款计数(去重)' : 'count'})
x32=gb4['用户标识'].agg({'放款前账单等于还款计数(去重)' : 'count'})
x33=gb5['用户标识'].agg({'放款前账单小于还款计数(去重)' : 'count'})

feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x32,how='left', on = "用户标识")
feature=pd.merge(feature, x33,how='left', on = "用户标识")

feature['放款前账单汇总还款差额(去重)']=feature['放款前该用户账单金额汇总(去重)']-feature['放款前该用户账单还款金额汇总(去重)']
feature['放款前账单最大值还款差额(去重)']=feature['放款前该用户账单金额最大值汇总(去重)']-feature['放款前该用户账单还款金额最大值汇总(去重)']

#统计放款前用户消费笔数，循环利息总计
gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['消费笔数'].agg({'用户放款前消费笔数' : 'sum'})
x2=gb['循环利息'].agg({'用户放款前循环利息' : 'sum'})
x3=gb['信用卡额度'].agg({'用户放款前信用卡额度最大值' : 'max'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

#----------------------------------------放款后特征统计------------------------------------------#
d=测试信用卡账单表
#统计放款后用户上期账单金额值总额以及用户账单金额为负数的情况统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款后账单金额统计' : 'sum'})
x['放款后账单金额为负数']=x1
x['放款后账单金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

#统计放款后用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
x1=gb.apply(lambda x:x.where(x<0).count())
x2=gb.apply(lambda x:x.where(x==0.000000).count())
x=gb.agg({'放款后还款金额统计' : 'sum'})
x['放款后还款金额为负数']=x1
x['放款后还款金额为零']=x2

feature=pd.merge(feature, x,how='left', on = "用户标识")

feature['放款后账单还款差额']=feature['放款后账单金额统计']-feature['放款后还款金额统计']

#删除0和负等异常值
d1=d[(d['上期账单金额']<=0)].index.tolist()
d=d.drop(d1,axis=0)
d2=d[(d['上期还款金额']<=0)].index.tolist()
d=d.drop(d2,axis=0)
#删除0和负等异常值后的d共1625621行

#按用户标识\时间\银行标识汇总统计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
x1=gb['上期账单金额'].agg({'放款后该用户该银行上月账单金额总计' : 'sum','放款后该用户该银行上月账单金额最大值' : 'max'})
x2=gb['上期还款金额'].agg({'放款后该用户该银行上月还款金额总计' : 'sum','放款后该用户该银行还款金额最大值' : 'max'})
x3=gb['消费笔数'].agg({'用户放款后消费笔数最大值' : 'max'})
x4=gb['循环利息'].agg({'用户放款后循环利息最大值' : 'max'})

gb1=x1.groupby(["用户标识"],as_index=False)
gb2=x2.groupby(["用户标识"],as_index=False)
gb3=x3.groupby(["用户标识"],as_index=False)
gb4=x4.groupby(["用户标识"],as_index=False)

x11=gb1['放款后该用户该银行上月账单金额总计'].agg({'放款后该用户账单金额汇总(去重)' : 'sum','放款后该用户账单数(去重)' : 'count'})
x12=gb1['放款后该用户该银行上月账单金额最大值'].agg({'放款后该用户账单金额最大值汇总(去重)' : 'sum'})

x21=gb2['放款后该用户该银行上月还款金额总计'].agg({'放款后该用户账单还款金额汇总(去重)' : 'sum'})
x22=gb2['放款后该用户该银行还款金额最大值'].agg({'放款后该用户账单还款金额最大值汇总(去重)' : 'sum'})

x31=gb3['用户放款后消费笔数最大值'].agg({'用户放款后消费笔数(去重)' : 'sum'})
x41=gb4['用户放款后循环利息最大值'].agg({'用户放款后循环利息(去重)' : 'sum'})

feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature=pd.merge(feature, x12,how='left', on = "用户标识")
feature=pd.merge(feature, x21,how='left', on = "用户标识")
feature=pd.merge(feature, x22,how='left', on = "用户标识")
feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x41,how='left', on = "用户标识")

x=pd.merge(x1, x2,how='inner')
gb3=x[(x['放款后该用户该银行上月账单金额最大值']>x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb4=x[(x['放款后该用户该银行上月账单金额最大值']==x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
gb5=x[(x['放款后该用户该银行上月账单金额最大值']<x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

x31=gb3['用户标识'].agg({'放款后账单大于还款计数(去重)' : 'count'})
x32=gb4['用户标识'].agg({'放款后账单等于还款计数(去重)' : 'count'})
x33=gb5['用户标识'].agg({'放款后账单小于还款计数(去重)' : 'count'})

feature=pd.merge(feature, x31,how='left', on = "用户标识")
feature=pd.merge(feature, x32,how='left', on = "用户标识")
feature=pd.merge(feature, x33,how='left', on = "用户标识")

feature['放款后账单汇总还款差额(去重)']=feature['放款后该用户账单金额汇总(去重)']-feature['放款后该用户账单还款金额汇总(去重)']
feature['放款后账单最大值还款差额(去重)']=feature['放款后该用户账单金额最大值汇总(去重)']-feature['放款后该用户账单还款金额最大值汇总(去重)']

#统计放款前用户消费笔数，循环利息总计
gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['消费笔数'].agg({'用户放款后消费笔数' : 'sum'})
x2=gb['循环利息'].agg({'用户放款后循环利息' : 'sum'})
x3=gb['信用卡额度'].agg({'用户放款后信用卡额度最大值' : 'max'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

#----------------------------------------总体特征统计------------------------------------------#
d=测试信用卡账单表
#爆卡指的是本期账单余额大于信用卡额度
gb=d[(d['信用卡额度']<d['本期账单余额'])].groupby(["用户标识"],as_index=False)
x1=gb['时间'].apply(lambda x:np.unique(x).size)
x=gb['时间'].agg({'爆卡次数' : 'count'})
x['爆卡次数(去重)']=x1
feature=pd.merge(feature, x,how='left', on = "用户标识")

#用户持卡数
gb=d.groupby(["用户标识"],as_index=False)
x=gb['银行标识'].apply(lambda x:np.unique(x).size)
x1=gb['银行标识'].agg({'用户银行卡账单计数' : 'count'})
x1['用户持卡数']=x
feature=pd.merge(feature,x1,how='left', on = "用户标识")

#----------------------------------------老段子的特征------------------------------------------#
d=测试信用卡账单表

#老段子的特征...神了个奇
t1=d[(d['时间']>d['放款时间'])].groupby("用户标识",as_index=False)
t2=d[(d['时间']>d['放款时间']+1)].groupby("用户标识",as_index=False)
t3=d[(d['时间']>d['放款时间']+2)].groupby("用户标识",as_index=False)

x=t1['时间'].apply(lambda x:np.unique(x).size)
x1=t1['时间'].agg({'老段子特征1' : 'count'})
x1['x1']=x

x=t2['时间'].apply(lambda x:np.unique(x).size)
x2=t2['时间'].agg({'老段子特征2' : 'count'})
x2['x2']=x

x=t3['时间'].apply(lambda x:np.unique(x).size)
x3=t3['时间'].agg({'老段子特征3' : 'count'})
x3['x3']=x

t=feature[['用户标识']]
t=pd.merge(t,x1,how='left',on = "用户标识")
t=pd.merge(t,x2,how='left',on = "用户标识")
t=pd.merge(t,x3,how='left',on = "用户标识")
t=t[['用户标识','x1','x2','x3','老段子特征1','老段子特征2','老段子特征3']]

feature=pd.merge(feature, t,how='left', on = "用户标识")

feature['老段子特征x']=(feature['x1']+1)*(feature['x2']+1)*(feature['x3']+1)
#from sklearn.preprocessing import MinMaxScaler
#feature.老段子特征x = MinMaxScaler().fit_transform(feature.老段子特征x)
feature
feature.to_csv("../feature/用户账单表初级特征测试表_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前账单金额统计,放款前账单金额为负数,放款前账单金额为零,放款前还款金额统计,放款前还款金额为负数,放款前还款金额为零,放款前账单还款差额,放款前该用户账单数(去重),...,爆卡次数(去重),用户银行卡账单计数,用户持卡数,x1,x2,x3,老段子特征1,老段子特征2,老段子特征3,老段子特征x
0,55597,68658,1040.964295,0.0,45.0,849.318939,0.0,52.0,191.645356,46.0,...,68.0,98.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,68658,964.349747,2.0,15.0,1021.231984,0.0,15.0,-56.882237,49.0,...,14.0,70.0,8.0,4.0,4.0,3.0,4.0,4.0,3.0,100.0
2,55599,68658,128.591186,1.0,1.0,147.763449,0.0,1.0,-19.172263,7.0,...,2.0,11.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,27.0
3,55600,68658,151.853337,0.0,3.0,154.238780,0.0,3.0,-2.385443,8.0,...,3.0,11.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,68658,0.000000,0.0,3.0,0.000000,0.0,3.0,0.000000,NaN,...,3.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,190.600997,0.0,0.0,176.207679,0.0,0.0,14.393318,10.0,...,6.0,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,68658,0.000000,0.0,4.0,0.000000,0.0,4.0,0.000000,NaN,...,NaN,6.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,27.0
8,55605,68658,0.000000,0.0,2.0,0.000000,0.0,2.0,0.000000,NaN,...,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,68658,715.238887,0.0,7.0,656.267082,0.0,7.0,58.971805,21.0,...,20.0,46.0,5.0,4.0,3.0,3.0,4.0,3.0,3.0,80.0


In [109]:
feature.to_csv("../feature/用户账单表初级特征测试表_20170119_A.csv",index=None,encoding="gb2312")

In [133]:
测试用户浏览行为 = pd.read_csv("../test/browse_history_test.txt",header=None,
                       names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
测试用户浏览行为['浏览时间']=测试用户浏览行为['浏览时间']//86400

测试银行流水记录=pd.read_csv("../test/bank_detail_test.txt",header=None,
                     names=['用户标识','流水时间','交易类型','交易金额','工资收入标记'])

测试银行流水记录['流水时间']=测试银行流水记录['流水时间']//86400


**银行流水记录特征提取:**  
13899 rows × 26 columns

In [140]:
#银行流水记录特征提取

#银行流水记录=pd.read_csv("../test/bank_detail_test.txt").rename(index=str,
#                                                        columns={"uid": "用户标识","timespan": "流水时间",
#                                                                 "type":"交易类型","amount":"交易金额","markup":"工资收入标记"})    
#银行流水记录['流水时间']=银行流水记录['流水时间']//86400
测试银行流水记录表 = pd.merge(测试银行流水记录,测试放款时间表,how='left',on = "用户标识")
#测试银行流水记录表#6070197行

#测试放款时间表 = pd.read_csv("../test/loan_time_test.txt",header=None,names=['用户标识','放款时间'])
#测试放款时间表['放款时间']=测试放款时间表['放款时间']//86400

#==========================================特征工程===============================================#
feature=测试放款时间表
d=测试银行流水记录表
#----------------------------------------放款前特征统计------------------------------------------#
t=d[(d['流水时间']<=d['放款时间'])]#5684742
gb1=t[(t['交易类型']==0)].groupby(["用户标识"],as_index=False)#收入统计
gb2=t[(t['交易类型']==1)].groupby(["用户标识"],as_index=False)#支出统计
gb3=t[(t['工资收入标记']==1)].groupby(["用户标识"],as_index=False)#工资收入统计
x1=gb1['交易金额'].agg({'放款前用户收入笔数' : 'count','放款前用户收入总计':'sum'})
x2=gb2['交易金额'].agg({'放款前用户支出笔数' : 'count','放款前用户支出总计':'sum'})
x3=gb3['交易金额'].agg({'放款前用户工资收入笔数' : 'count','放款前用户工资收入总计':'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

feature['放款前用户收入支出笔数差值']=feature['放款前用户收入笔数']-feature['放款前用户支出笔数']
feature['放款前用户收入支出总计差值']=feature['放款前用户收入总计']-feature['放款前用户支出总计']
feature['放款前用户非工资收入笔数']=feature['放款前用户收入笔数']-feature['放款前用户工资收入笔数']
feature['放款前用户非工资收入总计']=feature['放款前用户收入总计']-feature['放款前用户工资收入总计']
feature['放款前工资收入笔数乘以差值']=feature['放款前用户工资收入笔数']*feature['放款前用户收入支出笔数差值']
feature['放款前工资收入总计乘以差值']=feature['放款前用户工资收入总计']*feature['放款前用户收入支出总计差值']

#----------------------------------------放款后特征统计------------------------------------------#
t=d[(d['流水时间']>d['放款时间'])]#5684742
gb1=t[(t['交易类型']==0)].groupby(["用户标识"],as_index=False)#收入统计
gb2=t[(t['交易类型']==1)].groupby(["用户标识"],as_index=False)#支出统计
gb3=t[(t['工资收入标记']==1)].groupby(["用户标识"],as_index=False)#工资收入统计
x1=gb1['交易金额'].agg({'放款后用户收入笔数' : 'count','放款后用户收入总计':'sum'})
x2=gb2['交易金额'].agg({'放款后用户支出笔数' : 'count','放款后用户支出总计':'sum'})
x3=gb3['交易金额'].agg({'放款后用户工资收入笔数' : 'count','放款后用户工资收入总计':'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")

feature['放款后用户收入支出笔数差值']=feature['放款后用户收入笔数']-feature['放款后用户支出笔数']
feature['放款后用户收入支出总计差值']=feature['放款后用户收入总计']-feature['放款后用户支出总计']
feature['放款后用户非工资收入笔数']=feature['放款后用户收入笔数']-feature['放款后用户工资收入笔数']
feature['放款后用户非工资收入总计']=feature['放款后用户收入总计']-feature['放款后用户工资收入总计']
feature['放款后工资收入笔数乘以差值']=feature['放款后用户工资收入笔数']*feature['放款后用户收入支出笔数差值']
feature['放款后工资收入总计乘以差值']=feature['放款后用户工资收入总计']*feature['放款后用户收入支出总计差值']
feature
feature.to_csv("../feature/用户银行流水记录测试表_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前用户收入笔数,放款前用户收入总计,放款前用户支出总计,放款前用户支出笔数,放款前用户工资收入总计,放款前用户工资收入笔数,放款前用户收入支出笔数差值,放款前用户收入支出总计差值,...,放款后用户支出笔数,放款后用户支出总计,放款后用户工资收入笔数,放款后用户工资收入总计,放款后用户收入支出笔数差值,放款后用户收入支出总计差值,放款后用户非工资收入笔数,放款后用户非工资收入总计,放款后工资收入笔数乘以差值,放款后工资收入总计乘以差值
0,55597,68658,8.0,99.208314,114.742022,10.0,NaN,NaN,-2.0,-15.533708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55598,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55599,68658,218.0,2993.204413,11070.961352,886.0,NaN,NaN,-668.0,-8077.756939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55600,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,55602,68658,131.0,1812.100258,3620.579250,317.0,NaN,NaN,-186.0,-1808.478992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,55603,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,55604,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,55605,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,55606,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
feature.to_csv("../feature/用户银行流水记录测试表_20170119_A.csv",index=None,encoding="gb2312")

**用户浏览行为：**  
13899 rows × 40 columns

In [136]:
feature=测试放款时间表
d= pd.merge(测试用户浏览行为, 测试放款时间表,how='left', on = "用户标识")

#----------------------------------------放款前特征统计------------------------------------------#
#统计放款前用户浏览子行为总数以及浏览行为数据总和
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览行为数据'].agg({'放款前浏览行为数据sum' : 'sum','放款前浏览行为数据max' : 'max','放款前浏览行为数据mean' : 'mean'
                    ,'放款前浏览行为数据min' : 'min','放款前浏览行为数据std' : 'std','放款前浏览行为数据var' : 'var'})
xx=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
x2=gb['浏览子行为编号'].agg({'放款前浏览子行为编号count' : 'count'})
x2['放款前浏览子行为编号计数（去重）']=xx

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")

#统计放款前用户浏览子行为个类别统计信息
d=pd.get_dummies(d,columns=d[['浏览子行为编号']])#22919547 rows × 14 columns
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览子行为编号_1'].agg({'放款前浏览子行为编号_1' : 'sum'})
x2=gb['浏览子行为编号_2'].agg({'放款前浏览子行为编号_2' : 'sum'})
x3=gb['浏览子行为编号_3'].agg({'放款前浏览子行为编号_3' : 'sum'})
x4=gb['浏览子行为编号_4'].agg({'放款前浏览子行为编号_4' : 'sum'})
x5=gb['浏览子行为编号_5'].agg({'放款前浏览子行为编号_5' : 'sum'})
x6=gb['浏览子行为编号_6'].agg({'放款前浏览子行为编号_6' : 'sum'})
x7=gb['浏览子行为编号_7'].agg({'放款前浏览子行为编号_7' : 'sum'})
x8=gb['浏览子行为编号_8'].agg({'放款前浏览子行为编号_8' : 'sum'})
x9=gb['浏览子行为编号_9'].agg({'放款前浏览子行为编号_9' : 'sum'})
x10=gb['浏览子行为编号_10'].agg({'放款前浏览子行为编号_10' : 'sum'})
x11=gb['浏览子行为编号_11'].agg({'放款前浏览子行为编号_11' : 'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")
feature=pd.merge(feature, x4,how='left', on = "用户标识")
feature=pd.merge(feature, x5,how='left', on = "用户标识")
feature=pd.merge(feature, x6,how='left', on = "用户标识")
feature=pd.merge(feature, x7,how='left', on = "用户标识")
feature=pd.merge(feature, x8,how='left', on = "用户标识")
feature=pd.merge(feature, x9,how='left', on = "用户标识")
feature=pd.merge(feature, x10,how='left', on = "用户标识")
feature=pd.merge(feature, x11,how='left', on = "用户标识")

d= pd.merge(测试用户浏览行为, 测试放款时间表,how='left', on = "用户标识")#
#----------------------------------------放款后特征统计------------------------------------------#
#统计放款前用户浏览子行为总数以及浏览行为数据总和
gb=d[(d['浏览时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览行为数据'].agg({'放款后浏览行为数据sum' : 'sum','放款后浏览行为数据max' : 'max','放款后浏览行为数据mean' : 'mean'
                     ,'放款后浏览行为数据min' : 'min','放款后浏览行为数据std' : 'std','放款后浏览行为数据var' : 'var'})
xx=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
x2=gb['浏览子行为编号'].agg({'放款后浏览子行为编号count' : 'count'})
x2['放款后浏览子行为编号计数（去重）']=xx

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")

#统计放款前用户浏览子行为个类别统计信息
d=pd.get_dummies(d,columns=d[['浏览子行为编号']])#22919547 rows × 14 columns
gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
x1=gb['浏览子行为编号_1'].agg({'放款后浏览子行为编号_1' : 'sum'})
x2=gb['浏览子行为编号_2'].agg({'放款后浏览子行为编号_2' : 'sum'})
x3=gb['浏览子行为编号_3'].agg({'放款后浏览子行为编号_3' : 'sum'})
x4=gb['浏览子行为编号_4'].agg({'放款后浏览子行为编号_4' : 'sum'})
x5=gb['浏览子行为编号_5'].agg({'放款后浏览子行为编号_5' : 'sum'})
x6=gb['浏览子行为编号_6'].agg({'放款后浏览子行为编号_6' : 'sum'})
x7=gb['浏览子行为编号_7'].agg({'放款后浏览子行为编号_7' : 'sum'})
x8=gb['浏览子行为编号_8'].agg({'放款后浏览子行为编号_8' : 'sum'})
x9=gb['浏览子行为编号_9'].agg({'放款后浏览子行为编号_9' : 'sum'})
x10=gb['浏览子行为编号_10'].agg({'放款后浏览子行为编号_10' : 'sum'})
x11=gb['浏览子行为编号_11'].agg({'放款后浏览子行为编号_11' : 'sum'})

feature=pd.merge(feature, x1,how='left', on = "用户标识")
feature=pd.merge(feature, x2,how='left', on = "用户标识")
feature=pd.merge(feature, x3,how='left', on = "用户标识")
feature=pd.merge(feature, x4,how='left', on = "用户标识")
feature=pd.merge(feature, x5,how='left', on = "用户标识")
feature=pd.merge(feature, x6,how='left', on = "用户标识")
feature=pd.merge(feature, x7,how='left', on = "用户标识")
feature=pd.merge(feature, x8,how='left', on = "用户标识")
feature=pd.merge(feature, x9,how='left', on = "用户标识")
feature=pd.merge(feature, x10,how='left', on = "用户标识")
feature=pd.merge(feature, x11,how='left', on = "用户标识")
feature
feature.to_csv("../feature/用户浏览行为_20170119_A.csv",index=None,encoding="gb2312")

,用户标识,放款时间,放款前浏览行为数据var,放款前浏览行为数据mean,放款前浏览行为数据std,放款前浏览行为数据min,放款前浏览行为数据max,放款前浏览行为数据sum,放款前浏览子行为编号count,放款前浏览子行为编号计数（去重）,...,放款后浏览子行为编号_2,放款后浏览子行为编号_3,放款后浏览子行为编号_4,放款后浏览子行为编号_5,放款后浏览子行为编号_6,放款后浏览子行为编号_7,放款后浏览子行为编号_8,放款后浏览子行为编号_9,放款后浏览子行为编号_10,放款后浏览子行为编号_11
0,55597,68658,2503.160574,113.937500,50.031596,35.0,200.0,43752.0,384.0,6.0,...,0.0,0.0,56.0,16.0,24.0,64.0,0.0,0.0,24.0,0.0
1,55598,68658,3573.716033,117.193750,59.780566,15.0,213.0,450024.0,3840.0,7.0,...,0.0,0.0,632.0,376.0,248.0,336.0,184.0,0.0,176.0,0.0
2,55599,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55600,68658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55601,68658,3050.158869,119.343511,55.228243,38.0,213.0,125072.0,1048.0,7.0,...,0.0,0.0,176.0,136.0,72.0,88.0,8.0,0.0,72.0,0.0
5,55602,68658,2929.986097,112.666667,54.129346,23.0,213.0,113568.0,1008.0,7.0,...,0.0,0.0,120.0,144.0,48.0,108.0,36.0,0.0,108.0,0.0
6,55603,68658,3278.076146,118.277778,57.254486,35.0,213.0,17032.0,144.0,6.0,...,0.0,0.0,24.0,32.0,8.0,16.0,0.0,0.0,8.0,0.0
7,55604,68658,2906.719792,112.604651,53.914004,35.0,190.0,48420.0,430.0,6.0,...,0.0,0.0,60.0,30.0,30.0,60.0,0.0,0.0,30.0,0.0
8,55605,68658,3033.478949,120.687500,55.077027,38.0,201.0,81102.0,672.0,7.0,...,0.0,0.0,84.0,42.0,42.0,84.0,42.0,0.0,42.0,0.0
9,55606,68658,2473.220713,94.545455,49.731486,20.0,173.0,37440.0,396.0,4.0,...,0.0,0.0,72.0,0.0,36.0,72.0,0.0,0.0,0.0,0.0


In [137]:
feature.to_csv("../feature/用户浏览行为测试表_20170119_A.csv",index=None,encoding="gb2312")